<a href="https://colab.research.google.com/github/Amarantine-xiv/Another-FF14-Combat-Sim/blob/main/CoreSimulator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Copyright 2024 A. Falena

Licensed under the Apache License, Version 2.0 (the "License");
you may not use this file except in compliance with the License.
You may obtain a copy of the License at

    https://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software
distributed under the License is distributed on an "AS IS" BASIS,
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
See the License for the specific language governing permissions and
limitations under the License.

In [39]:
VERSION=0.06

In [40]:
#@title Imports

import numpy as np
import matplotlib.pyplot as plt
import time
import copy
import math
from dataclasses import dataclass
import heapq
from enum import Enum
from collections import namedtuple
from typing import Any, Union

In [41]:
#@title Game Constants, Util fns
@dataclass(frozen=True)
class GameConsts:
  GCD_RECAST_TIME: float = 2500
  DOT_TICK_INTERVAL = 3000
  DAMAGE_SNAPSHOT_TIME_BEFORE_CAST_FINISHES = 500
  DH_DAMAGE_MULT_BONUS = 0.25
  # level mods from akhmorning
  LEVEL_DIVS = {90: 1900}
  LEVEL_MAINS= {90: 390}
  LEVEL_SUBS= {90: 400}

class Utils:
  def transform_time_to_prio(t):
    return int(1000*t)

  def truncate_to_digit(x, digit_to_round_to):
    modifier = math.pow(10,digit_to_round_to-1)
    return int(modifier*np.floor(x/modifier))

In [42]:
class JobClassFns:

  # See https://www.akhmorning.com/allagan-studies/modifiers/ . Each class' class num is its main attribute #.
  JOB_MODS = {'SCH': 115, 'SGE': 115, 'WHM': 115, 'AST': 115, 'WAR': 105, 'SAM': 112, 'RPR': 115, 'DRK': 105, 'PLD': 100, 'GNB': 100}

  @staticmethod
  def compute_trait_damage_mult(job_class):
    job_to_trait_damage_mult = {'SCH': 1.30,
                                'SGE': 1.30,
                                'WHM': 1.30,
                                'AST': 1.30}
    if job_class in job_to_trait_damage_mult:
      return job_to_trait_damage_mult[job_class]
    return 1

  @staticmethod
  def isCaster(job_class):
    return job_class in ['RDM', 'BLM', 'BLU', 'SMN']

  @staticmethod
  def isHealer(job_class):
    return job_class in ['SCH', 'WHM', 'AST', 'SGE']

  @staticmethod
  def isTank(job_class):
    return job_class in ['GNB', 'WAR', 'PLD', 'DRK']

#@title StatFns, Stats, ProcessedStats, JobClassFns, DamageClass, ForedCritOrDH
class StatFns:
  @staticmethod
  def get_time_using_speed_stat(t_ms, speed_stat, level=90):
    level_sub = GameConsts.LEVEL_SUBS[level]
    level_div = GameConsts.LEVEL_DIVS[level]

    tmp = np.ceil(130*(level_sub-speed_stat)/level_div)
    tmp2 = t_ms*(1000+tmp)/10000
    tmp3 = np.floor(tmp2)/100
    return int(1000*tmp3)

  @staticmethod
  def get_crit_stats(crit_stat, level=90):
    level_sub = GameConsts.LEVEL_SUBS[level]
    level_div = GameConsts.LEVEL_DIVS[level]

    crit_rate = (np.floor(200*(crit_stat-level_sub)/level_div)+50)/1000
    crit_bonus = (np.floor(200*(crit_stat-level_sub)/level_div)+400)/1000
    return crit_rate, crit_bonus

  @staticmethod
  def get_dh_rate(dh_stat, level=90):
    level_sub = GameConsts.LEVEL_SUBS[level]
    level_div = GameConsts.LEVEL_DIVS[level]
    return np.floor(550*(dh_stat-level_sub)/level_div)/1000

  @staticmethod
  # from Hint's repo, https://github.com/hintxiv/reassemble
  def fWD(wd, job_mod, level=90):
    level_main = GameConsts.LEVEL_MAINS[level]
    return np.floor(level_main*job_mod/1000 + wd)

  @staticmethod
  # from Hint's repo, https://github.com/hintxiv/reassemble
  def fSpd(speed_stat, level=90):
    level_sub = GameConsts.LEVEL_SUBS[level]
    level_div = GameConsts.LEVEL_DIVS[level]
    return np.floor(130 * (speed_stat - level_sub) / level_div + 1000)

  @staticmethod
  def fTnc(tenacity, level=90):
    level_sub = GameConsts.LEVEL_SUBS[level]
    level_div = GameConsts.LEVEL_DIVS[level]
    return (np.floor(100*(tenacity-level_sub)/level_div) + 1000)

  @staticmethod
  # from Hint's repo, https://github.com/hintxiv/reassemble
  def fAP(main_stat, is_tank= False, level=90):
    level_main = GameConsts.LEVEL_MAINS[level]
    if is_tank:
      return np.floor(156*(main_stat-level_main)/level_main) + 100
    return np.floor(195*(main_stat-level_main)/level_main) + 100

  @staticmethod
  # from Hint's repo, https://github.com/hintxiv/reassemble
  def fDet(det_stat, level=90):
    level_main = GameConsts.LEVEL_MAINS[level]
    level_div = GameConsts.LEVEL_DIVS[level]

    return np.floor(140*(det_stat-level_main)/level_div) + 1000

  #Used for auto dh
  @staticmethod
  def fDetDH(det_stat, dh_stat, level= 90):
    level_sub = GameConsts.LEVEL_SUBS[level]
    level_div = GameConsts.LEVEL_DIVS[level]
    return StatFns.fDet(det_stat) + np.floor(140*(dh_stat-level_sub)/level_div)

  @staticmethod
  def fAuto(wd, weapon_delay, job_mod, level= 90):
    level_main = GameConsts.LEVEL_MAINS[level]
    return np.floor(np.floor(level_main * job_mod / 1000 + wd) * (weapon_delay / 3))

@dataclass(frozen=True)
class Stats():
  wd: float
  weapon_delay: float
  main_stat: float
  det_stat: float
  dh_stat: float
  crit_stat: float
  speed_stat: float
  job_class: str
  tenacity: float = None
  num_roles_in_party: float = 5
  healer_or_caster_strength: float= None
  level: int = 90

@dataclass(frozen=True)
class ProcessedStats():
  stats: Stats

  def __init__(self, stats, job_class_fns=JobClassFns):
    object.__setattr__(self, "stats", stats)
    crit_rate, crit_bonus = StatFns.get_crit_stats(stats.crit_stat)
    object.__setattr__(self, "crit_rate", crit_rate)
    object.__setattr__(self, "crit_bonus", crit_bonus)
    object.__setattr__(self, "det_bonus", StatFns.fDet(stats.det_stat))
    object.__setattr__(self, "dh_rate", StatFns.get_dh_rate(stats.dh_stat))
    object.__setattr__(self, 'job_mod', job_class_fns.JOB_MODS[stats.job_class])
    object.__setattr__(self, 'trait_damage_mult', job_class_fns.compute_trait_damage_mult(stats.job_class))

  def  __str__(self):
    res = 'Crit rate: {}\n'.format(self.crit_rate)
    res += 'Crit bonus: {}\n'.format(self.crit_bonus)
    res += 'Det bonus: {}\n'.format(self.det_bonus)
    res += 'DH Rate: {}\n'.format(self.dh_rate)
    res += 'Job mod: {}\n'.format(self.job_mod)
    res += 'Trait damage mult: {}\n'.format(self.trait_damage_mult)
    return res

class DamageClass(Enum):
    UNKNOWN = 0
    DIRECT = 1
    DOT = 2
    AUTO = 3
    PET = 4

class ForcedCritOrDH(Enum):
  UNKNOWN = 0
  DEFAULT = 1
  FORCE_NO = 2
  FORCE_YES = 3


In [43]:
#@title DamageSpec, TimingSpec, StatusEffectSpec, FollowUp

@dataclass(frozen=True)
class DamageSpec:
  potency: float = None
  damage_class: DamageClass = DamageClass.DIRECT
  guaranteed_crit: ForcedCritOrDH = ForcedCritOrDH.DEFAULT
  guaranteed_dh: ForcedCritOrDH = ForcedCritOrDH.DEFAULT
  # Used for overriding class trait damage multipliers where necessary. Set to
  # 1 if this damage instance should not use class traits for computing damage.
  trait_damage_mult_override: float = None
  pet_job_mod_override: float = None #only used if the damage_class is DamageClass.PET
  status_effect_denylist: tuple = ()

  def  __str__(self):
    res = '   potency:{}\n'.format(self.potency)
    res += '   damage_class: {}\n'.format(self.damage_class)
    res += '   trait_damage_mult_override:{}\n'.format(self.trait_damage_mult_override)
    res += '   guaranteed_crit:{}\n'.format(self.guaranteed_crit)
    res += '   guaranteed_dh:{}'.format(self.guaranteed_dh)
    res += '   trait_damage_mult_override:{}\n'.format(self.trait_damage_mult_override)
    res += '   pet_job_mod_override:{}'.format(self.pet_job_mod_override)
    res += '   status_effect_denylist:{}\n'.format(self.status_effect_denylist)
    return res

  def __post_init__(self):
    assert isinstance(self.status_effect_denylist, tuple), "status_effect_denylist must be encoded as a tuple for immutability. Did you encode it as a single string by accident, when it should be a tuple of length 1?"

@dataclass(frozen=True)
class TimingSpec:
  base_cast_time: int
  is_GCD: bool
  gcd_base_recast_time: int = None #after you use this skill, how long until you can use a gcd?
  animation_lock: int = 650 #does not take into account ping
  application_delay: int = 0 #how long after the cast finishes does the skill get applied
  affected_by_speed_stat: bool = True
  affected_by_cast_time_buffs: bool = True

  def __error_check(self):
    #This is such a common error, that we need to defend against it.
    assert isinstance(self.base_cast_time, int), "base_cast_time should be an int in ms. Did you put it in seconds?"
    assert isinstance(self.gcd_base_recast_time, int), "gcd_base_recast_time should be an int in ms. Did you put it in seconds?"
    assert isinstance(self.application_delay, int), "application_delay should be an int in ms. Did you put it in seconds?"
    assert isinstance(self.animation_lock, int), "animation_lock should be an int in ms. Did you put it in seconds?"

  def __post_init__(self):
    if self.gcd_base_recast_time is None:
      if self.is_GCD:
        # assume a default of 2500ms recast time for all gcds, unless otherwise stated
        object.__setattr__(self, 'gcd_base_recast_time', GameConsts.GCD_RECAST_TIME)
      else:
        # By default, non-gcds don't affect gcd_base_recast_time. If this is not
        # true for this skill, then set gcd_base_recast_time explicitly.
        object.__setattr__(self, 'gcd_base_recast_time', 0)
    self.__error_check()

  def __str__(self):
    res = "  Base cast time: {}\n".format(self.base_cast_time)
    res += "  is_GCD: {}\n".format(self.is_GCD)
    res += "  gcd_base_recast_time: {}\n".format(self.gcd_base_recast_time)
    res += "  animation_lock: {}\n".format(self.animation_lock)
    res += "  application_delay: {}\n".format(self.application_delay)
    res += "  affected_by_speed_stat: {}\n".format(self.affected_by_speed_stat)
    res += "  affected_by_cast_time_buffs: {}\n".format(self.affected_by_cast_time_buffs)
    return res

@dataclass(frozen=True)
class StatusEffectSpec:
  duration: int
  max_duration: int = None
  crit_rate_add: float= 0
  dh_rate_add: float= 0
  damage_mult: float= 1
  main_stat_add: float= 0
  #auto-attack time is ~= weapon_delay*(1-auto_attack_delay_reduction)
  auto_attack_delay_reduction: float = 0
  #this applies to cast/gcd timer (but NOT necessarily recast time).
  # cast/gcd time is ~= cast_time*(1-haste_time_reduction)
  haste_time_reduction: float = 0
  flat_cast_time_reduction: float = 0

  def __post_init__(self):
    if self.max_duration is None:
      object.__setattr__(self, 'max_duration', self.duration)
    assert isinstance(self.duration, int), "duration should be an int in ms. Did you put it in seconds?"
    assert isinstance(self.max_duration, int), "max_duration should be an int in ms. Did you put it in seconds?"
    assert isinstance(self.flat_cast_time_reduction, int), "flat_cast_time_reduction should be an int in ms. Did you put it in seconds?"

  def  __str__(self):
    res = '   duration:{}\n'.format(self.duration)
    res += '   max_duration: {}\n'.format(self.max_duration)
    res += '   crit_rate_add: {}\n'.format(self.crit_rate_add)
    res += '   dh_rate_add: {}\n'.format(self.dh_rate_add)
    res += '   damage_mult: {}\n'.format(self.damage_mult)
    res += '   main_stat_add: {}'.format(self.main_stat_add)
    res += '   auto_attack_delay_reduction: {}'.format(self.auto_attack_delay_reduction)
    res += '   haste_time_reduction: {}'.format(self.haste_time_reduction)
    res += '   flat_cast_time_reduction: {}'.format(self.flat_cast_time_reduction)
    return res

@dataclass(frozen=True)
class FollowUp:
  skill: Any #This should be Skill, but we don't want to bother with forward declaring classes.
  delay_after_parent_application: int
  snapshot_buffs_with_parent: bool = True
  snapshot_debuffs_with_parent: bool = True
  dot_duration: int = None
  dot_tick_interval: int = 3000 #only used if dot_duration is not None

  def __post_init__(self):
    assert self.delay_after_parent_application is None or isinstance(self.delay_after_parent_application, int), "Delay after parent application should an int to represent ms. Did you put it in seconds?"
    assert self.dot_duration is None or isinstance(self.dot_duration, int), "Dot duration should either be none, or an int to represent ms. Did you put it in seconds?"
    assert self.dot_tick_interval is None or isinstance(self.dot_tick_interval, int), "Dot tick interval should either be none, or an int to represent ms. Did you put it in seconds?"
    assert self.delay_after_parent_application >= 0, "Follow up skills must have delay_after_parent_application >= 0 (must occur AFTER the parent)"

  def __hash__(self):
    return hash(self.skill.name)

In [44]:
#@title Skill, SkillModifier, SkillLibrary

@dataclass(frozen=True, order=True)
class SkillModifier:
  guaranteed_crit: ForcedCritOrDH = ForcedCritOrDH.DEFAULT
  guaranteed_dh: ForcedCritOrDH = ForcedCritOrDH.DEFAULT
  # with_condition is used to modify skills that changer under different
  # conditions (eg: "Combo, positional)". These are used as a into
  # each relevant field of Skill.
  with_condition: str = None

@dataclass(frozen=True, order=True)
class Skill:
  # Note: All fields must be either 1) a primitive, 2) a class with the dataclass
  # decorator, or 3) defined with an appropriate __eq__ function.
  name: str
  damage_spec: Union[dict[str, DamageSpec], DamageSpec] = None
  timing_spec: Union[dict[str, TimingSpec], TimingSpec] = None
  buff_spec: Union[dict[str, StatusEffectSpec], StatusEffectSpec] = None
  debuff_spec: Union[dict[str, StatusEffectSpec], StatusEffectSpec] = None
  # __damage_spec: DamageSpec = None
  # Follow up skills will be executed in the order given. Use this fact
  # to control whether a buff applies before or after damage has gone out from
  # the skill.
  follow_up_skills: tuple[FollowUp] = ()

  def __post_init__(self):

    assert isinstance(self.follow_up_skills, tuple), "Follow up skills must be encoded as a tuple for immutability."
    for sk in self.follow_up_skills:
      assert sk.skill.timing_spec is None, "Follow up skills are not allowed to have a TimingSpec:\n{}".format(sk.skill)

  def __str__(self):
    res = "---Skill name: {}---\n".format(self.name)
    res += "TimingSpec:\n{}\n".format(str(self.timing_spec))
    res += "DamageSpec:\n{}\n".format(self.damage_spec)
    res += "Buffs:\n{}\n".format(self.buff_spec)
    res += "Follow up skills:\n{}\n".format(str(self.follow_up_skills))
    return res

  def has_damage_spec(self):
    return self.damage_spec is not None

  def get_damage_spec(self, skill_modifier):
    if self.damage_spec is None or isinstance(self.damage_spec, DamageSpec):
      return self.damage_spec

    assert skill_modifier.with_condition is not None, "Cannot use a skill that has many with_conditions, but specify no with_condition in the SkillModifier."
    return self.damage_spec[skill_modifier.with_condition]

  def __hash__(self):
    return hash(self.name)

class SkillLibrary:
  def __init__(self):
    self._skills = {}

  def get_skill(self, skill_name, job_class):
    return self._skills[job_class][skill_name]

  def add_job_class(self, job_name):
    self._skills[job_name] = {}

  def add_skill(self, skill, job_class):
    skill_name = skill.name
    if skill_name in self._skills[job_class]:
      raise RuntimeError('Duplicate skill being added to the skill library (this is probably a naming error). Job: {}, Skill name: {}'.format(job_class, skill_name))
    self._skills[job_class][skill.name] = skill

  def print_skills(self):
    for job_name in self._skills:
      for skill_name in self._skills[job_name]:
        print("Job name: {}, Skill name: {}".format(job_name, skill_name))

In [45]:
#@title StatusEffects, StatusEffectTracker
@dataclass(frozen=True)
class StatusEffects():
  crit_rate_add: float= 0
  dh_rate_add: float= 0
  damage_mult: float= 1
  main_stat_add: float= 0
  auto_attack_delay_mult: float= 1
  haste_time_mult: float= 1
  flat_cast_time_reduction: float = 0

class StatusEffectTracker():
  def __init__(self):
    self.buffs = {}
    self.debuffs = {}

  @staticmethod
  def __expire_status_effects(t, status_effects):
    se_skills = list(status_effects.keys())
    for se_skill in se_skills:
      (end_time, _) = status_effects[se_skill]
      if t > end_time:
        del status_effects[se_skill]

  def expire_status_effects(self, t):
    self.__expire_status_effects(t, self.buffs)
    self.__expire_status_effects(t, self.debuffs)

  @staticmethod
  def __add_to_status_effects(status_effects, start_time, skill, is_buff):
    status_effect_spec = skill.buff_spec if is_buff else skill.debuff_spec
    if skill not in status_effects:
      end_time = start_time + status_effect_spec.duration
      status_effects[skill] = (end_time, status_effect_spec)
    else:
      current_end_time, _ = status_effects[skill]
      time_left = max(current_end_time - start_time, 0)
      new_duration = min(status_effect_spec.max_duration, time_left + status_effect_spec.duration)
      status_effects[skill] = (start_time + new_duration, status_effect_spec)

  def add_to_status_effects(self, t, skill):
    if skill.buff_spec is not None:
      self.__add_to_status_effects(self.buffs, t, skill, is_buff=True)
    if skill.debuff_spec is not None:
      self.__add_to_status_effects(self.debuffs, t, skill, is_buff=False)

  @staticmethod
  def __compile_status_effects(status_effects, status_effect_denylist):
    crit_rate_add = 0.0
    dh_rate_add = 0.0
    damage_mult = 1.0
    main_stat_add = 0.0
    auto_attack_delay_mult = 1.0
    haste_time_mult = 1.0
    flat_cast_time_reduction = 0

    for skill in status_effects:
      (_, spec) = status_effects[skill]
      if skill.name in status_effect_denylist:
        continue
      crit_rate_add += spec.crit_rate_add
      dh_rate_add += spec.dh_rate_add
      damage_mult *= spec.damage_mult
      main_stat_add += spec.main_stat_add
      auto_attack_delay_mult *= (1-spec.auto_attack_delay_reduction)
      haste_time_mult *= (1-spec.haste_time_reduction)
      flat_cast_time_reduction += spec.flat_cast_time_reduction
    return StatusEffects(crit_rate_add=crit_rate_add,
                         dh_rate_add=dh_rate_add,
                         damage_mult=damage_mult,
                         main_stat_add=main_stat_add,
                         auto_attack_delay_mult=auto_attack_delay_mult,
                         haste_time_mult=haste_time_mult,
                         flat_cast_time_reduction=flat_cast_time_reduction)

  # TODO: to take into account certain buffs/debuffs not applying to certain skills, we can pass in the skill to be used here and do the check.
  def compile_status_effects(self, skill=Skill(name=''), skill_modifier=None):
    if skill.has_damage_spec():
      status_effect_denylist = skill.get_damage_spec(skill_modifier).status_effect_denylist
    else:
      status_effect_denylist = ()
    return (self.__compile_status_effects(self.buffs, status_effect_denylist), self.__compile_status_effects(self.debuffs, status_effect_denylist))

In [46]:
#@title SnapshotAndApplicationEvents, RotationBuilder
class SnapshotAndApplicationEvents:
  EventTimes = namedtuple('EventTimes', ['primary', 'secondary'])
  # Encapsulates a heap with data values in the format
  # (priority, EventTimes, skill, (snapshots_buffs, snapshots_debuffs)),
  # indicating whether buffs and debuffs snapshot at the given given time for
  # that skill snapshot_time.
  Event = namedtuple('Event', ['priority', 'event_times', 'skill', 'skill_modifier', 'snapshot_status'])
  def __init__(self):
    self.__q = []

  @staticmethod
  def __create_event(priority, primary_time, secondary_time, skill, skill_modifier, snapshot_status):
    event_times = SnapshotAndApplicationEvents.EventTimes(primary=primary_time, secondary=secondary_time)
    return SnapshotAndApplicationEvents.Event(priority=priority,
                                              event_times=event_times,
                                              skill=skill,
                                              skill_modifier=skill_modifier,
                                              snapshot_status=snapshot_status)

  def add(self, priority, primary_time, secondary_time, skill, skill_modifier, snapshot_status):
    # If the application and snapshot times are the same, then we can collapse them into 1 timing.
    if primary_time == secondary_time:
      secondary_time = None
      snapshot_status = [True, True]
    self.__q.append(self.__create_event(priority, primary_time, secondary_time, skill, skill_modifier, snapshot_status))

  def get_last_event_time(self):
    last_event_time = None
    for (_,event_times,_,_,_) in self.__q:
      curr_time = event_times.primary if event_times.secondary is None else event_times.secondary
      last_event_time = curr_time if last_event_time is None else max(curr_time, last_event_time)
    return last_event_time

  def clear(self):
    self.__q.clear()

  def is_empty(self):
    return len(self.__q) == 0

  def finalize(self):
    self.__q.sort(key=lambda x: x[0])
    return self

  def get_next(self):
    return heapq.heappop(self.__q)

  def get_q(self):
    res = copy.deepcopy(self.__q)
    res.sort(key=lambda x: x[0])
    return res

  def __str__(self):
    q = copy.deepcopy(self.__q)
    res = ""
    for r in q:
      res += str(r) + '\n'
    return res

class RotationBuilder:
  """ A utility class used to turn 1) a series of button (skills) pressed and optionally the specific time they were pressed, and 2) proc application times into a series of damage instances and applicable buffs/debuffs."""
  def __init__(self, stats, skill_library, snap_dots_to_server_tick_starting_at=None):
    self.__stats = stats
    #snap_dots_to_server_tick_starting_at is in SECONDS
    self.__snap_dots_to_server_tick_starting_at = snap_dots_to_server_tick_starting_at
    self._skill_library = skill_library
    self._q_timed = [] #(time, skill, skill_modifier)
    self._q_sequence = [] #(skill, skill_modifier)
    self._q_dot_skills = {} #this is a map
    self._q_snapshot_and_applications = SnapshotAndApplicationEvents()

  @staticmethod
  def _print_q(q):
    q.sort(key=lambda x: x[0])
    for (time, skill) in q:
      print('{}: {}'.format(time, skill.name))

  def add_next(self, skill_name, skill_modifier=SkillModifier(), job_class=None):
    job_class = self.__stats.job_class if job_class is None else job_class
    skill = self._skill_library.get_skill(skill_name, job_class)
    self._q_sequence.append((skill, skill_modifier))

  def add(self, t, skill_name, skill_modifier=SkillModifier(), job_class=None):
    """ Time (t) is assumed to be in seconds"""
    job_class = self.__stats.job_class if job_class is None else job_class
    skill = self._skill_library.get_skill(skill_name, job_class)
    self._q_timed.append((int(1000*t), skill, skill_modifier))

  @staticmethod
  def __follow_up_is_dot(follow_up_skill):
    return follow_up_skill.dot_duration is not None

  def __get_cast_time(self, timing_spec, curr_buffs, curr_debuffs):
    cast_time = StatFns.get_time_using_speed_stat(timing_spec.base_cast_time, self.__stats.speed_stat) if timing_spec.affected_by_speed_stat else timing_spec.base_cast_time
    cast_time = Utils.truncate_to_digit(cast_time*curr_buffs.haste_time_mult*curr_debuffs.haste_time_mult,2) if timing_spec.affected_by_cast_time_buffs else cast_time
    cast_time -= (curr_buffs.flat_cast_time_reduction + curr_debuffs.flat_cast_time_reduction)
    cast_time = max(0, cast_time)
    return cast_time

  def __process_dot_follow_up_skill(self, follow_up_dot_skill, priority_modifier, parent_snapshot_time, parent_application_time):
    if follow_up_dot_skill not in self._q_dot_skills:
      self._q_dot_skills[follow_up_dot_skill] = []
    self._q_dot_skills[follow_up_dot_skill].append((parent_snapshot_time, parent_application_time, priority_modifier))

  def __process_non_dot_follow_up_skill(self, follow_up_skill, priority_modifier, parent_snapshot_time, parent_application_time):
      skill = follow_up_skill.skill
      application_time = parent_application_time + follow_up_skill.delay_after_parent_application

      if follow_up_skill.snapshot_buffs_with_parent or follow_up_skill.snapshot_debuffs_with_parent:
        snapshot_time = parent_snapshot_time
        snapshot_status = [follow_up_skill.snapshot_buffs_with_parent, follow_up_skill.snapshot_debuffs_with_parent]
      else:
        snapshot_time = application_time
        snapshot_status = [True, True]
      priority = Utils.transform_time_to_prio(snapshot_time)
      self._q_snapshot_and_applications.add(priority+priority_modifier, snapshot_time, application_time, skill, SkillModifier(), snapshot_status)

  def _process_follow_up_skills(self, follow_up_skills, parent_snapshot_time, parent_application_time):
    for i in range(0, len(follow_up_skills)):
      # priority modifier is used to ensure follow up skills is such that it happens after its parent, and in order of follow up skills specified
      priority_modifier = i+1
      follow_up_skill = follow_up_skills[i]
      if RotationBuilder.__follow_up_is_dot(follow_up_skill):
        self.__process_dot_follow_up_skill(follow_up_skill, priority_modifier, parent_snapshot_time, parent_application_time)
      else:
        self.__process_non_dot_follow_up_skill(follow_up_skill, priority_modifier, parent_snapshot_time, parent_application_time)

  def __get_base_dot_timings(self, follow_up_dot_skill):
    app_times = []
    dot_times = self._q_dot_skills[follow_up_dot_skill]
    for (parent_snapshot_time, parent_application_time, priority_modifier) in dot_times:
      dot_max_end_time = parent_application_time + follow_up_dot_skill.dot_duration
      # dots may start ticking once the parent skill has been applied
      app_times.append((parent_application_time, dot_max_end_time, parent_snapshot_time, priority_modifier))
    return sorted(app_times, key= lambda x: x[0])

  def __get_consolidated_dot_timing(self, dot_name, base_dot_times):
    consolidated_dots = []
    for i in range(0, len(base_dot_times)):
      curr_start_time, curr_end_time, parent_snapshot_time, priority_modifier = base_dot_times[i]
      if i == len(base_dot_times) - 1:
        possible_end_time = math.inf
      else:
        # We make the current dot end here and don't merge in, because the new
        # dot may snapshot a different set of buffs/debuffs.
        possible_end_time = base_dot_times[i+1][0]
      end_time = min(curr_end_time, possible_end_time)
      consolidated_dots.append((curr_start_time, end_time, parent_snapshot_time, priority_modifier))
    return consolidated_dots

  def __process_all_dots(self, last_event_time):
    for follow_up_dot_skill in self._q_dot_skills:
      base_dot_times = self.__get_base_dot_timings(follow_up_dot_skill)
      consolidated_dot_times = self.__get_consolidated_dot_timing(follow_up_dot_skill, base_dot_times)
      dot_skill = follow_up_dot_skill.skill

      # We use the priority modifier to ensure dot skills 1) are processed after their parent, and
      # 2) a dot tick will be processed after the early dot ticks, even if they snapshot at different times.
      dot_num = 0
      for _dot_start_time, dot_end_time, parent_snapshot_time, priority_modifier in consolidated_dot_times:
        if self.__snap_dots_to_server_tick_starting_at is None:
          dot_start_time = _dot_start_time
        else:
          shave_off = (_dot_start_time - 1000*self.__snap_dots_to_server_tick_starting_at) % GameConsts.DOT_TICK_INTERVAL
          if shave_off <= 1e-6: # allow some tolerance for floating point precision
            dot_start_time = _dot_start_time
          else:
            dot_start_time = int(_dot_start_time - shave_off + GameConsts.DOT_TICK_INTERVAL)

        for application_time in range(dot_start_time, dot_end_time, follow_up_dot_skill.dot_tick_interval):
          if (last_event_time) and (application_time > last_event_time):
            continue
          priority = Utils.transform_time_to_prio(parent_snapshot_time) + (priority_modifier + dot_num)
          snapshot_status = [follow_up_dot_skill.snapshot_buffs_with_parent, follow_up_dot_skill.snapshot_debuffs_with_parent]
          self._q_snapshot_and_applications.add(priority, parent_snapshot_time, application_time, dot_skill, SkillModifier(), snapshot_status)
          dot_num += 1

  def __process_skill(self, t, skill, skill_modifier, curr_buffs=StatusEffects(), curr_debuffs=StatusEffects()):
      timing_spec = skill.timing_spec

      cast_time = self.__get_cast_time(timing_spec, curr_buffs, curr_debuffs)
      snapshot_time = t + max(0, cast_time - GameConsts.DAMAGE_SNAPSHOT_TIME_BEFORE_CAST_FINISHES)
      application_time = t + cast_time + skill.timing_spec.application_delay
      priority = Utils.transform_time_to_prio(snapshot_time)
      self._q_snapshot_and_applications.add(priority, snapshot_time, application_time, skill, skill_modifier, [True, True])

      if skill.follow_up_skills:
        self._process_follow_up_skills(skill.follow_up_skills, snapshot_time, application_time)

  def __process_q_timed(self):
    q = copy.deepcopy(self._q_timed)

    while len(q) > 0:
      status_effects_timeline = self.__assemble_speed_status_effects_timeline()
      (curr_t, skill, skill_modifier) = heapq.heappop(q)
      (curr_buffs, curr_debuffs) = self.__get_applicable_status_effects(status_effects_timeline, curr_t, skill, skill_modifier)
      self.__process_skill(curr_t, skill, skill_modifier, curr_buffs, curr_debuffs)

  def __process_q_sequence(self):
    next_gcd_time = 0
    curr_t = 0 #this represents the next time we could possibly use any skill button
    haste_buff_skills = []
    q = copy.deepcopy(self._q_sequence)

    for skill, skill_modifier in q:
      status_effects_timeline = self.__assemble_speed_status_effects_timeline()
      (curr_buffs, curr_debuffs) = self.__get_applicable_status_effects(status_effects_timeline, curr_t, skill, skill_modifier)

      timing_spec = skill.timing_spec
      if timing_spec.is_GCD:
        curr_t = max(curr_t, next_gcd_time)
        self.__process_skill(curr_t, skill, skill_modifier, curr_buffs, curr_debuffs)

        recast_time = StatFns.get_time_using_speed_stat(timing_spec.gcd_base_recast_time, self.__stats.speed_stat) if timing_spec.affected_by_speed_stat else timing_spec.gcd_base_recast_time
        recast_time = Utils.truncate_to_digit(recast_time*curr_buffs.haste_time_mult*curr_debuffs.haste_time_mult,2) if timing_spec.affected_by_cast_time_buffs else recast_time
        next_gcd_time = curr_t + recast_time

        cast_time = self.__get_cast_time(timing_spec, curr_buffs, curr_debuffs)
        curr_t += cast_time + timing_spec.animation_lock
      else:
        self.__process_skill(curr_t, skill, skill_modifier, curr_buffs, curr_debuffs)
        curr_t += timing_spec.animation_lock

  # Result: a heap encapsualted by SnapshotAndApplicationEvents. See
  # SnapshotAndApplicationEvents's documentation for what the data format is.
  def get_skill_timing(self, add_autos=False, ignore_trailing_dots=False):
    self._q_snapshot_and_applications.clear()
    self.__process_q_sequence()
    self.__process_q_timed()

    last_event_time = self._q_snapshot_and_applications.get_last_event_time() if (ignore_trailing_dots) else None
    self.__process_all_dots(last_event_time)
    if(add_autos):
      self.__add_autos(last_event_time)
    return copy.deepcopy(self._q_snapshot_and_applications)

  def __assemble_speed_status_effects_timeline(self):
    res = []
    for event in self._q_snapshot_and_applications.get_q():
      skill = event.skill
      buff_spec_has_speed = skill.buff_spec is not None and (skill.buff_spec.auto_attack_delay_reduction > 0 or skill.buff_spec.haste_time_reduction > 0 or skill.buff_spec.flat_cast_time_reduction > 0)
      debuff_spec_has_speed = skill.debuff_spec is not None and (skill.debuff_spec.auto_attack_delay_reduction > 0 or skill.debuff_spec.haste_time_reduction > 0  or skill.debuff_spec.flat_cast_time_reduction > 0)
      if not buff_spec_has_speed and not debuff_spec_has_speed:
        continue
      event_times = event.event_times
      application_time = event_times.secondary if event_times.secondary is not None else event_times.primary
      res.append((application_time, skill))
    res.sort(key=lambda x: x[0])
    return res

  @staticmethod
  def __get_applicable_status_effects(status_effects_timeline, curr_t, skill, skill_modifier):
    applicable_status_effect_events = list(filter(lambda x: x[0] <= curr_t, status_effects_timeline))
    se_tracker = StatusEffectTracker()
    for (t,skill) in applicable_status_effect_events:
      se_tracker.expire_status_effects(t)
      se_tracker.add_to_status_effects(t, skill)
    return se_tracker.compile_status_effects(skill, skill_modifier)

  def get_cast_periods(self, status_effects_timeline):
    q = copy.deepcopy(self._q_timed)
    res = []
    while len(q) > 0:
      (t, skill, skill_modifier) = heapq.heappop(q)
      (curr_buffs, curr_debuffs) = self.__get_applicable_status_effects(status_effects_timeline, t, skill, skill_modifier)
      cast_time = self.__get_cast_time(skill.timing_spec, curr_buffs, curr_debuffs)
      if cast_time > 0:
        res.append((t, t + cast_time))
    res.sort()
    return res

  @staticmethod
  def __get_next_auto_time(candidate_t, cast_periods):
    intersecting_cast_periods = list(filter(lambda x: (x[0] <= candidate_t) and (x[1] >= candidate_t), cast_periods))
    return candidate_t if len(intersecting_cast_periods) == 0 else intersecting_cast_periods[-1][1]

  # Autos, if enabled, always start at t=0.
  def __add_autos(self, last_event_time=None):
    status_effects_timeline = self.__assemble_speed_status_effects_timeline();
    cast_periods = self.get_cast_periods(status_effects_timeline)
    weapon_delay = int(1000*self.__stats.weapon_delay) #convert to ms
    auto_skill = self._skill_library.get_skill('Auto', self.__stats.job_class)
    if last_event_time is None:
      last_event_time = self._q_snapshot_and_applications.get_last_event_time()

    curr_t = 0
    while curr_t < last_event_time:
      self._q_snapshot_and_applications.add(Utils.transform_time_to_prio(curr_t), curr_t, curr_t, auto_skill, SkillModifier(), [True, True])
      (curr_buffs, curr_debuffs) = self.__get_applicable_status_effects(status_effects_timeline, curr_t, auto_skill, SkillModifier())
      candidate_t = curr_t + Utils.truncate_to_digit(weapon_delay*curr_buffs.auto_attack_delay_mult*curr_debuffs.auto_attack_delay_mult,2)
      curr_t = self.__get_next_auto_time(candidate_t, cast_periods)

  def get_stats(self):
    return self.__stats

  def print_sequential_rotation(self):
    q = copy.deepcopy(self._q_timed)
    RotationBuilder._print_q(q)

In [47]:
#@title DamageBuilder

class DamageBuilder():
  def __init__(self, skill_library):
    self.__skill_library = skill_library
    self.se = StatusEffectTracker()

  @staticmethod
  def __is_application_time(event_times):
    return event_times.secondary is None

  # output is a list, sorted by timestamp of damage instance (not necessarily in stable-sort order, according to the rotation).
  # Format of the elements of the output: (time, skill, (buffs, debuffs))
  def get_damage_instances(self, q_snapshot_and_applications: SnapshotAndApplicationEvents):
    q = [] # (current_time, skill, skill_modifier, (buffs, debuffs))
    q_snapshot_and_applications = copy.deepcopy(q_snapshot_and_applications).finalize()
    while not q_snapshot_and_applications.is_empty():
      [priority, event_times, skill, skill_modifier, snapshot_status] = q_snapshot_and_applications.get_next()
      current_time = event_times.primary
      self.se.expire_status_effects(current_time)
      (current_buffs, current_debuffs) = self.se.compile_status_effects(skill, skill_modifier)

      is_application_time = self.__is_application_time(event_times)
      if is_application_time:
        snapshot_status[0] = snapshot_status[0] if isinstance(snapshot_status[0], StatusEffects) else current_buffs
        snapshot_status[1] = snapshot_status[1] if isinstance(snapshot_status[1], StatusEffects) else current_debuffs
        if skill.has_damage_spec():
          heapq.heappush(q, (current_time, skill, skill_modifier, tuple(snapshot_status)))
      else:
        snapshot_status[0] = snapshot_status[0] if snapshot_status[0] is not True else current_buffs
        snapshot_status[1] = snapshot_status[1] if snapshot_status[1] is not True else current_debuffs
        priority_modifier = Utils.transform_time_to_prio(event_times.primary) - priority
        new_priority = Utils.transform_time_to_prio(event_times.secondary) + priority_modifier
        q_snapshot_and_applications.add(new_priority, event_times.secondary, None, skill, skill_modifier, copy.deepcopy(snapshot_status))

      # by default, we apply buffs as the last step of any skill application
      if is_application_time:
        self.se.add_to_status_effects(current_time, skill)

    q.sort(key=lambda x: x[0])
    return q


In [48]:
#@title ComputeDamageUtils
class ComputeDamageUtils():

  @staticmethod
  def get_guaranteed_dh_bonus_dmg_multiplier(status_effects):
    bonus_dh_rate_add = status_effects[0].dh_rate_add + status_effects[1].dh_rate_add
    return GameConsts.DH_DAMAGE_MULT_BONUS*bonus_dh_rate_add

  def get_guaranteed_crit_bonus_dmg_multiplier(processed_stats, status_effects):
    bonus_crit_rate_add = status_effects[0].crit_rate_add + status_effects[1].crit_rate_add
    return processed_stats.crit_bonus*bonus_crit_rate_add

  @staticmethod
  def get_guaranteed_dh_and_crit_bonus_dmg(processed_stats, skill, skill_modifier, status_effects):
    damage_spec = skill.get_damage_spec(skill_modifier)

    forced_dh = damage_spec.guaranteed_dh == ForcedCritOrDH.FORCE_YES or skill_modifier.guaranteed_dh == ForcedCritOrDH.FORCE_YES
    forced_crit = damage_spec.guaranteed_crit == ForcedCritOrDH.FORCE_YES or skill_modifier.guaranteed_crit == ForcedCritOrDH.FORCE_YES

    bonus_dh_multiplier = ComputeDamageUtils.get_guaranteed_dh_bonus_dmg_multiplier(status_effects) if forced_dh else 0
    bonus_crit_multiplier = ComputeDamageUtils.get_guaranteed_crit_bonus_dmg_multiplier(processed_stats, status_effects) if forced_crit else 0
    return (bonus_dh_multiplier, bonus_crit_multiplier)

  @staticmethod
  def compute_damage_mult(status_effects):
    return status_effects[0].damage_mult*status_effects[1].damage_mult

  @staticmethod
  def compute_crit_rates_and_bonuses(processed_stats, skill, skill_modifier, status_effects):
    damage_spec = skill.get_damage_spec(skill_modifier)
    forced_dh = damage_spec.guaranteed_dh == ForcedCritOrDH.FORCE_YES or skill_modifier.guaranteed_dh == ForcedCritOrDH.FORCE_YES
    forced_crit = damage_spec.guaranteed_crit == ForcedCritOrDH.FORCE_YES or skill_modifier.guaranteed_crit == ForcedCritOrDH.FORCE_YES

    forced_no_dh = damage_spec.guaranteed_dh == ForcedCritOrDH.FORCE_NO or skill_modifier.guaranteed_dh == ForcedCritOrDH.FORCE_NO
    forced_no_crit = damage_spec.guaranteed_crit == ForcedCritOrDH.FORCE_NO or skill_modifier.guaranteed_crit == ForcedCritOrDH.FORCE_NO

    if forced_dh and forced_no_dh:
      raise RuntimeError('Skill specifies both forced_dh and forced_no_dh; only one of these may be true. On skill: {}. On skill modifier: {}'.format(damage_spec.guaranteed_dh, skill_modifier.guaranteed_dh))
    if forced_crit and forced_no_crit:
      raise RuntimeError('Skill specifies both forced_crit and forced_no_crit; only one of these may be true. On skill: {}. On skill modifier: {}'.format(damage_spec.guaranteed_crit, skill_modifier.guaranteed_crit))

    if forced_dh:
      dh_rate = 1
    elif forced_no_dh:
      dh_rate = 0
    else:
      dh_rate = processed_stats.dh_rate + status_effects[0].dh_rate_add + status_effects[1].dh_rate_add

    crit_bonus = processed_stats.crit_bonus
    if forced_crit:
      crit_rate = 1
    elif forced_no_crit:
      crit_rate = 0
    else:
      crit_rate = processed_stats.crit_rate + status_effects[0].crit_rate_add + status_effects[1].crit_rate_add

    return (dh_rate, crit_rate, crit_bonus)

  @staticmethod
  def compute_direct_damage(skill, skill_modifier, processed_stats, status_effects, job_class_fns=JobClassFns):
    stats = processed_stats.stats
    is_tank = job_class_fns.isTank(stats.job_class)
    main_stat = stats.main_stat+status_effects[0].main_stat_add+status_effects[1].main_stat_add
    main_stat = np.floor(main_stat*(1+0.01*stats.num_roles_in_party))

    damage_spec = skill.get_damage_spec(skill_modifier)

    # from HINT
    potency = damage_spec.potency
    ap = StatFns.fAP(main_stat, is_tank)
    wd = StatFns.fWD(stats.wd, processed_stats.job_mod)

    forced_dh = damage_spec.guaranteed_dh == ForcedCritOrDH.FORCE_YES or skill_modifier.guaranteed_dh == ForcedCritOrDH.FORCE_YES
    det_dh = StatFns.fDetDH(processed_stats.stats.det_stat, processed_stats.stats.dh_stat) if forced_dh else processed_stats.det_bonus

    # TODO: move to readme or somewhere better than this.
    # 1) main stat changed by pots/party buffs (# of roles)
    # 2) potency and fAP are taken into account
    # 3) det (and potentially dh if autocrit) is taken into account
    # 4) tnc is taken into account, if applicable (tanks)
    # 5) sps/sks is taken into account, if applicable (eg, dots)
    # 6) weapon damage and weapon delay are taken account, if applicable
    # 7) apply the +1 for dots
    # 8) apply guaranteed crit bonus using crit rate up, if applicable
    # 9) apply guaranteed direct hit bonus dh up, if applicable
    # 10) apply +-5% damage variance (on ALL sources of damage, direct, auto, dot)
    # 11) crit/direct hit multipliers are taken into account, if applicable. NO FLOORING IS DONE HERE (but i can if you think i should)
    # 12) traits are taken into account, if applicable
    # 13) each straight up damage buff/enemy increased damage taken is taken into account multiplicatively. NO FLOORING IS DONE BETWEEN APPLYING BUFFS

    base_damage = np.floor(potency*ap/100)/100
    base_damage = np.floor(base_damage*det_dh/10)/100
    if is_tank:
      base_damage = np.floor(base_damage*StatFns.fTnc(stats.tenacity)/1000)
    base_damage = np.floor(base_damage*wd)

    bonus_damage_multipliers_from_guaranteeds = ComputeDamageUtils.get_guaranteed_dh_and_crit_bonus_dmg(processed_stats, skill, skill_modifier, status_effects)
    base_damage += np.floor(base_damage*bonus_damage_multipliers_from_guaranteeds[0]) + np.floor(base_damage*bonus_damage_multipliers_from_guaranteeds[1])

    return base_damage

  @staticmethod
  def compute_dot_damage(skill, skill_modifier, processed_stats, status_effects, job_class_fns=JobClassFns):
    stats = processed_stats.stats
    is_tank = job_class_fns.isTank(stats.job_class)
    main_stat = stats.main_stat+status_effects[0].main_stat_add+status_effects[1].main_stat_add
    main_stat = np.floor(main_stat*(1+0.01*stats.num_roles_in_party))

    damage_spec = skill.get_damage_spec(skill_modifier)

    potency = damage_spec.potency
    spd = StatFns.fSpd(stats.speed_stat)
    ap = StatFns.fAP(main_stat, is_tank)
    wd = StatFns.fWD(stats.wd, processed_stats.job_mod)

    forced_dh = damage_spec.guaranteed_dh == ForcedCritOrDH.FORCE_YES or skill_modifier.guaranteed_dh == ForcedCritOrDH.FORCE_YES
    det_dh = StatFns.fDetDH(processed_stats.stats.det_stat, processed_stats.stats.dh_stat) if forced_dh else processed_stats.det_bonus

    base_damage = np.floor(potency*ap/100)/100
    base_damage = np.floor(base_damage*det_dh/10)/100
    if is_tank:
      base_damage = np.floor(base_damage*StatFns.fTnc(stats.tenacity)/1000)
    base_damage = np.floor(base_damage*spd)/1000
    base_damage = np.floor(base_damage*wd)

    bonus_damage_multipliers_from_guaranteeds = ComputeDamageUtils.get_guaranteed_dh_and_crit_bonus_dmg(processed_stats, skill, skill_modifier, status_effects)
    base_damage += np.floor(base_damage*bonus_damage_multipliers_from_guaranteeds[0]) + np.floor(base_damage*bonus_damage_multipliers_from_guaranteeds[1])

    base_damage = base_damage + 1
    return base_damage

  @staticmethod
  def compute_pet_damage(skill, skill_modifier, processed_stats, status_effects, job_class_fns=JobClassFns):
    stats = processed_stats.stats
    level_main = GameConsts.LEVEL_MAINS[stats.level]
    is_tank =  job_class_fns.isTank(stats.job_class)

    damage_spec = skill.get_damage_spec(skill_modifier)

    # # party bonus to stats does not apply to pets (eg, no 5% bonus for 1 of each type)
    job_mod_use = processed_stats.job_mod if damage_spec.pet_job_mod_override is None else damage_spec.pet_job_mod_override
    job_mod_diff = processed_stats.job_mod - job_mod_use
    main_stat_diff = np.floor(level_main*job_mod_diff/1000)
    main_stat = (stats.main_stat - main_stat_diff) + status_effects[0].main_stat_add+status_effects[1].main_stat_add

    potency = damage_spec.potency
    # For whatever reason, pets are never considered tanks, even if they come from a tank. Don't think too hard about it.
    ap = StatFns.fAP(main_stat, is_tank=False)
    wd = StatFns.fWD(stats.wd, job_mod_use)
    forced_dh = damage_spec.guaranteed_dh == ForcedCritOrDH.FORCE_YES or skill_modifier.guaranteed_dh == ForcedCritOrDH.FORCE_YES
    det_dh = StatFns.fDetDH(processed_stats.stats.det_stat, processed_stats.stats.dh_stat) if forced_dh else processed_stats.det_bonus

    base_damage = np.floor(potency*ap/100)/100
    base_damage = np.floor(base_damage*det_dh/10)/100
    if is_tank:
      base_damage = np.floor(base_damage*StatFns.fTnc(stats.tenacity)/1000)
    base_damage = np.floor(base_damage*wd)

    bonus_damage_multipliers_from_guaranteeds = ComputeDamageUtils.get_guaranteed_dh_and_crit_bonus_dmg(processed_stats, skill, skill_modifier, status_effects)
    base_damage += np.floor(base_damage*bonus_damage_multipliers_from_guaranteeds[0]) + np.floor(base_damage*bonus_damage_multipliers_from_guaranteeds[1])
    return base_damage

  @staticmethod
  def compute_auto_damage(skill, skill_modifier, processed_stats, status_effects, job_class_fns=JobClassFns):
    stats = processed_stats.stats

    if (job_class_fns.isHealer(stats.job_class) or job_class_fns.isCaster(stats.job_class)):
      # for healers/casters, the stat used to compute autos is strength, not main stat. So use that and pots and party bonus doesn't affect it
      if stats.healer_or_caster_strength is None:
        raise RuntimeError('Cannot compute auto attack damage for healer/caster unless healer_or_caster_strength stat is specified.')
      main_stat = stats.healer_or_caster_strength
    else:
      # for non-healers/casters, the stat used to compute autos is our main stat, and we get all the buffs/bonuses on it
      main_stat = stats.main_stat+status_effects[0].main_stat_add+status_effects[1].main_stat_add
      main_stat = np.floor(main_stat*(1+0.01*stats.num_roles_in_party))

    damage_spec = skill.get_damage_spec(skill_modifier)

    is_tank = job_class_fns.isTank(stats.job_class)
    potency = damage_spec.potency
    spd = StatFns.fSpd(stats.speed_stat)
    ap = StatFns.fAP(main_stat, is_tank)
    weapon_delay = stats.weapon_delay
    auto = StatFns.fAuto(stats.wd, weapon_delay, processed_stats.job_mod)

    forced_dh = damage_spec.guaranteed_dh == ForcedCritOrDH.FORCE_YES or skill_modifier.guaranteed_dh == ForcedCritOrDH.FORCE_YES
    det_dh = StatFns.fDetDH(processed_stats.stats.det_stat, processed_stats.stats.dh_stat) if forced_dh else processed_stats.det_bonus

    base_damage = np.floor(potency*ap/100)/100
    base_damage = np.floor(base_damage*det_dh/10)/100
    if is_tank:
      base_damage = np.floor(base_damage*StatFns.fTnc(stats.tenacity)/1000)
    base_damage = np.floor(base_damage*spd)/1000
    base_damage = np.floor(base_damage*auto)

    bonus_damage_multipliers_from_guaranteeds = ComputeDamageUtils.get_guaranteed_dh_and_crit_bonus_dmg(processed_stats, skill, skill_modifier, status_effects)
    base_damage += np.floor(base_damage*bonus_damage_multipliers_from_guaranteeds[0]) + np.floor(base_damage*bonus_damage_multipliers_from_guaranteeds[1])

    return base_damage

  @staticmethod
  def get_base_damage(skill, skill_modifier, processed_stats, status_effects, job_class_fns=JobClassFns):
    if not skill.has_damage_spec():
      return None

    damage_spec = skill.get_damage_spec(skill_modifier)

    if (damage_spec.damage_class == DamageClass.DIRECT):
      base_damage = ComputeDamageUtils.compute_direct_damage(skill, skill_modifier, processed_stats, status_effects, job_class_fns)
    elif (damage_spec.damage_class == DamageClass.DOT):
      base_damage = ComputeDamageUtils.compute_dot_damage(skill, skill_modifier, processed_stats, status_effects, job_class_fns)
    elif (damage_spec.damage_class == DamageClass.AUTO):
      base_damage = ComputeDamageUtils.compute_auto_damage(skill, skill_modifier, processed_stats, status_effects, job_class_fns)
    elif (damage_spec.damage_class == DamageClass.PET):
      base_damage = ComputeDamageUtils.compute_pet_damage(skill, skill_modifier, processed_stats, status_effects, job_class_fns)
    else:
      raise RuntimeError("No support damage fn for damage class: {}". format(damage_spec.damage_class))
    return base_damage


In [49]:
#@title DamageTracker

class DamageTracker():
  def __init__(self):
    self.base_damage=np.zeros((5000,1))
    self.crit_rate=np.zeros((5000,1))
    self.crit_bonus=np.zeros((5000,1))
    self.dh_rate = np.zeros((5000,1))
    self.trait_damage_mult = np.zeros((5000,1))
    self.damage_mult = np.ones((5000,1))
    self.time = np.zeros(5000)
    self.it = 0
    self.isFinalized = False

  def add_damage(self, base_damage, crit_rate, crit_bonus, dh_rate, trait_damage_mult, damage_mult, time):
    if self.isFinalized:
      raise RuntimeError('DamageTracker is finalized. Cannot add damage instances.')
    self.base_damage[self.it] = base_damage
    self.crit_rate[self.it] = crit_rate
    self.crit_bonus[self.it] = crit_bonus
    self.dh_rate[self.it] = dh_rate
    self.trait_damage_mult[self.it] = trait_damage_mult
    self.damage_mult[self.it] = damage_mult
    self.time[self.it] = time
    self.it +=1

  def finalize(self):
    self.base_damage = self.base_damage[0:self.it]
    self.crit_rate = self.crit_rate[0:self.it]
    self.crit_bonus = self.crit_bonus[0:self.it]
    self.dh_rate = self.dh_rate[0:self.it]
    self.trait_damage_mult = self.trait_damage_mult[0:self.it]
    self.damage_mult = self.damage_mult[0:self.it]
    self.time = self.time[0:self.it]
    self.isFinalized = True

  def __sample_damage_instances(self, num_samples):
    if not self.isFinalized:
      raise RuntimeError('DamageTracker must be finalized before sampling damage')
    num_damage_instances = self.base_damage.shape[0]
    total_damage = np.floor(self.base_damage*(0.95+np.random.rand(num_damage_instances,num_samples)/10))
    total_damage += np.floor(np.multiply(total_damage, np.multiply(self.crit_bonus, self.crit_rate >= np.random.rand(num_damage_instances,num_samples))))
    total_damage += np.floor(np.multiply(total_damage, GameConsts.DH_DAMAGE_MULT_BONUS*(self.dh_rate >= np.random.rand(num_damage_instances,num_samples))))
    total_damage = np.floor(np.multiply(total_damage, self.trait_damage_mult))
    total_damage = np.floor(np.multiply(total_damage, self.damage_mult))
    return total_damage

  #Return matrix of [#of damages instances x # of samples]
  def compute_damage(self, num_samples):
    return self.__sample_damage_instances(num_samples)

  def get_crit_and_dh_rates(self):
    if not self.isFinalized:
      raise RuntimeError('DamageTracker must be finalized before calling get_crit_and_dh_rates')
    return (np.squeeze(self.crit_rate), np.squeeze(self.dh_rate))

  def get_trait_damage_mult(self):
    if not self.isFinalized:
      raise RuntimeError('DamageTracker must be finalized before calling get_trait_damage_mult')
    return np.squeeze(self.trait_damage_mult)

In [50]:
#@title DamageSimulator
class DamageSimulator():
  def __init__(self, stats, dmg_instances, num_samples, verbose=False, job_class_fns=JobClassFns):
    self.__dmg_instances = copy.deepcopy(dmg_instances)
    self.__job_class_fns = job_class_fns
    # defensively sort
    self.__dmg_instances.sort(key=lambda x: x[0])
    self.__processed_stats = ProcessedStats(stats, job_class_fns=job_class_fns)
    self.__damage_tracker = DamageTracker()

    start_time = time.time()
    self.__compile_damage(num_samples)
    end_time = time.time()
    if(verbose):
      print('Simulation time took: ' + str(end_time-start_time))

  # dmg_instances is a list in format: # (current_time, skill, (buffs, debuffs))
  def __compile_damage(self, num_samples):
    for (t, skill, skill_modifier, status_effects) in self.__dmg_instances:
      base_damage = ComputeDamageUtils.get_base_damage(skill, skill_modifier, self.__processed_stats, status_effects, self.__job_class_fns)
      assert base_damage is not None, "base_damage should not be None."

      (dh_rate, crit_rate, crit_bonus) = ComputeDamageUtils.compute_crit_rates_and_bonuses(self.__processed_stats, skill, skill_modifier, status_effects)
      damage_mult = ComputeDamageUtils.compute_damage_mult(status_effects)

      damage_spec = skill.get_damage_spec(skill_modifier)
      trait_damage_mult = self.__processed_stats.trait_damage_mult if damage_spec.trait_damage_mult_override is None else damage_spec.trait_damage_mult_override
      self.__damage_tracker.add_damage(base_damage, crit_rate, crit_bonus, dh_rate, trait_damage_mult, damage_mult, t)
    self.__damage_tracker.finalize()
    damage_matrix = self.__damage_tracker.compute_damage(num_samples)

    fight_time = (self.__dmg_instances[-1][0] - self.__dmg_instances[0][0])/1000 #convert to s
    self.__damage = np.sum(damage_matrix, axis=0)
    self.__dps = self.__damage/fight_time if fight_time > 0 else math.inf
    self.__per_skill_damage_mean = np.mean(damage_matrix, axis=1)
    self.__per_skill_damage_std = np.std(damage_matrix, axis=1)

  def get_crit_and_dh_rates(self):
    return self.__damage_tracker.get_crit_and_dh_rates()

  def get_trait_damage_mult(self):
    return self.__damage_tracker.get_trait_damage_mult()

  def get_raw_damage(self):
    return self.__damage

  def get_dps(self):
    return (self.__dps)

  # Assigns the first damage instance to t=0
  def get_damage_time(self):
    t = self.__damage_tracker.time
    return t-t[0]

  def get_per_skill_damage(self):
    # make the coupling between all these fields tighter, in a single class?
    t = self.__damage_tracker.time
    first_damage_instance = t[0]
    skill_names = [x[1].name for x in self.__dmg_instances]
    res = tuple([t[i]-first_damage_instance, skill_names[i], self.__per_skill_damage_mean[i], self.__per_skill_damage_std[i]] for i in range(0, len(t)))
    return res

In [51]:
#@title DisplayUtils
class DisplayUtils:
  PERCENTILES_TO_USE = [25,50,75,95,99]
  COLOURS_TO_USE = ['g', 'b', (0.36, 0.28, 0.55), (1,0.5,0), (1,0.0,0.5)]

  @staticmethod
  def print_results(total_damage):
    percs = np.percentile(total_damage, DisplayUtils.PERCENTILES_TO_USE)
    print('Average damage: {mean:.2f}'.format(mean=np.mean(total_damage)))
    for i in range(0, len(DisplayUtils.PERCENTILES_TO_USE)):
      print('Percentile {}: {percs:.2f}'.format(DisplayUtils.PERCENTILES_TO_USE[i], percs=percs[i]))

  @staticmethod
  def display_results(total_damage, xlabel='DPS', subplot_args=[1,1,1]):
    PERCENTILES_TO_USE = [25,50,75,95,99]
    percs = np.percentile(total_damage, DisplayUtils.PERCENTILES_TO_USE)
    count, bins_count = np.histogram(total_damage, bins=500)
    pdf = count / sum(count)
    plt.subplot(subplot_args[0], subplot_args[1], subplot_args[2])
    plt.plot(bins_count[1:], pdf)
    for i in range(len(percs)):
      plt.plot([percs[i], percs[i]], [0, max(pdf)], color=DisplayUtils.COLOURS_TO_USE[i])
    plt.ylabel('Probability')
    plt.xlabel(xlabel)
    plt.show()

  @staticmethod
  def display_damage_over_time(damage, t, window_length):
    res = np.zeros((len(t),1))
    t = np.asarray(t)
    damage = np.asarray(damage)

    for i in range(0, len(t)):
      time_to_use = t[i]
      idx = np.argwhere((t >= time_to_use) & (t <= time_to_use+window_length*1000))[:,0]
      res[i] = np.sum(damage[idx])
    plt.plot(t/1000, res)
    plt.xlabel('Starting Time of Window')
    plt.ylabel('Average damage in time window')
    return res

In [52]:
#@title Convenience functions for timing specs
def get_auto_timing():
  return TimingSpec(base_cast_time=0, is_GCD=False, animation_lock=0)

def get_cast_gcd_timing_spec():
  return TimingSpec(base_cast_time=1500, is_GCD=True, animation_lock=100)

def get_instant_gcd_timing_spec():
  return TimingSpec(base_cast_time=0, is_GCD=True, animation_lock=650)

def get_instant_ogcd_timing_spec():
  return TimingSpec(base_cast_time=0, is_GCD=False, animation_lock=650)

In [53]:
#@title add_healer_skills_to_skill_library
def add_sch_skills(skill_library):
  auto_timing = get_auto_timing()
  cast_gcd_timing_spec = get_cast_gcd_timing_spec()
  instant_gcd_timing_spec = get_instant_gcd_timing_spec()
  instant_ogcd_timing_spec = get_instant_ogcd_timing_spec()

  skill_library.add_job_class('SCH')
  skill_library.add_skill(Skill(name='Auto',
                              timing_spec=auto_timing,
                              damage_spec=DamageSpec(potency=80, damage_class=DamageClass.AUTO, trait_damage_mult_override=1)),
                          'SCH')
  skill_library.add_skill(Skill(name='Broil IV',
                                timing_spec = cast_gcd_timing_spec,
                                damage_spec=DamageSpec(potency=295)),
                          'SCH')
  skill_library.add_skill(Skill(name='Ruin II',
                                timing_spec=instant_gcd_timing_spec,
                                damage_spec=DamageSpec(potency=220)),
                          'SCH')
  skill_library.add_skill(Skill(name='Energy Drain',
                                timing_spec=instant_ogcd_timing_spec,
                                damage_spec=DamageSpec(potency=100)),
                          'SCH')
  skill_library.add_skill(Skill(name='Art of War II',
                                timing_spec=instant_gcd_timing_spec,
                                damage_spec=DamageSpec(potency=180)),
                          'SCH')
  skill_library.add_skill(Skill(name='Chain Strategem',
                                timing_spec=instant_ogcd_timing_spec,
                                debuff_spec= StatusEffectSpec(duration=15000, crit_rate_add=0.10)),
                          'SCH')
  biolysis_dot_sch = Skill(name='_Biolysis dot',
                           damage_spec=DamageSpec(potency=70,
                           damage_class=DamageClass.DOT))
  skill_library.add_skill(biolysis_dot_sch, 'SCH')
  skill_library.add_skill(Skill(name='Biolysis',
                                timing_spec=instant_gcd_timing_spec,
                                follow_up_skills = (FollowUp(skill=biolysis_dot_sch,
                                                             delay_after_parent_application=0, dot_duration=30*1000,
                                                             snapshot_buffs_with_parent= True,
                                                             snapshot_debuffs_with_parent=True),)),
                          'SCH')
  return skill_library


def add_whm_skills(skill_library):
  auto_timing = get_auto_timing()
  cast_gcd_timing_spec = get_cast_gcd_timing_spec()
  instant_gcd_timing_spec = get_instant_gcd_timing_spec()
  instant_ogcd_timing_spec = get_instant_ogcd_timing_spec()

  skill_library.add_job_class('WHM')
  #WHM skills
  skill_library.add_skill(Skill(name='Auto',
                                timing_spec=auto_timing,
                                damage_spec=DamageSpec(potency=80, damage_class=DamageClass.AUTO, trait_damage_mult_override=1)),
                          'WHM')
  skill_library.add_skill(Skill(name='Glare III',
                                timing_spec = cast_gcd_timing_spec,
                                damage_spec=DamageSpec(potency=310)),
                          'WHM')
  skill_library.add_skill(Skill(name='Assize',
                                timing_spec = instant_ogcd_timing_spec,
                                damage_spec=DamageSpec(potency=400)),
                          'WHM')
  dia_dot_whm = Skill(name='_Dia dot',
                      damage_spec=DamageSpec(potency=65,
                      damage_class=DamageClass.DOT))
  skill_library.add_skill(Skill(name='Dia',
                                timing_spec=instant_gcd_timing_spec,
                                damage_spec=DamageSpec(potency=65),
                                follow_up_skills = (FollowUp(skill=dia_dot_whm,
                                                             delay_after_parent_application=0, dot_duration=30*1000,
                                                             snapshot_buffs_with_parent= True,
                                                             snapshot_debuffs_with_parent=True),)),
                          'WHM')
  skill_library.add_skill(Skill(name='Afflatus Misery',
                                timing_spec=instant_gcd_timing_spec,
                                damage_spec=DamageSpec(potency=1240)),
                          'WHM')
  skill_library.add_skill(Skill(name='Holy III',
                                timing_spec=TimingSpec(base_cast_time=2500, is_GCD=True),
                                damage_spec=DamageSpec(potency=1240)),
                          'WHM')
  skill_library.add_skill(Skill(name='Presence of Mind',
                                timing_spec=instant_ogcd_timing_spec,
                                debuff_spec= StatusEffectSpec(duration=15000, haste_time_reduction=0.25)),
                          'WHM')
  skill_library.add_skill(Skill(name='Afflatus Rapture',
                                timing_spec=instant_gcd_timing_spec),
                          'WHM')
  skill_library.add_skill(Skill(name='Afflatus Solace',
                                timing_spec=instant_gcd_timing_spec),
                          'WHM')
  return skill_library

def add_sge_skills(skill_library):
  auto_timing = get_auto_timing()
  cast_gcd_timing_spec = get_cast_gcd_timing_spec()
  instant_gcd_timing_spec = get_instant_gcd_timing_spec()
  instant_ogcd_timing_spec = get_instant_ogcd_timing_spec()

  skill_library.add_job_class('SGE')
  skill_library.add_skill(Skill(name='Auto',
                                timing_spec=auto_timing,
                                damage_spec=DamageSpec(potency=80, damage_class=DamageClass.AUTO, trait_damage_mult_override=1)),
                          'SGE')
  skill_library.add_skill(Skill(name='Dosis III',
                                timing_spec = cast_gcd_timing_spec,
                                damage_spec=DamageSpec(potency=330)),
                          'SGE')
  skill_library.add_skill(Skill(name='Phlegma III',
                                timing_spec = instant_gcd_timing_spec,
                                damage_spec=DamageSpec(potency=600)),
                          'SGE')
  skill_library.add_skill(Skill(name='Toxikon II',
                                timing_spec = instant_gcd_timing_spec,
                                damage_spec=DamageSpec(potency=330)),
                          'SGE')
  skill_library.add_skill(Skill(name='Dykrasia II',
                                timing_spec = instant_gcd_timing_spec,
                                damage_spec=DamageSpec(potency=170)),
                          'SGE')
  skill_library.add_skill(Skill(name='Pneuma',
                                timing_spec = cast_gcd_timing_spec,
                                damage_spec=DamageSpec(potency=330)),
                          'SGE')
  skill_library.add_skill(Skill(name='Eukrasia',
                                timing_spec = TimingSpec(base_cast_time=1, is_GCD=True, affected_by_speed_stat=False, affected_by_cast_time_buffs=False)),
                          'SGE')
  dot_sge = Skill(name='_Eukrasian Dosis III dot',
                  damage_spec=DamageSpec(potency=75,
                  damage_class=DamageClass.DOT))
  skill_library.add_skill(Skill(name='Eukrasian Dosis III',
                                timing_spec=instant_gcd_timing_spec,
                                follow_up_skills = (FollowUp(skill=dot_sge,
                                                             delay_after_parent_application=0, dot_duration=30*1000,
                                                             snapshot_buffs_with_parent= True,
                                                             snapshot_debuffs_with_parent=True),)),
                          'SGE')
  return skill_library

def add_ast_skills(skill_library):
  auto_timing = get_auto_timing()
  cast_gcd_timing_spec = get_cast_gcd_timing_spec()
  instant_gcd_timing_spec = get_instant_gcd_timing_spec()
  instant_ogcd_timing_spec = get_instant_ogcd_timing_spec()

  skill_library.add_job_class('AST')
  skill_library.add_skill(Skill(name='Auto',
                              timing_spec=auto_timing,
                              damage_spec=DamageSpec(potency=80, damage_class=DamageClass.AUTO, trait_damage_mult_override=1)),
                          'AST')
  skill_library.add_skill(Skill(name='Divination',
                                timing_spec=instant_ogcd_timing_spec,
                                buff_spec= StatusEffectSpec(duration=1500, damage_mult=1.06)),
                          'AST')
  skill_library.add_skill(Skill(name='Fall Malefic',
                                timing_spec=cast_gcd_timing_spec,
                                damage_spec=DamageSpec(potency=250)),
                          'AST')
  dot_ast = Skill(name='_Combust III dot',
                           damage_spec=DamageSpec(potency=55,
                           damage_class=DamageClass.DOT))
  skill_library.add_skill(Skill(name='Combust III',
                                timing_spec=instant_gcd_timing_spec,
                                follow_up_skills = (FollowUp(skill=dot_ast,
                                                             delay_after_parent_application=0, dot_duration=30*1000,
                                                             snapshot_buffs_with_parent= True,
                                                             snapshot_debuffs_with_parent=True),)),
                          'AST')
  skill_library.add_skill(Skill(name='Astrodyne 1',
                                timing_spec=instant_ogcd_timing_spec),
                          'AST')
  skill_library.add_skill(Skill(name='Astrodyne 2',
                                timing_spec=instant_ogcd_timing_spec,
                                buff_spec=StatusEffectSpec(duration=15000, haste_time_reduction=0.10)),
                          'AST')
  skill_library.add_skill(Skill(name='Astrodyne 3',
                                timing_spec=instant_ogcd_timing_spec,
                                buff_spec=StatusEffectSpec(duration=15000, haste_time_reduction=0.10, damage_mult=1.05)),
                          'AST')
  skill_library.add_skill(Skill(name='Astrodyne',
                                timing_spec=instant_ogcd_timing_spec,
                                buff_spec=StatusEffectSpec(duration=15000, haste_time_reduction=0.10, damage_mult=1.05)),
                          'AST')
  ast_earthly_small = Skill(name='_Earthly Star (Small)',
                            damage_spec=DamageSpec(damage_class=DamageClass.PET, potency=205))
  ast_earthly_big = Skill(name='_Earthly Star Big',
                          damage_spec=DamageSpec(damage_class=DamageClass.PET, potency=310))
  skill_library.add_skill(Skill(name='Earthly Star',
                                timing_spec=instant_gcd_timing_spec,
                                follow_up_skills = (FollowUp(skill=ast_earthly_big,
                                                             delay_after_parent_application=20*1000,
                                                             snapshot_buffs_with_parent= True,
                                                             snapshot_debuffs_with_parent=True),)),
                          'AST')
  skill_library.add_skill(Skill(name='Earthly Star (Manual)',
                                timing_spec=instant_gcd_timing_spec),
                          'AST')
  skill_library.add_skill(Skill(name='Stellar Detonation',
                                timing_spec=instant_gcd_timing_spec,
                                follow_up_skills = (FollowUp(skill=ast_earthly_big,
                                                             delay_after_parent_application=0*1000,
                                                             snapshot_buffs_with_parent= True,
                                                             snapshot_debuffs_with_parent=True),)),
                          'AST')
  skill_library.add_skill(Skill(name='Stellar Detonation (Small)',
                                timing_spec=instant_gcd_timing_spec,
                                follow_up_skills = (FollowUp(skill=ast_earthly_small,
                                                             delay_after_parent_application=0*1000,
                                                             snapshot_buffs_with_parent= True,
                                                             snapshot_debuffs_with_parent=True),)),
                          'AST')
  skill_library.add_skill(Skill(name='Lightspeed',
                                timing_spec=instant_ogcd_timing_spec,
                                buff_spec=StatusEffectSpec(duration=15000, flat_cast_time_reduction=2500)),
                          'AST')
  skill_library.add_skill(Skill(name='Gravity II',
                                timing_spec=cast_gcd_timing_spec,
                                damage_spec=DamageSpec(potency=130)),
                          'AST')
  skill_library.add_skill(Skill(name='Macrocosmos',
                                timing_spec=instant_ogcd_timing_spec,
                                damage_spec=DamageSpec(potency=250)),
                          'AST')
  skill_library.add_skill(Skill(name='Lord of Crowns',
                                timing_spec=instant_ogcd_timing_spec,
                                damage_spec=DamageSpec(potency=250)),
                          'AST')
  skill_library.add_skill(Skill(name='Card',
                                timing_spec=instant_ogcd_timing_spec,
                                buff_spec=StatusEffectSpec(duration=15000, damage_mult=1.06)),
                          'AST')
  skill_library.add_skill(Skill(name='Card Small',
                                timing_spec=instant_ogcd_timing_spec,
                                buff_spec=StatusEffectSpec(duration=15000, damage_mult=1.03)),
                          'AST')
  skill_library.add_skill(Skill(name='Draw',
                                timing_spec=instant_ogcd_timing_spec),
                          'AST')
  skill_library.add_skill(Skill(name='Redraw',
                                timing_spec=instant_ogcd_timing_spec),
                          'AST')
  skill_library.add_skill(Skill(name='Play',
                                timing_spec=instant_ogcd_timing_spec),
                          'AST')
  return skill_library

def add_healer_skills_to_skill_library(skill_library):
  skill_library = add_sch_skills(skill_library)
  skill_library = add_whm_skills(skill_library)
  skill_library = add_sge_skills(skill_library)
  skill_library = add_ast_skills(skill_library)
  return skill_library

In [54]:
#@title add_tank_skills_to_skill_library (add_war_skills, add_pld_skills, add_gnb_skills, add_drk_skills)
def add_war_skills(skill_library):
  auto_timing = get_auto_timing()
  instant_gcd_timing_spec = get_instant_gcd_timing_spec()
  instant_ogcd_timing_spec = get_instant_ogcd_timing_spec()

  skill_library.add_job_class('WAR')
  skill_library.add_skill(Skill(name='Auto',
                                timing_spec=auto_timing,
                                damage_spec=DamageSpec(potency=90, damage_class=DamageClass.AUTO, trait_damage_mult_override=1)),
                          'WAR')
  skill_library.add_skill(Skill(name='Heavy Swing',
                                timing_spec=instant_gcd_timing_spec,
                                damage_spec=DamageSpec(potency=200)),
                          'WAR')
  skill_library.add_skill(Skill(name='Maim',
                                timing_spec=instant_gcd_timing_spec,
                                damage_spec=DamageSpec(potency=300)),
                          'WAR')
  skill_library.add_skill(Skill(name='Storm\'s Path',
                                timing_spec=instant_gcd_timing_spec,
                                damage_spec=DamageSpec(potency=440)),
                          'WAR')
  skill_library.add_skill(Skill(name='Storm\'s Eye',
                                timing_spec=instant_gcd_timing_spec,
                                damage_spec=DamageSpec(potency=440),
                                buff_spec=StatusEffectSpec(duration=30000, max_duration=60000, damage_mult=1.10)),
                          'WAR')
  skill_library.add_skill(Skill(name='Upheaval',
                                timing_spec=instant_ogcd_timing_spec,
                                damage_spec=DamageSpec(potency=400)),
                          'WAR')
  skill_library.add_skill(Skill(name='Onslaught',
                                timing_spec=instant_ogcd_timing_spec,
                                damage_spec=DamageSpec(potency=150)),
                          'WAR')
  skill_library.add_skill(Skill(name='Fell Cleave',
                                timing_spec=instant_gcd_timing_spec,
                                damage_spec=DamageSpec(potency=520)),
                          'WAR')
  skill_library.add_skill(Skill(name='Primal Rend',
                                timing_spec=TimingSpec(base_cast_time=0, animation_lock=1300, is_GCD=True),
                                damage_spec=DamageSpec(potency=700)),
                          'WAR')
  skill_library.add_skill(Skill(name='Inner Chaos',
                                timing_spec=instant_gcd_timing_spec,
                                damage_spec=DamageSpec(potency=660,
                                                       guaranteed_crit=ForcedCritOrDH.FORCE_YES,
                                                       guaranteed_dh=ForcedCritOrDH.FORCE_YES)),
                          'WAR')
  skill_library.add_skill(Skill(name='Tomahawk',
                                timing_spec=instant_gcd_timing_spec,
                                damage_spec=DamageSpec(potency=150)),
                          'WAR')
  skill_library.add_skill(Skill(name='Overpower',
                                timing_spec=instant_gcd_timing_spec,
                                damage_spec=DamageSpec(potency=110)),
                          'WAR')
  skill_library.add_skill(Skill(name='Mythril Tempest',
                                timing_spec=instant_gcd_timing_spec,
                                damage_spec=DamageSpec(potency=150)),
                          'WAR')
  skill_library.add_skill(Skill(name='Orogeny',
                                timing_spec=instant_ogcd_timing_spec,
                                damage_spec=DamageSpec(potency=150)),
                          'WAR')
  skill_library.add_skill(Skill(name='Decimate',
                                timing_spec=instant_gcd_timing_spec,
                                damage_spec=DamageSpec(potency=200)),
                          'WAR')
  return skill_library

def add_gnb_skills(skill_library):
  auto_timing = get_auto_timing()
  instant_gcd_timing_spec = get_instant_gcd_timing_spec()
  instant_ogcd_timing_spec = get_instant_ogcd_timing_spec()

  skill_library.add_job_class('GNB')
  skill_library.add_skill(Skill(name='Auto',
                                timing_spec=auto_timing,
                                damage_spec=DamageSpec(potency=90, damage_class=DamageClass.AUTO, trait_damage_mult_override=1)),
                          'GNB')
  skill_library.add_skill(Skill(name='Keen Edge',
                                damage_spec=DamageSpec(potency=200),
                                timing_spec=instant_gcd_timing_spec),
                          'GNB')
  skill_library.add_skill(Skill(name='No Mercy',
                                buff_spec=StatusEffectSpec(duration=20000, damage_mult=1.20),
                                timing_spec=instant_ogcd_timing_spec),
                          'GNB')
  skill_library.add_skill(Skill(name='Brutal Shell',
                                damage_spec=DamageSpec(potency=300),
                                timing_spec=instant_gcd_timing_spec),
                          'GNB')
  skill_library.add_skill(Skill(name='Demon Slice',
                                damage_spec=DamageSpec(potency=100),
                                timing_spec=instant_gcd_timing_spec),
                          'GNB')
  skill_library.add_skill(Skill(name='Lightning Shot',
                                damage_spec=DamageSpec(potency=150),
                                timing_spec=instant_gcd_timing_spec),
                          'GNB')
  skill_library.add_skill(Skill(name='Solid Barrel',
                                damage_spec=DamageSpec(potency=360),
                                timing_spec=instant_gcd_timing_spec),
                          'GNB')
  skill_library.add_skill(Skill(name='Burst Strike',
                                damage_spec=DamageSpec(potency=380),
                                timing_spec=instant_gcd_timing_spec),
                          'GNB')
  skill_library.add_skill(Skill(name='Demon Slaughter',
                                damage_spec=DamageSpec(potency=160),
                                timing_spec=instant_gcd_timing_spec),
                          'GNB')
  sonic_break_dot_gnb = Skill(name='_Sonic Break dot',
                               damage_spec=DamageSpec(potency=60,
                               damage_class=DamageClass.DOT))
  skill_library.add_skill(sonic_break_dot_gnb, 'GNB')
  skill_library.add_skill(Skill(name='Sonic Break',
                                timing_spec=instant_gcd_timing_spec,
                                damage_spec=DamageSpec(potency=300),
                                follow_up_skills = (FollowUp(skill=sonic_break_dot_gnb,
                                                             delay_after_parent_application=0,
                                                             dot_duration=30*1000,
                                                             snapshot_buffs_with_parent= True,
                                                             snapshot_debuffs_with_parent=True),)),
                          'GNB')
  skill_library.add_skill(Skill(name='Rough Divide',
                                damage_spec=DamageSpec(potency=150),
                                timing_spec=instant_ogcd_timing_spec),
                          'GNB')
  skill_library.add_skill(Skill(name='Gnashing Fang',
                                damage_spec=DamageSpec(potency=380),
                                timing_spec=instant_gcd_timing_spec),
                          'GNB')
  skill_library.add_skill(Skill(name='Savage Claw',
                                damage_spec=DamageSpec(potency=460),
                                timing_spec=instant_gcd_timing_spec),
                          'GNB')
  skill_library.add_skill(Skill(name='Wicked Talon',
                                damage_spec=DamageSpec(potency=540),
                                timing_spec=instant_gcd_timing_spec),
                          'GNB')
  sonick_break_dot_gnb = Skill(name='_Bow Shock dot',
                               damage_spec=DamageSpec(potency=60,
                               damage_class=DamageClass.DOT))
  skill_library.add_skill(sonick_break_dot_gnb, 'GNB')
  skill_library.add_skill(Skill(name='Bow Shock',
                                timing_spec=instant_ogcd_timing_spec,
                                damage_spec=DamageSpec(potency=150),
                                follow_up_skills = (FollowUp(skill=sonick_break_dot_gnb,
                                                             delay_after_parent_application=0,
                                                             dot_duration=15*1000,
                                                             snapshot_buffs_with_parent= True,
                                                             snapshot_debuffs_with_parent=True),)),
                          'GNB')

  skill_library.add_skill(Skill(name='Jugular Rip',
                                damage_spec=DamageSpec(potency=200),
                                timing_spec=instant_ogcd_timing_spec),
                          'GNB')
  skill_library.add_skill(Skill(name='Abdomen Tear',
                                damage_spec=DamageSpec(potency=240),
                                timing_spec=instant_ogcd_timing_spec),
                          'GNB')
  skill_library.add_skill(Skill(name='Eye Gouge',
                                damage_spec=DamageSpec(potency=280),
                                timing_spec=instant_ogcd_timing_spec),
                          'GNB')
  skill_library.add_skill(Skill(name='Fated Circle',
                                damage_spec=DamageSpec(potency=300),
                                timing_spec=instant_gcd_timing_spec),
                          'GNB')
  skill_library.add_skill(Skill(name='Blasting Zone',
                                damage_spec=DamageSpec(potency=720),
                                timing_spec=instant_ogcd_timing_spec),
                          'GNB')
  skill_library.add_skill(Skill(name='Hypervelocity',
                                damage_spec=DamageSpec(potency=180),
                                timing_spec=instant_ogcd_timing_spec),
                          'GNB')
  skill_library.add_skill(Skill(name='Double Down',
                                damage_spec=DamageSpec(potency=1200),
                                timing_spec=instant_gcd_timing_spec),
                          'GNB')
  return skill_library

def add_pld_skills(skill_library):
  auto_timing = get_auto_timing()
  instant_gcd_timing_spec = get_instant_gcd_timing_spec()
  instant_ogcd_timing_spec = get_instant_ogcd_timing_spec()

  skill_library.add_job_class('PLD')
  skill_library.add_skill(Skill(name='Auto',
                                timing_spec=auto_timing,
                                damage_spec=DamageSpec(potency=90, damage_class=DamageClass.AUTO, trait_damage_mult_override=1)),
                          'PLD')
  skill_library.add_skill(Skill(name='Fast Blade',
                              damage_spec=DamageSpec(potency=200),
                              timing_spec=instant_gcd_timing_spec),
                          'PLD')
  skill_library.add_skill(Skill(name='Fight or Flight',
                                buff_spec=StatusEffectSpec(duration=20000, damage_mult=1.25),
                                timing_spec=instant_ogcd_timing_spec),
                          'PLD')
  skill_library.add_skill(Skill(name='Riot Blade',
                                damage_spec=DamageSpec(potency=300),
                                timing_spec=instant_gcd_timing_spec),
                          'PLD')
  skill_library.add_skill(Skill(name='Total Eclipse',
                                damage_spec=DamageSpec(potency=100),
                                timing_spec=instant_gcd_timing_spec),
                          'PLD')
  skill_library.add_skill(Skill(name='Shield Bash',
                                damage_spec=DamageSpec(potency=100),
                                timing_spec=instant_gcd_timing_spec),
                          'PLD')
  skill_library.add_skill(Skill(name='Shield Lob',
                                damage_spec=DamageSpec(potency=100),
                                timing_spec=instant_gcd_timing_spec),
                          'PLD')
  skill_library.add_skill(Skill(name='Prominence',
                                damage_spec=DamageSpec(potency=170),
                                timing_spec=instant_gcd_timing_spec),
                          'PLD')
  circle_of_scorn_dot_pld = Skill(name='_Circle of Scorn dot',
                                  damage_spec=DamageSpec(potency=30,
                                  damage_class=DamageClass.DOT))
  skill_library.add_skill(circle_of_scorn_dot_pld, 'PLD')
  skill_library.add_skill(Skill(name='Circle of Scorn',
                                timing_spec=instant_ogcd_timing_spec,
                                damage_spec=DamageSpec(potency=140),
                                follow_up_skills = (FollowUp(skill=circle_of_scorn_dot_pld,
                                                             delay_after_parent_application=0,
                                                             dot_duration=15*1000,
                                                             snapshot_buffs_with_parent= True,
                                                             snapshot_debuffs_with_parent=True),)),
                          'PLD')
  skill_library.add_skill(Skill(name='Goring Blade',
                                damage_spec=DamageSpec(potency=700),
                                timing_spec=instant_gcd_timing_spec),
                          'PLD')
  skill_library.add_skill(Skill(name='Royal Authority',
                                damage_spec=DamageSpec(potency=400),
                                timing_spec=instant_gcd_timing_spec),
                          'PLD')
  skill_library.add_skill(Skill(name='Holy Spirit',
                                damage_spec=DamageSpec(potency=350),
                                timing_spec=TimingSpec(base_cast_time=1500, animation_lock=100, is_GCD=True)),
                          'PLD')
  skill_library.add_skill(Skill(name='Holy Spirit (Divine Might)',
                                damage_spec=DamageSpec(potency=450),
                                timing_spec=TimingSpec(base_cast_time=0, animation_lock=100, is_GCD=True)),
                          'PLD')
  skill_library.add_skill(Skill(name='Holy Spirit (Requiescat)',
                                damage_spec=DamageSpec(potency=650),
                                timing_spec=TimingSpec(base_cast_time=0, animation_lock=100, is_GCD=True)),
                          'PLD')
  skill_library.add_skill(Skill(name='Requiescat',
                                damage_spec=DamageSpec(potency=320),
                                timing_spec=instant_ogcd_timing_spec),
                          'PLD')
  skill_library.add_skill(Skill(name='Holy Circle',
                                damage_spec=DamageSpec(potency=100),
                                timing_spec=TimingSpec(base_cast_time=1500, animation_lock=100, is_GCD=True)),
                          'PLD')
  skill_library.add_skill(Skill(name='Holy Circle (Divine Might)',
                                damage_spec=DamageSpec(potency=200),
                                timing_spec=TimingSpec(base_cast_time=0, animation_lock=100, is_GCD=True)),
                          'PLD')
  skill_library.add_skill(Skill(name='Holy Circle (Requiescat)',
                                damage_spec=DamageSpec(potency=300),
                                timing_spec=TimingSpec(base_cast_time=0, animation_lock=100, is_GCD=True)),
                          'PLD')
  skill_library.add_skill(Skill(name='Intervene',
                                damage_spec=DamageSpec(potency=150),
                                timing_spec=instant_ogcd_timing_spec),
                          'PLD')
  skill_library.add_skill(Skill(name='Atonement',
                                damage_spec=DamageSpec(potency=400),
                                timing_spec=instant_gcd_timing_spec),
                          'PLD')
  skill_library.add_skill(Skill(name='Confiteor',
                                damage_spec=DamageSpec(potency=420),
                                timing_spec=instant_gcd_timing_spec),
                          'PLD')
  skill_library.add_skill(Skill(name='Confiteor (Requiescat)',
                                damage_spec=DamageSpec(potency=920),
                                timing_spec=instant_gcd_timing_spec),
                          'PLD')
  skill_library.add_skill(Skill(name='Explacion',
                                damage_spec=DamageSpec(potency=450),
                                timing_spec=instant_ogcd_timing_spec),
                          'PLD')
  skill_library.add_skill(Skill(name='Blade of Faith',
                                damage_spec=DamageSpec(potency=220),
                                timing_spec=instant_gcd_timing_spec),
                          'PLD')
  skill_library.add_skill(Skill(name='Blade of Faith (Requiescat)',
                                damage_spec=DamageSpec(potency=720),
                                timing_spec=instant_gcd_timing_spec),
                          'PLD')
  skill_library.add_skill(Skill(name='Blade of Truth',
                                damage_spec=DamageSpec(potency=320),
                                timing_spec=instant_gcd_timing_spec),
                          'PLD')
  skill_library.add_skill(Skill(name='Blade of Truth (Requiescat)',
                                damage_spec=DamageSpec(potency=820),
                                timing_spec=instant_gcd_timing_spec),
                          'PLD')
  skill_library.add_skill(Skill(name='Blade of Valor',
                                damage_spec=DamageSpec(potency=420),
                                timing_spec=instant_gcd_timing_spec),
                          'PLD')
  skill_library.add_skill(Skill(name='Blade of Valor (Requiescat)',
                                damage_spec=DamageSpec(potency=920),
                                timing_spec=instant_gcd_timing_spec),
                          'PLD')
  return skill_library

def add_drk_skills(skill_library):
  auto_timing = get_auto_timing()
  instant_gcd_timing_spec = get_instant_gcd_timing_spec()
  instant_ogcd_timing_spec = get_instant_ogcd_timing_spec()

  skill_library.add_job_class('DRK')
  skill_library.add_skill(Skill(name='Auto',
                                timing_spec=auto_timing,
                                damage_spec=DamageSpec(potency=90, damage_class=DamageClass.AUTO, trait_damage_mult_override=1)),
                          'DRK')
  skill_library.add_skill(Skill(name='Hard Slash',
                                damage_spec=DamageSpec(potency=170),
                                timing_spec=instant_gcd_timing_spec),
                          'DRK')

  skill_library.add_skill(Skill(name='Syphon Strike',
                                damage_spec=DamageSpec(potency=260),
                                timing_spec=instant_gcd_timing_spec),
                          'DRK')
  skill_library.add_skill(Skill(name='Unleash',
                                damage_spec=DamageSpec(potency=120),
                                timing_spec=instant_gcd_timing_spec),
                          'DRK')
  skill_library.add_skill(Skill(name='Unmend',
                                damage_spec=DamageSpec(potency=150),
                                timing_spec=instant_gcd_timing_spec),
                          'DRK')
  skill_library.add_skill(Skill(name='Souleater',
                                damage_spec=DamageSpec(potency=340),
                                timing_spec=instant_gcd_timing_spec),
                          'DRK')
  skill_library.add_skill(Skill(name='Flood of Shadow',
                                damage_spec=DamageSpec(potency=160),
                                buff_spec=StatusEffectSpec(duration=30000, max_duration=60000, damage_mult=1.10),
                                timing_spec=instant_ogcd_timing_spec),
                          'DRK')
  skill_library.add_skill(Skill(name='Stalwart Soul',
                                damage_spec=DamageSpec(potency=140),
                                timing_spec=instant_gcd_timing_spec),
                          'DRK')
  skill_library.add_skill(Skill(name='Edge of Shadow',
                                damage_spec=DamageSpec(potency=460),
                                buff_spec=StatusEffectSpec(duration=30000, max_duration=60000, damage_mult=1.10),
                                timing_spec=instant_ogcd_timing_spec),
                          'DRK')
  salted_earth_dot_drk = Skill(name='_Salted Earth dot',
                                  damage_spec=DamageSpec(potency=50,
                                  damage_class=DamageClass.DOT))
  skill_library.add_skill(salted_earth_dot_drk, 'DRK')
  skill_library.add_skill(Skill(name='Salted Earth',
                                timing_spec=instant_ogcd_timing_spec,
                                follow_up_skills = (FollowUp(skill=salted_earth_dot_drk,
                                                             delay_after_parent_application=0,
                                                             dot_duration=15*1000,
                                                             snapshot_buffs_with_parent=True,
                                                             snapshot_debuffs_with_parent=False),)),
                          'DRK')
  skill_library.add_skill(Skill(name='Salt and Darkness',
                                damage_spec=DamageSpec(potency=500),
                                timing_spec=instant_ogcd_timing_spec),
                          'DRK')
  skill_library.add_skill(Skill(name='Plunge',
                                damage_spec=DamageSpec(potency=150),
                                timing_spec=instant_ogcd_timing_spec),
                          'DRK')
  skill_library.add_skill(Skill(name='Abyssal Drain',
                                damage_spec=DamageSpec(potency=240),
                                timing_spec=instant_ogcd_timing_spec),
                          'DRK')
  skill_library.add_skill(Skill(name='Carve and Spit',
                                damage_spec=DamageSpec(potency=510),
                                timing_spec=instant_ogcd_timing_spec),
                          'DRK')
  skill_library.add_skill(Skill(name='Bloodspiller',
                                damage_spec=DamageSpec(potency=500),
                                timing_spec=instant_gcd_timing_spec),
                          'DRK')
  skill_library.add_skill(Skill(name='Quietus',
                                damage_spec=DamageSpec(potency=200),
                                timing_spec=instant_gcd_timing_spec),
                          'DRK')
  skill_library.add_skill(Skill(name='Shadowbringer',
                                damage_spec=DamageSpec(potency=600),
                                timing_spec=instant_ogcd_timing_spec),
                          'DRK')
  _living_shadow=Skill(name='_Living Shadow',
                       damage_spec=DamageSpec(potency=350, damage_class=DamageClass.PET, pet_job_mod_override=100, trait_damage_mult_override=1, status_effect_denylist=('Flood of Shadow','Edge of Shadow','Dragon Sight')))
  _living_shadow_shadowbringer=Skill(name='_Living Shadow Shadowbringer',
                                     damage_spec=DamageSpec(potency=500, damage_class=DamageClass.PET, pet_job_mod_override=100, trait_damage_mult_override=1, status_effect_denylist=('Flood of Shadow','Edge of Shadow','Dragon Sight')))
  skill_library.add_skill(_living_shadow, 'DRK')
  skill_library.add_skill(_living_shadow_shadowbringer, 'DRK')
  _living_shadow_follow_up_skills = (_living_shadow,
                                     _living_shadow,
                                     _living_shadow_shadowbringer,
                                     _living_shadow,
                                     _living_shadow,
                                     _living_shadow)
  _living_shadow_follow_ups = tuple(FollowUp(skill=_living_shadow_follow_up_skills[i],
                                             delay_after_parent_application=6800+i*2200,
                                             snapshot_buffs_with_parent=False,
                                             snapshot_debuffs_with_parent=False)
                                    for i in range(0,len(_living_shadow_follow_up_skills)))
  skill_library.add_skill(Skill(name='Living Shadow',
                                follow_up_skills= _living_shadow_follow_ups,
                                timing_spec=instant_ogcd_timing_spec),
                          'DRK')
  return skill_library

def add_tank_skills_to_skill_library(skill_library):
  skill_library = add_war_skills(skill_library)
  skill_library = add_gnb_skills(skill_library)
  skill_library = add_pld_skills(skill_library)
  skill_library = add_drk_skills(skill_library)
  return skill_library

In [55]:
#@title create_skill_library (actual in-game skills)

def create_skill_library():
  skill_library = SkillLibrary()
  skill_library.add_job_class('GENERAL')
  skill_library.add_skill(Skill(name='Grade 8 Tincture',
                                timing_spec=TimingSpec(base_cast_time=0, is_GCD=False, animation_lock=1300),
                                buff_spec=StatusEffectSpec(duration= 30*1000, main_stat_add=262)),
                          'GENERAL')
  skill_library.add_skill(Skill(name='Grade 7 Tincture',
                                timing_spec=TimingSpec(base_cast_time=0, is_GCD=False, animation_lock=1300),
                                buff_spec=StatusEffectSpec(duration= 30*1000, main_stat_add=223)),
                          'GENERAL')
  skill_library = add_healer_skills_to_skill_library(skill_library)
  skill_library = add_tank_skills_to_skill_library(skill_library)

  return skill_library


In [56]:
#@title SKILL_LIBRARY creation, and execute_rotation
SKILL_LIBRARY = create_skill_library()

def execute_rotation(rb, ignore_trailing_dots= False, skill_library=SKILL_LIBRARY):
  db = DamageBuilder(skill_library)
  start = time.time()
  sim = DamageSimulator(rb.get_stats(), db.get_damage_instances(rb.get_skill_timing(ignore_trailing_dots=ignore_trailing_dots)), 1000000)
  end = time.time()
  print('Time taken: {}'.format(end-start))
  dps = sim.get_dps()
  damage = sim.get_raw_damage()
  per_skill_damage = sim.get_per_skill_damage()
  t = sim.get_damage_time()
  return dps, damage, per_skill_damage, t

In [57]:
#@title Healer rotations

def get_rotation_AST():
  stats = Stats(wd=132, weapon_delay=3.2, main_stat=3366, det_stat=2047, crit_stat=2430, dh_stat=400, speed_stat=1350, job_class = 'AST')
  rb = RotationBuilder(stats, skill_library=SKILL_LIBRARY)

  rb.add_next('Earthly Star')
  rb.add_next('Grade 8 Tincture', job_class='GENERAL')
  rb.add_next('Fall Malefic')
  rb.add_next('Lightspeed')
  rb.add_next('Combust III')
  rb.add_next('Fall Malefic')
  rb.add_next('Fall Malefic')
  rb.add_next('Divination')
  rb.add_next('Astrodyne')
  rb.add_next('Lord of Crowns')
  rb.add_next('Fall Malefic')
  rb.add_next('Fall Malefic')
  rb.add_next('Fall Malefic')
  rb.add_next('Fall Malefic')
  rb.add_next('Fall Malefic')
  rb.add_next('Fall Malefic')
  rb.add_next('Fall Malefic')
  rb.add_next('Fall Malefic')
  return rb

def get_rotation_SGE():
  stats = Stats(wd=132, weapon_delay=2.8, main_stat=3366, det_stat=2047, crit_stat=2502, dh_stat=1012, speed_stat=664, job_class = 'SGE')
  rb = RotationBuilder(stats, skill_library=SKILL_LIBRARY)

  rb.add_next('Grade 8 Tincture', job_class='GENERAL')
  rb.add_next('Dosis III')
  rb.add_next('Eukrasia')
  rb.add_next('Eukrasian Dosis III')
  rb.add_next('Dosis III')
  rb.add_next('Dosis III')
  rb.add_next('Phlegma III')
  rb.add_next('Phlegma III')
  rb.add_next('Dosis III')
  rb.add_next('Dosis III')
  rb.add_next('Dosis III')
  rb.add_next('Dosis III')
  rb.add_next('Dosis III')
  rb.add_next('Dosis III')
  rb.add_next('Dosis III')
  rb.add_next('Dosis III')
  rb.add_next('Dosis III')
  return rb

def get_rotation_WHM(skill_library=SKILL_LIBRARY):
  stats = Stats(wd=132, weapon_delay=3.44, main_stat=3366, det_stat=2047, crit_stat=2502, dh_stat=616, speed_stat=1062, job_class = 'WHM')
  rb = RotationBuilder(stats, skill_library)

  rb.add_next('Grade 8 Tincture', job_class='GENERAL')
  rb.add_next('Glare III')
  rb.add_next('Dia')
  rb.add_next('Glare III')
  rb.add_next('Glare III')
  rb.add_next('Presence of Mind')
  rb.add_next('Glare III')
  rb.add_next('Assize')
  rb.add_next('Glare III')
  rb.add_next('Glare III')
  rb.add_next('Glare III')
  rb.add_next('Glare III')
  rb.add_next('Glare III')
  rb.add_next('Glare III')
  rb.add_next('Glare III')
  rb.add_next('Glare III')
  rb.add_next('Glare III')
  rb.add_next('Glare III')
  return rb

def get_rotation_SCH(skill_library=SKILL_LIBRARY):
  stats = Stats(wd=126, weapon_delay=3.12, main_stat=3366, det_stat=1948, crit_stat=2498, dh_stat=688, speed_stat=954, job_class = 'SCH', healer_or_caster_strength=351)
  rb = RotationBuilder(stats, skill_library)

  rb.add_next('Grade 8 Tincture', job_class='GENERAL')
  rb.add_next('Broil IV')
  rb.add_next('Biolysis')
  rb.add_next('Broil IV')
  rb.add_next('Broil IV')
  rb.add_next('Chain Strategem')
  rb.add_next('Energy Drain')
  rb.add_next('Broil IV')
  rb.add_next('Energy Drain')
  rb.add_next('Broil IV')
  rb.add_next('Energy Drain')
  rb.add_next('Broil IV')
  rb.add_next('Energy Drain')
  rb.add_next('Broil IV')
  rb.add_next('Energy Drain')
  rb.add_next('Broil IV')
  rb.add_next('Energy Drain')
  rb.add_next('Broil IV')
  rb.add_next('Broil IV')
  rb.add_next('Broil IV')
  rb.add_next('Broil IV')
  rb.add_next('Broil IV')
  rb.add_next('Broil IV')
  rb.add_next('Broil IV')
  return rb


In [58]:
#@title Tank rotations
def get_rotation_WAR(skill_library=SKILL_LIBRARY):
  stats = Stats(wd=126, weapon_delay=3.36, main_stat=2910, det_stat=1980, crit_stat=2313, dh_stat=868, speed_stat=592, tenacity=631, job_class = 'WAR')
  rb = RotationBuilder(stats, skill_library)

  rb.add_next('Heavy Swing')
  rb.add_next('Maim')
  rb.add_next('Grade 8 Tincture', job_class='GENERAL')
  rb.add_next("Storm's Eye")
  rb.add_next('Inner Chaos')
  rb.add_next('Upheaval')
  rb.add_next('Onslaught')
  rb.add_next('Primal Rend')
  rb.add_next('Inner Chaos')
  rb.add_next('Onslaught')
  rb.add_next('Fell Cleave', SkillModifier(guaranteed_crit=ForcedCritOrDH.FORCE_YES, guaranteed_dh=ForcedCritOrDH.FORCE_YES))
  rb.add_next('Onslaught')
  rb.add_next('Fell Cleave', SkillModifier(guaranteed_crit=ForcedCritOrDH.FORCE_YES, guaranteed_dh=ForcedCritOrDH.FORCE_YES))
  rb.add_next('Fell Cleave', SkillModifier(guaranteed_crit=ForcedCritOrDH.FORCE_YES, guaranteed_dh=ForcedCritOrDH.FORCE_YES))
  rb.add_next('Heavy Swing')
  rb.add_next('Maim')
  rb.add_next("Storm's Path")
  rb.add_next('Fell Cleave')
  rb.add_next('Inner Chaos')
  return rb

def get_rotation_GNB(skill_library=SKILL_LIBRARY):
  stats = Stats(wd=126, weapon_delay=2.80, main_stat=2891, det_stat=1844, crit_stat=2377, dh_stat=1012, speed_stat=400, tenacity=751, job_class = 'GNB')

  rb = RotationBuilder(stats, skill_library)
  rb.add_next('Keen Edge')
  rb.add_next('Grade 8 Tincture', job_class='GENERAL')
  rb.add_next('Brutal Shell')
  rb.add_next('No Mercy')
  rb.add_next('Gnashing Fang')
  rb.add_next('Jugular Rip')
  rb.add_next('Sonic Break')
  rb.add_next('Blasting Zone')
  rb.add_next('Bow Shock')
  rb.add_next('Double Down')
  rb.add_next('Rough Divide')
  rb.add_next('Savage Claw')
  rb.add_next('Abdomen Tear')
  rb.add_next('Rough Divide')
  rb.add_next('Wicked Talon')
  rb.add_next('Eye Gouge')
  rb.add_next('Solid Barrel')
  rb.add_next('Burst Strike')
  rb.add_next('Hypervelocity')
  rb.add_next('Keen Edge')
  return rb

def get_rotation_PLD(skill_library=SKILL_LIBRARY):
  stats = Stats(wd=126, weapon_delay=2.24, main_stat=2891, det_stat=1844, crit_stat=2377, dh_stat=1012, speed_stat=400, tenacity=751, job_class = 'PLD')
  rb = RotationBuilder(stats, skill_library)

  rb.add_next('Holy Spirit')
  rb.add_next('Fast Blade')
  rb.add_next('Grade 8 Tincture', job_class='GENERAL')
  rb.add_next('Riot Blade')
  rb.add_next('Royal Authority')
  rb.add_next('Fight or Flight')
  rb.add_next('Requiescat')
  rb.add_next('Goring Blade')
  rb.add_next('Circle of Scorn')
  rb.add_next('Explacion')
  rb.add_next('Confiteor (Requiescat)')
  rb.add_next('Intervene')
  rb.add_next('Blade of Faith (Requiescat)')
  rb.add_next('Intervene')
  rb.add_next('Blade of Truth (Requiescat)')
  rb.add_next('Blade of Valor (Requiescat)')
  rb.add_next('Holy Spirit (Divine Might)')
  rb.add_next('Atonement')
  rb.add_next('Atonement')
  rb.add_next('Atonement')
  return rb

def get_rotation_DRK(skill_library=SKILL_LIBRARY):
  stats = Stats(wd=126, weapon_delay=2.96, main_stat=2906, det_stat=1883, crit_stat=2352, dh_stat=868, speed_stat=650, tenacity=631, job_class = 'DRK')
  rb = RotationBuilder(stats, skill_library)

  rb.add_next('Hard Slash')
  rb.add_next('Edge of Shadow')
  rb.add_next('Syphon Strike')
  rb.add_next('Grade 8 Tincture', job_class='GENERAL')
  rb.add_next('Souleater')
  rb.add_next('Living Shadow')
  rb.add_next('Salted Earth')
  rb.add_next('Bloodspiller')
  rb.add_next('Shadowbringer')
  rb.add_next('Edge of Shadow')
  rb.add_next('Bloodspiller')
  rb.add_next('Carve and Spit')
  rb.add_next('Plunge')
  rb.add_next('Bloodspiller')
  rb.add_next('Edge of Shadow')
  rb.add_next('Salt and Darkness')
  rb.add_next('Hard Slash')
  rb.add_next('Plunge')
  rb.add_next('Edge of Shadow')
  rb.add_next('Syphon Strike')
  rb.add_next('Shadowbringer')
  rb.add_next('Edge of Shadow')
  rb.add_next('Souleater')
  rb.add_next('Hard Slash')
  rb.add_next('Syphon Strike')
  rb.add_next('Souleater')
  return rb

In [59]:
# #e2e example, with displaying of results
# dps, damage, per_skill_damage, t = execute_rotation(get_rotation_GNB(), ignore_trailing_dots=True)

# print('Results: ')
# DisplayUtils.print_results(dps)
# plt.figure()
# DisplayUtils.display_results(dps)

# DisplayUtils.print_results(damage)
# plt.figure()
# DisplayUtils.display_results(damage, xlabel='Damage')

# WINDOW_LENGTH= 15
# plt.figure()
# tmp = DisplayUtils.display_damage_over_time([x[2] for x in per_skill_damage],
#                                             [x[0] for x in per_skill_damage],
#                                              WINDOW_LENGTH)


In [60]:
#@title create_test_skill_library, JobClassFnsTest

def create_test_skill_library():
  skill_library = SkillLibrary()
  skill_library.add_job_class('test_job')
  skill_library.add_job_class('test_job2')
  skill_library.add_job_class('test_tank_job')
  skill_library.add_job_class('test_healer_job')

  # TimingSpecs
  gcd_2500 = TimingSpec(base_cast_time=2500, is_GCD=True, animation_lock=5)
  gcd_1500_lock = TimingSpec(base_cast_time=1500, is_GCD=True, animation_lock=50)
  gcd_instant = TimingSpec(base_cast_time=0, is_GCD=True, animation_lock=5)
  gcd_instant_no_lock = TimingSpec(base_cast_time=0, is_GCD=True, animation_lock=0)
  gcd_2500_app_delay = TimingSpec(base_cast_time=2500, application_delay=100, is_GCD=True)
  ogcd_instant = TimingSpec(base_cast_time=0, is_GCD=False, animation_lock=0)
  ogcd_instant_animation_lock = TimingSpec(base_cast_time=0, is_GCD=False, animation_lock=35)
  gcd_1500_const_cast = TimingSpec(base_cast_time=1500, is_GCD=True, animation_lock=5, affected_by_speed_stat=False, affected_by_cast_time_buffs=False)
  gcd_1500_no_haste = TimingSpec(base_cast_time=1500, is_GCD=True, animation_lock=5, affected_by_cast_time_buffs=False)
  auto_timing = TimingSpec(base_cast_time=0, is_GCD=False, animation_lock=0)

  # DamageSpecs
  simple_damage = DamageSpec(potency=660)
  simple_tank_damage = DamageSpec(potency=200)
  simple_dot_damage = DamageSpec(potency=70, damage_class=DamageClass.DOT)
  simple_tank_dot_damage = DamageSpec(potency=50, damage_class=DamageClass.DOT)
  guaranteed_dh_damage = DamageSpec(potency=660, guaranteed_dh=ForcedCritOrDH.FORCE_YES)
  guaranteed_crit_damage = DamageSpec(potency=660, guaranteed_crit=ForcedCritOrDH.FORCE_YES)
  guaranteed_crit_dh_damage = DamageSpec(potency=660, guaranteed_crit=ForcedCritOrDH.FORCE_YES, guaranteed_dh=ForcedCritOrDH.FORCE_YES)
  guaranteed_no_crit_dh_damage = DamageSpec(potency=660, guaranteed_crit=ForcedCritOrDH.FORCE_NO, guaranteed_dh=ForcedCritOrDH.FORCE_NO)
  auto_damage = DamageSpec(potency=110, damage_class=DamageClass.AUTO, trait_damage_mult_override= 1.0)
  healer_auto_damage = DamageSpec(potency=80, damage_class=DamageClass.AUTO, trait_damage_mult_override= 1.0)
  simple_damage_trait_override = DamageSpec(potency=660, trait_damage_mult_override=1.0)
  guaranteed_dh_dot_damage = DamageSpec(potency=50, guaranteed_dh=ForcedCritOrDH.FORCE_YES, damage_class=DamageClass.DOT)
  guaranteed_crit_dot_damage = DamageSpec(potency=50, guaranteed_crit=ForcedCritOrDH.FORCE_YES, damage_class=DamageClass.DOT)
  guaranteed_crit_dh_dot_damage = DamageSpec(potency=50, guaranteed_crit=ForcedCritOrDH.FORCE_YES, guaranteed_dh=ForcedCritOrDH.FORCE_YES, damage_class=DamageClass.DOT)
  pet_damage = DamageSpec(potency=350, pet_job_mod_override=100, trait_damage_mult_override=1, damage_class=DamageClass.PET)
  damage_with_denylist = DamageSpec(potency=660, status_effect_denylist=('test_simple_buff_gcd_2','test_simple_debuff_gcd'))

  # StatusEffectSpecs (buff/debuff)
  simple_buff = StatusEffectSpec(duration=30000, max_duration=60000, crit_rate_add=0.05)
  simple_buff_2 = StatusEffectSpec(duration=10000, crit_rate_add=0.06, dh_rate_add=0.2)
  simple_debuff = StatusEffectSpec(duration=30000, max_duration=60000, damage_mult=1.2)
  simple_debuff_2 = StatusEffectSpec(duration=10000, damage_mult=1.3)
  auto_attack_buff = StatusEffectSpec(duration= 10000, auto_attack_delay_reduction=0.25)
  haste_buff1 = StatusEffectSpec(duration= 15000, haste_time_reduction=0.25)
  haste_buff2 = StatusEffectSpec(duration= 15000, haste_time_reduction=0.10)
  flat_cast_time_reduction_buff = StatusEffectSpec(duration=15000, flat_cast_time_reduction= 2500)

  # Skill creation
  test_gcd = Skill(name='test_gcd', timing_spec = gcd_2500, damage_spec=simple_damage)
  test_gcd_1500_lock = Skill(name='test_gcd_1500_lock', timing_spec = gcd_1500_lock, damage_spec=simple_damage)
  test_instant_gcd = Skill(name='test_instant_gcd', timing_spec = gcd_instant, damage_spec=simple_damage)
  test_instant_gcd_no_lock = Skill(name='test_instant_gcd_no_lock', timing_spec = gcd_instant_no_lock, damage_spec=simple_damage)
  test_tank_gcd = Skill(name='test_tank_gcd', timing_spec = gcd_2500, damage_spec=simple_tank_damage)
  test_gcd_with_app_delay = Skill(name='test_gcd_with_app_delay', timing_spec = gcd_2500_app_delay)
  test_ogcd = Skill(name='test_ogcd', timing_spec = ogcd_instant)
  test_ogcd_animation_lock = Skill(name='test_ogcd_animation_lock', timing_spec = ogcd_instant_animation_lock)
  test_non_dot_follow_up = Skill(name='test_non_dot_follow_up')
  test_follow_up = Skill(name='test_follow_up', timing_spec=gcd_2500, follow_up_skills = (FollowUp(skill=test_non_dot_follow_up, delay_after_parent_application=0, snapshot_buffs_with_parent=True, snapshot_debuffs_with_parent=True),
                                                                                          FollowUp(skill=test_non_dot_follow_up, delay_after_parent_application=3000, snapshot_buffs_with_parent=False, snapshot_debuffs_with_parent=True),
                                                                                          FollowUp(skill=test_non_dot_follow_up, delay_after_parent_application=7000, snapshot_buffs_with_parent=False, snapshot_debuffs_with_parent=False)))
  test_dot_tick = Skill(name='test_dot_tick', damage_spec=simple_dot_damage)
  test_tank_dot_tick = Skill(name='test_tank_dot_tick', damage_spec=simple_tank_dot_damage)
  test_dot_gcd = Skill(name='test_dot_gcd', timing_spec=gcd_2500, follow_up_skills = (FollowUp(skill=test_dot_tick, delay_after_parent_application=0, dot_duration=15*1000, snapshot_buffs_with_parent= True, snapshot_debuffs_with_parent=True),))
  test_dot_biolysis = Skill(name='test_dot_biolysis', timing_spec=gcd_2500, follow_up_skills = (FollowUp(skill=test_dot_tick, delay_after_parent_application=0, dot_duration=30*1000, snapshot_buffs_with_parent= True, snapshot_debuffs_with_parent=True),))
  test_dot_instant_gcd = Skill(name='test_dot_instant_gcd', timing_spec=gcd_instant, follow_up_skills = (FollowUp(skill=test_dot_tick, delay_after_parent_application=0, dot_duration=15*1000, snapshot_buffs_with_parent= True, snapshot_debuffs_with_parent=True),))
  test_dot_gcd_with_other_follow_up = Skill(name='test_dot_gcd_with_other_follow_up', timing_spec=gcd_2500, follow_up_skills = (FollowUp(skill=test_non_dot_follow_up, delay_after_parent_application=0, snapshot_buffs_with_parent=True, snapshot_debuffs_with_parent=True),
                                                                                                                                FollowUp(skill=test_dot_tick, delay_after_parent_application=0, dot_duration=15*1000, snapshot_buffs_with_parent= True, snapshot_debuffs_with_parent=True)))
  test_ground_dot_gcd = Skill(name='test_ground_dot_gcd', timing_spec=gcd_2500, follow_up_skills = (FollowUp(skill=test_dot_tick, delay_after_parent_application=0, dot_duration=15*1000, snapshot_buffs_with_parent= True, snapshot_debuffs_with_parent=False),))
  test_simple_buff_gcd = Skill('test_simple_buff_gcd', timing_spec=gcd_instant, buff_spec=simple_buff)
  test_party_buff = Skill('test_party_buff', timing_spec=gcd_instant, buff_spec=simple_buff)
  test_simple_buff_gcd_2 = Skill('test_simple_buff_gcd_2', timing_spec=gcd_instant, buff_spec=simple_buff_2)
  test_simple_buff_gcd_3 = Skill('test_simple_buff_gcd_3', timing_spec=gcd_2500, buff_spec=simple_buff) #NOT instant cast
  test_simple_debuff_gcd = Skill('test_simple_debuff_gcd', timing_spec=gcd_instant, debuff_spec=simple_debuff)
  test_simple_debuff_gcd_2 = Skill('test_simple_debuff_gcd_2', timing_spec=gcd_instant, debuff_spec=simple_debuff_2)
  test_guaranteed_dh = Skill('test_guaranteed_dh', timing_spec=gcd_instant, damage_spec=guaranteed_dh_damage)
  test_guaranteed_crit = Skill('test_guaranteed_crit', timing_spec=gcd_instant, damage_spec=guaranteed_crit_damage)
  test_guaranteed_crit_dh = Skill('test_guaranteed_crit_dh', timing_spec=gcd_instant, damage_spec=guaranteed_crit_dh_damage)
  test_guaranteed_no_crit_dh = Skill('test_guaranteed_no_crit_dh', timing_spec=gcd_instant, damage_spec=guaranteed_no_crit_dh_damage)
  test_guaranteed_dh_dot = Skill('test_guaranteed_dh_dot', timing_spec=gcd_instant, damage_spec=guaranteed_dh_dot_damage)
  test_guaranteed_crit_dot = Skill('test_guaranteed_crit_dot', timing_spec=gcd_instant, damage_spec=guaranteed_crit_dot_damage)
  test_guaranteed_crit_dh_dot = Skill('test_guaranteed_crit_dh_dot', timing_spec=gcd_instant, damage_spec=guaranteed_crit_dh_dot_damage)
  test_auto = Skill('Auto', timing_spec=auto_timing, damage_spec=auto_damage)
  test_healer_auto = Skill('Auto', timing_spec=auto_timing, damage_spec=healer_auto_damage)
  test_gcd_trait_override = Skill(name='test_gcd_trait_override', timing_spec = gcd_2500, damage_spec=simple_damage_trait_override)
  test_auto_attack_buff = Skill(name='test_auto_attack_buff', timing_spec=gcd_2500, buff_spec=auto_attack_buff)
  test_auto_attack_buff_instant = Skill(name='test_auto_attack_buff_instant', timing_spec=gcd_instant, buff_spec=auto_attack_buff)
  test_auto_attack_buff_instant_follow_up = Skill(name='test_auto_attack_buff_instant_follow_up', buff_spec=auto_attack_buff)
  test_auto_attack_buff_on_follow_up = Skill(name='test_auto_attack_buff_on_follow_up', timing_spec=gcd_2500, follow_up_skills = (FollowUp(skill=test_auto_attack_buff_instant_follow_up, delay_after_parent_application=0),))
  test_auto_attack_buff2 = Skill(name='test_auto_attack_buff2', timing_spec=gcd_2500, buff_spec=auto_attack_buff)
  test_haste_buff1 = Skill(name='test_haste_buff1', timing_spec=ogcd_instant, buff_spec=haste_buff1)
  test_haste_buff2 = Skill(name='test_haste_buff2', timing_spec=ogcd_instant, buff_spec=haste_buff2)
  _test_haste_follow_up = Skill(name='_test_haste_follow_up', buff_spec=haste_buff1)
  test_haste_follow_up = Skill(name='test_haste_follow_up', timing_spec=ogcd_instant, follow_up_skills = (FollowUp(skill=_test_haste_follow_up, delay_after_parent_application=0),))
  test_gcd_1500_const_cast = Skill(name='test_gcd_1500_const_cast', timing_spec=gcd_1500_const_cast, damage_spec=simple_damage)
  test_gcd_1500_no_haste = Skill(name='test_gcd_1500_no_haste', timing_spec=gcd_1500_no_haste, damage_spec=simple_damage)
  test_pet_gcd = Skill(name='test_pet_gcd', timing_spec=gcd_1500_lock, damage_spec=pet_damage)
  test_gcd_with_denylist = Skill(name='test_gcd_with_denylist', timing_spec = gcd_2500, damage_spec=damage_with_denylist)
  test_flat_cast_time_reduction = Skill(name='test_flat_cast_time_reduction', timing_spec=ogcd_instant, buff_spec=flat_cast_time_reduction_buff)
  _test_buff_then_damage = Skill('_test_buff_then_damage', damage_spec=simple_damage)
  test_buff_then_damage = Skill(name='test_buff_then_damage',
                                timing_spec=gcd_instant,
                                buff_spec=simple_buff,
                                follow_up_skills=(FollowUp(skill=_test_buff_then_damage, delay_after_parent_application=0),))
  test_default_buff_damage_order = Skill(name='test_default_buff_damage_order',
                                         timing_spec=gcd_instant,
                                         buff_spec=simple_buff,
                                         damage_spec=simple_damage)

  skill_library.add_skill(test_gcd, 'test_job')
  skill_library.add_skill(test_ogcd, 'test_job')
  skill_library.add_skill(test_gcd_with_app_delay, 'test_job')
  skill_library.add_skill(test_non_dot_follow_up, 'test_job')
  skill_library.add_skill(test_follow_up, 'test_job')
  skill_library.add_skill(test_dot_gcd, 'test_job')
  skill_library.add_skill(test_ground_dot_gcd, 'test_job')
  skill_library.add_skill(test_dot_tick, 'test_job')
  skill_library.add_skill(test_dot_gcd_with_other_follow_up, 'test_job')
  skill_library.add_skill(test_dot_instant_gcd, 'test_job')
  skill_library.add_skill(test_simple_buff_gcd, 'test_job')
  skill_library.add_skill(test_simple_buff_gcd_2, 'test_job')
  skill_library.add_skill(test_simple_buff_gcd_3, 'test_job')
  skill_library.add_skill(test_simple_debuff_gcd, 'test_job')
  skill_library.add_skill(test_simple_debuff_gcd_2, 'test_job')
  skill_library.add_skill(test_dot_biolysis, 'test_job')
  skill_library.add_skill(test_guaranteed_dh, 'test_job')
  skill_library.add_skill(test_guaranteed_crit, 'test_job')
  skill_library.add_skill(test_guaranteed_crit_dh, 'test_job')
  skill_library.add_skill(test_guaranteed_no_crit_dh, 'test_job')
  skill_library.add_skill(test_guaranteed_dh_dot, 'test_job')
  skill_library.add_skill(test_guaranteed_crit_dot, 'test_job')
  skill_library.add_skill(test_guaranteed_crit_dh_dot, 'test_job')
  skill_library.add_skill(test_auto_attack_buff, 'test_job')
  skill_library.add_skill(test_auto_attack_buff2, 'test_job')
  skill_library.add_skill(test_auto, 'test_job')
  skill_library.add_skill(test_instant_gcd, 'test_job')
  skill_library.add_skill(test_instant_gcd_no_lock, 'test_job')
  skill_library.add_skill(test_auto_attack_buff_instant, 'test_job')
  skill_library.add_skill(test_auto_attack_buff_on_follow_up, 'test_job')
  skill_library.add_skill(test_auto_attack_buff_instant_follow_up, 'test_job')
  skill_library.add_skill(test_ogcd_animation_lock, 'test_job')
  skill_library.add_skill(test_gcd_1500_lock, 'test_job')
  skill_library.add_skill(test_haste_buff1, 'test_job')
  skill_library.add_skill(test_haste_buff2, 'test_job')
  skill_library.add_skill(_test_haste_follow_up, 'test_job')
  skill_library.add_skill(test_haste_follow_up, 'test_job')
  skill_library.add_skill(test_pet_gcd, 'test_job')
  skill_library.add_skill(test_gcd_with_denylist, 'test_job')
  skill_library.add_skill(test_gcd_1500_const_cast, 'test_job')
  skill_library.add_skill(test_gcd_1500_no_haste, 'test_job')
  skill_library.add_skill(test_flat_cast_time_reduction, 'test_job')
  skill_library.add_skill(test_buff_then_damage, 'test_job')
  skill_library.add_skill(_test_buff_then_damage, 'test_job')
  skill_library.add_skill(test_default_buff_damage_order, 'test_job')


  skill_library.add_skill(test_auto, 'test_job2')
  skill_library.add_skill(test_gcd, 'test_job2')
  skill_library.add_skill(test_gcd_trait_override, 'test_job2')
  skill_library.add_skill(test_party_buff, 'test_job2')

  skill_library.add_skill(test_auto, 'test_tank_job')
  skill_library.add_skill(test_tank_gcd, 'test_tank_job')
  skill_library.add_skill(test_tank_dot_tick, 'test_tank_job')
  skill_library.add_skill(test_pet_gcd, 'test_tank_job')

  skill_library.add_skill(test_healer_auto, 'test_healer_job')

  return skill_library

class JobClassFnsTest:
  JOB_MODS = {'test_job': 115, 'test_job2': 105, 'test_tank_job': 105, 'test_healer_job': 115}

  @staticmethod
  def compute_trait_damage_mult(job_class):
    if job_class == 'test_job2':
      return 1.40
    elif job_class == 'test_healer_job':
      return 1.30
    else:
      return 1

  @staticmethod
  def isCaster(job_class):
    return False

  @staticmethod
  def isHealer(job_class):
    return job_class in ['test_healer_job']

  @staticmethod
  def isTank(job_class):
    return job_class in ['test_tank_job']


In [61]:
#@title Test utils

class TestClass:
  def __init__(self):
    self.test_fns = []

  def is_a_test(f):
    f.__is_a_test__ = True
    return f

  def _get_test_methods(self):
    test_methods = []
    for fn_name in dir(self):
      fn = getattr(self, fn_name)
      if getattr(fn, '__is_a_test__', False):
        test_methods.append(fn)
    return test_methods

  @staticmethod
  def _compare_sequential(result, expected):
    test_passed = True
    err_msg = ""
    if len(expected) != len(result):
      test_passed = False
      err_msg += "Expected {} skills returned. Instead got {}. ".format(len(expected), len(result))
      return test_passed, err_msg
    for i in range(0, len(expected)):
      if expected[i] != result[i]:
        test_passed = False
        err_msg += "Position {} was not the same.\n Expected: {}\n Actual: {}\n".format(i, expected[i], result[i])
    return test_passed, err_msg

  def print_result(self, passing, failing):
    print('Testing for {}. {}/{} tests passed.'.format(self.__class__.__name__, len(passing), len(passing)+len(failing)))
    if len(failing) > 0:
      print('Failing tests:')
      for test_info in failing:
        print("{}: {}\n".format(test_info[0], test_info[1]))
    print('Passing tests:')
    for test_name in passing:
      print("{}".format(test_name))

  def run_single(self, test_name):
    for test_fn in self._get_test_methods():
      if test_name == test_fn.__name__:
        test_passed, err_msg = test_fn()
        if test_passed:
          print('{} passed!'.format(test_name))
          return
        else:
          print('{} failed: {}'.format(test_name, err_msg))
          return
    print('No test found with name {}'.format(test_name))

  def run_all(self, verbose=False):
    passing = []
    failing = []
    for test_fn in self._get_test_methods():
      test_name = test_fn.__name__
      test_passed, err_msg = test_fn()
      if (test_passed):
        passing.append(test_name)
      else:
        failing.append((test_name, err_msg))
    if(verbose or len(failing) > 0):
      self.print_result(passing, failing)
      print("================")
    return (len(passing), len(failing))

class AllTest():
  def __init__(self):
    self.__all_test_classes = []

  def register_test_class(self, test_class):
    self.__all_test_classes.append(test_class)

  def run_all(self):
    total_passing = 0
    total_failing = 0
    for test_class in self.__all_test_classes:
      print('Running: {}'.format(test_class.__class__.__name__))
      (num_pass, num_fail) = test_class.run_all()
      total_passing += num_pass
      total_failing += num_fail
    print('All tests done. Num passing: {}. Num failing: {}'.format(total_passing, total_failing))


In [62]:
#@title display_example
def display_example():
  SKILL_LIBRARY = create_test_skill_library()
  stats = Stats(wd=126, weapon_delay=3.36, main_stat=2910, det_stat=1980, crit_stat=2313, dh_stat=868, speed_stat=592, job_class = 'test_job')
  processed_stats =  ProcessedStats(stats)
  skill = SKILL_LIBRARY.get_skill('test_guaranteed_dh', 'test_job')

  dmg_instances = [
                  (3040, SKILL_LIBRARY.get_skill('test_guaranteed_dh', 'test_job'), SkillModifier(), (StatusEffects(), StatusEffects())),
                  (6000, SKILL_LIBRARY.get_skill('test_gcd', 'test_job'), SkillModifier(), (StatusEffects(), StatusEffects())),
                  (7000, SKILL_LIBRARY.get_skill('test_gcd', 'test_job'), SkillModifier(), (StatusEffects(dh_rate_add=0.1), StatusEffects())),
                  (8000, SKILL_LIBRARY.get_skill('test_gcd', 'test_job'), SkillModifier(), (StatusEffects(dh_rate_add=0.1), StatusEffects())),
                  (9000, SKILL_LIBRARY.get_skill('test_gcd', 'test_job'), SkillModifier(), (StatusEffects(dh_rate_add=0.1), StatusEffects()))
                  ]

  sim = DamageSimulator(stats, dmg_instances, 1000000)

  damage = sim.get_raw_damage()
  DisplayUtils.display_results(damage)

  res = sim.get_per_skill_damage(dmg_instances)
  print(res)
# display_example()

In [63]:
#@title TimingSpecTest
class TimingSpecTest(TestClass):

  @TestClass.is_a_test
  def gcd_override_gcd_test(self):
    test_passed = True
    err_msg=""
    timing_spec = TimingSpec(base_cast_time=2000, is_GCD=True)
    if timing_spec.gcd_base_recast_time != 2500:
      err_msg = "Recast time was expected to be 2500, but it was {}.".format(timing_spec.gcd_base_recast_time)
      test_passed = False
    return test_passed, err_msg

  @TestClass.is_a_test
  def gcd_override_ogcd_test(self):
    test_passed = True
    err_msg=""
    timing_spec = TimingSpec(base_cast_time=2000, is_GCD=False)
    if timing_spec.gcd_base_recast_time != 0:
      err_msg = "Recast time was expected to be 0, but it was {}.".format(timing_spec.gcd_base_recast_time)
      test_passed = False
    return test_passed, err_msg

  @TestClass.is_a_test
  def gcd_override_set_recast_time(self):
    test_passed = True
    err_msg=""
    timing_spec = TimingSpec(base_cast_time=2000, gcd_base_recast_time=1500, is_GCD=False)
    if timing_spec.gcd_base_recast_time != 1500:
      err_msg = "Recast time was expected to be 1500, but it was {}.".format(timing_spec.gcd_base_recast_time)
      test_passed = False
    return test_passed, err_msg

In [64]:
#@title SnapshotAndApplicationEventsTest
class SnapshotAndApplicationEventsTest(TestClass):
  def __init__(self):
    self.__skill_library = create_test_skill_library()

  @TestClass.is_a_test
  def test_get_last_event(self):
    test_passed = True
    err_msg = ""
    rb_result = SnapshotAndApplicationEvents()
    rb_result.add(Utils.transform_time_to_prio(2940), 2940, 3440, self.__skill_library.get_skill('test_dot_gcd', 'test_job'), SkillModifier(), [True, True])
    rb_result.add(Utils.transform_time_to_prio(2940)+1, 2940, 3440, self.__skill_library.get_skill('test_dot_tick', 'test_job'), SkillModifier(), [True, True])
    rb_result.add(Utils.transform_time_to_prio(2940)+5, 2940, 15440, self.__skill_library.get_skill('test_dot_tick', 'test_job'), SkillModifier(), [True, True])
    rb_result.add(Utils.transform_time_to_prio(12940), 12940, 13440, self.__skill_library.get_skill('test_gcd', 'test_job'), SkillModifier(), [True, True])

    actual_last_event_time = rb_result.get_last_event_time()
    expected_last_event_time = 15440
    if actual_last_event_time != expected_last_event_time:
      err_msg = "Last event time was expected to be {}, but was {}.".format(expected_last_event_time, actual_last_event_time)
      test_passed = False
    return test_passed, err_msg

In [65]:
#@title RotationBuilderTest
class RotationBuilderTest(TestClass):
  def __init__(self):
    self.__stats = Stats(wd=126, weapon_delay=3.44, main_stat=2945, det_stat=1620, crit_stat=2377, dh_stat=1048, speed_stat=708, job_class = 'test_job')
    self.__skill_library = create_test_skill_library()

  @TestClass.is_a_test
  def test_snap_dots_to_server_tick_starting_at(self):
    rb = RotationBuilder(self.__stats, self.__skill_library, snap_dots_to_server_tick_starting_at= -1.5)
    rb.add(1.0, 'test_dot_gcd')
    rb.add(29.06, 'test_dot_gcd')

    expected = ((SnapshotAndApplicationEvents.EventTimes(2940, 3440), self.__skill_library.get_skill('test_dot_gcd', 'test_job'), SkillModifier(), [True, True]),
                (SnapshotAndApplicationEvents.EventTimes(2940, 4500), self.__skill_library.get_skill('test_dot_tick', 'test_job'), SkillModifier(), [True, True]),
                (SnapshotAndApplicationEvents.EventTimes(2940, 7500), self.__skill_library.get_skill('test_dot_tick', 'test_job'), SkillModifier(), [True, True]),
                (SnapshotAndApplicationEvents.EventTimes(2940, 10500), self.__skill_library.get_skill('test_dot_tick', 'test_job'), SkillModifier(), [True, True]),
                (SnapshotAndApplicationEvents.EventTimes(2940, 13500), self.__skill_library.get_skill('test_dot_tick', 'test_job'), SkillModifier(), [True, True]),
                (SnapshotAndApplicationEvents.EventTimes(2940, 16500), self.__skill_library.get_skill('test_dot_tick', 'test_job'), SkillModifier(), [True, True]),
                (SnapshotAndApplicationEvents.EventTimes(31000, 31500), self.__skill_library.get_skill('test_dot_gcd', 'test_job'), SkillModifier(), [True, True]),
                (SnapshotAndApplicationEvents.EventTimes(31000, 31500), self.__skill_library.get_skill('test_dot_tick', 'test_job'), SkillModifier(), [True, True]),
                (SnapshotAndApplicationEvents.EventTimes(31000, 34500), self.__skill_library.get_skill('test_dot_tick', 'test_job'), SkillModifier(), [True, True]),
                (SnapshotAndApplicationEvents.EventTimes(31000, 37500), self.__skill_library.get_skill('test_dot_tick', 'test_job'), SkillModifier(), [True, True]),
                (SnapshotAndApplicationEvents.EventTimes(31000, 40500), self.__skill_library.get_skill('test_dot_tick', 'test_job'), SkillModifier(), [True, True]),
                (SnapshotAndApplicationEvents.EventTimes(31000, 43500), self.__skill_library.get_skill('test_dot_tick', 'test_job'), SkillModifier(), [True, True]))

    result = rb.get_skill_timing().get_q()
    result = [result[i][1:] for i in range(0, len(result))]
    return self._compare_sequential(result, expected)

  @TestClass.is_a_test
  def test_ignore_trailing_dot(self):
    rb = RotationBuilder(self.__stats, self.__skill_library)
    rb.add(1, 'test_dot_gcd')
    rb.add(8, 'test_gcd')

    expected = ((SnapshotAndApplicationEvents.EventTimes(2940, 3440), self.__skill_library.get_skill('test_dot_gcd', 'test_job'), SkillModifier(), [True, True]),
                (SnapshotAndApplicationEvents.EventTimes(2940, 3440), self.__skill_library.get_skill('test_dot_tick', 'test_job'), SkillModifier(), [True, True]),
                (SnapshotAndApplicationEvents.EventTimes(2940, 6440), self.__skill_library.get_skill('test_dot_tick', 'test_job'), SkillModifier(), [True, True]),
                (SnapshotAndApplicationEvents.EventTimes(2940, 9440), self.__skill_library.get_skill('test_dot_tick', 'test_job'), SkillModifier(), [True, True]),
                (SnapshotAndApplicationEvents.EventTimes(9940, 10440), self.__skill_library.get_skill('test_gcd', 'test_job'), SkillModifier(), [True, True]))

    result = rb.get_skill_timing(ignore_trailing_dots=True).get_q()
    result = [result[i][1:] for i in range(0, len(result))]
    return self._compare_sequential(result, expected)

  @TestClass.is_a_test
  def test_buff_then_damage(self):
    rb = RotationBuilder(self.__stats, self.__skill_library)
    rb.add(0, 'test_buff_then_damage')
    expected = ((SnapshotAndApplicationEvents.EventTimes(0, None), self.__skill_library.get_skill('test_buff_then_damage', self.__stats.job_class), SkillModifier(), [True, True]),
                (SnapshotAndApplicationEvents.EventTimes(0, None), self.__skill_library.get_skill('_test_buff_then_damage', self.__stats.job_class), SkillModifier(), [True, True]),)

    result = rb.get_skill_timing().get_q()
    result = [result[i][1:] for i in range(0, len(result))]
    return self._compare_sequential(result, expected)

  @TestClass.is_a_test
  def test_flat_cast_time_reduction_add(self):
    rb = RotationBuilder(self.__stats, self.__skill_library)
    rb.add(0, 'test_gcd_1500_lock')
    rb.add(1.6, 'test_flat_cast_time_reduction')
    rb.add(6, 'test_gcd_1500_lock')
    rb.add(9, 'test_gcd')

    expected = ((SnapshotAndApplicationEvents.EventTimes(960, 1460), self.__skill_library.get_skill('test_gcd_1500_lock', self.__stats.job_class), SkillModifier(), [True, True]),
                (SnapshotAndApplicationEvents.EventTimes(1600, None), self.__skill_library.get_skill('test_flat_cast_time_reduction', self.__stats.job_class), SkillModifier(), [True, True]),
                (SnapshotAndApplicationEvents.EventTimes(6000, None), self.__skill_library.get_skill('test_gcd_1500_lock', self.__stats.job_class), SkillModifier(), [True, True]),
                (SnapshotAndApplicationEvents.EventTimes(9000, None), self.__skill_library.get_skill('test_gcd', self.__stats.job_class), SkillModifier(), [True, True]),)

    result = rb.get_skill_timing().get_q()
    result = [result[i][1:] for i in range(0, len(result))]
    return self._compare_sequential(result, expected)

  @TestClass.is_a_test
  def test_flat_cast_time_reduction_add_next(self):
    rb = RotationBuilder(self.__stats, self.__skill_library)
    rb.add_next('test_gcd_1500_lock')
    rb.add_next('test_flat_cast_time_reduction')
    rb.add_next('test_gcd_1500_lock')
    rb.add_next('test_gcd')

    expected = ((SnapshotAndApplicationEvents.EventTimes(960, 1460), self.__skill_library.get_skill('test_gcd_1500_lock', self.__stats.job_class), SkillModifier(), [True, True]),
                (SnapshotAndApplicationEvents.EventTimes(1460+50, None), self.__skill_library.get_skill('test_flat_cast_time_reduction', self.__stats.job_class), SkillModifier(), [True, True]),
                (SnapshotAndApplicationEvents.EventTimes(2440, None), self.__skill_library.get_skill('test_gcd_1500_lock', self.__stats.job_class), SkillModifier(), [True, True]),
                (SnapshotAndApplicationEvents.EventTimes(4880, None), self.__skill_library.get_skill('test_gcd', self.__stats.job_class), SkillModifier(), [True, True]),)

    result = rb.get_skill_timing().get_q()
    result = [result[i][1:] for i in range(0, len(result))]
    return self._compare_sequential(result, expected)

  @TestClass.is_a_test
  def test_add_next_const_and_no_speed_skills(self):
    rb = RotationBuilder(self.__stats, self.__skill_library)
    rb.add_next('test_haste_buff1')
    rb.add_next('test_gcd_1500_const_cast')
    rb.add_next('test_gcd_1500_no_haste')

    expected = ((SnapshotAndApplicationEvents.EventTimes(0, None), self.__skill_library.get_skill('test_haste_buff1', self.__stats.job_class), SkillModifier(), [True, True]),
                (SnapshotAndApplicationEvents.EventTimes(1000, 1500), self.__skill_library.get_skill('test_gcd_1500_const_cast', self.__stats.job_class), SkillModifier(), [True, True]),
                (SnapshotAndApplicationEvents.EventTimes(2500+1460-500, 2500+1460), self.__skill_library.get_skill('test_gcd_1500_no_haste', self.__stats.job_class), SkillModifier(), [True, True]),)

    result = rb.get_skill_timing().get_q()
    result = [result[i][1:] for i in range(0, len(result))]
    return self._compare_sequential(result, expected)

  @TestClass.is_a_test
  def test_add_const_and_no_speed_skills(self):
    rb = RotationBuilder(self.__stats, self.__skill_library)
    rb.add(0, 'test_haste_buff1')
    rb.add(3, 'test_gcd_1500_const_cast')
    rb.add(6, 'test_gcd_1500_no_haste')

    expected = ((SnapshotAndApplicationEvents.EventTimes(0, None), self.__skill_library.get_skill('test_haste_buff1', self.__stats.job_class), SkillModifier(), [True, True]),
                (SnapshotAndApplicationEvents.EventTimes(4000, 4500), self.__skill_library.get_skill('test_gcd_1500_const_cast', self.__stats.job_class), SkillModifier(), [True, True]),
                (SnapshotAndApplicationEvents.EventTimes(6000+1460-500, 6000+1460), self.__skill_library.get_skill('test_gcd_1500_no_haste', self.__stats.job_class), SkillModifier(), [True, True]),)

    result = rb.get_skill_timing().get_q()
    result = [result[i][1:] for i in range(0, len(result))]
    return self._compare_sequential(result, expected)

  @TestClass.is_a_test
  def test_stable_skill_sequence(self):
    rb = RotationBuilder(self.__stats, self.__skill_library)
    rb.add_next('test_instant_gcd_no_lock')
    rb.add_next('test_haste_buff1')

    expected = ((SnapshotAndApplicationEvents.EventTimes(0, None), self.__skill_library.get_skill('test_instant_gcd_no_lock', self.__stats.job_class), SkillModifier(), [True, True]),
                (SnapshotAndApplicationEvents.EventTimes(0, None), self.__skill_library.get_skill('test_haste_buff1', self.__stats.job_class), SkillModifier(), [True, True]),)

    result = rb.get_skill_timing().get_q()
    result = [result[i][1:] for i in range(0, len(result))]
    return self._compare_sequential(result, expected)

  @TestClass.is_a_test
  def test_simple_with_haste(self):
    rb = RotationBuilder(self.__stats, create_test_skill_library())
    rb.add(0, 'test_haste_buff1')
    rb.add(1, 'test_simple_buff_gcd')
    rb.add(3, 'test_gcd')

    expected = ((SnapshotAndApplicationEvents.EventTimes(0, None), self.__skill_library.get_skill('test_haste_buff1', self.__stats.job_class), SkillModifier(), [True, True]),
                (SnapshotAndApplicationEvents.EventTimes(1000, None), self.__skill_library.get_skill('test_simple_buff_gcd', self.__stats.job_class), SkillModifier(), [True, True]),
                (SnapshotAndApplicationEvents.EventTimes(4330, 4830), self.__skill_library.get_skill('test_gcd', self.__stats.job_class), SkillModifier(), [True, True]),)

    result = rb.get_skill_timing().get_q()
    result = [result[i][1:] for i in range(0, len(result))]
    return self._compare_sequential(result, expected)

  @TestClass.is_a_test
  def test_simple_with_haste_followup(self):
    rb = RotationBuilder(self.__stats, create_test_skill_library())
    rb.add(0, 'test_haste_follow_up')
    rb.add(3, 'test_gcd')

    expected = ((SnapshotAndApplicationEvents.EventTimes(0, None), self.__skill_library.get_skill('test_haste_follow_up', self.__stats.job_class), SkillModifier(), [True, True]),
                (SnapshotAndApplicationEvents.EventTimes(0, None), self.__skill_library.get_skill('_test_haste_follow_up', self.__stats.job_class), SkillModifier(), [True, True]),
                (SnapshotAndApplicationEvents.EventTimes(4330, 4830), self.__skill_library.get_skill('test_gcd', self.__stats.job_class), SkillModifier(), [True, True]),)

    result = rb.get_skill_timing().get_q()
    result = [result[i][1:] for i in range(0, len(result))]
    return self._compare_sequential(result, expected)

  @TestClass.is_a_test
  def test_add_next_with_haste(self):
    rb = RotationBuilder(self.__stats, create_test_skill_library())
    rb.add_next('test_gcd')
    rb.add_next('test_haste_buff1')
    rb.add_next('test_gcd')
    rb.add_next('test_haste_buff2')
    rb.add_next('test_gcd_1500_lock')
    rb.add_next('test_gcd_1500_lock')

    expected = ((SnapshotAndApplicationEvents.EventTimes(1940, 2440), self.__skill_library.get_skill('test_gcd', self.__stats.job_class), SkillModifier(), [True, True]),
                (SnapshotAndApplicationEvents.EventTimes(2440+5, None), self.__skill_library.get_skill('test_haste_buff1', self.__stats.job_class), SkillModifier(), [True, True]),
                (SnapshotAndApplicationEvents.EventTimes(2440+5+0.75*2440-500, 2440+5+0.75*2440), self.__skill_library.get_skill('test_gcd', self.__stats.job_class), SkillModifier(), [True, True]),
                (SnapshotAndApplicationEvents.EventTimes(2440+5+0.75*2440+5,  None), self.__skill_library.get_skill('test_haste_buff2', self.__stats.job_class), SkillModifier(), [True, True]),
                (SnapshotAndApplicationEvents.EventTimes((2440+5+0.75*2440+5) + 980-500, (2440+5+0.75*2440+5) + 980), self.__skill_library.get_skill('test_gcd_1500_lock', self.__stats.job_class), SkillModifier(), [True, True]),
                (SnapshotAndApplicationEvents.EventTimes(((2440+5+0.75*2440+5)+1640)+980-500, ((2440+5+0.75*2440+5)+1640)+980), self.__skill_library.get_skill('test_gcd_1500_lock', self.__stats.job_class), SkillModifier(), [True, True]),)

    result = rb.get_skill_timing().get_q()
    result = [result[i][1:] for i in range(0, len(result))]
    return self._compare_sequential(result, expected)

  @TestClass.is_a_test
  def test_add_next_with_haste_followup(self):
    rb = RotationBuilder(self.__stats, create_test_skill_library())
    rb.add_next('test_gcd')
    rb.add_next('test_haste_follow_up')
    rb.add_next('test_gcd')
    rb.add_next('test_haste_buff2')
    rb.add_next('test_gcd_1500_lock')
    rb.add_next('test_gcd_1500_lock')

    expected = ((SnapshotAndApplicationEvents.EventTimes(1940, 2440), self.__skill_library.get_skill('test_gcd', self.__stats.job_class), SkillModifier(), [True, True]),
            (SnapshotAndApplicationEvents.EventTimes(2440+5, None), self.__skill_library.get_skill('test_haste_follow_up', self.__stats.job_class), SkillModifier(), [True, True]),
            (SnapshotAndApplicationEvents.EventTimes(2440+5, None), self.__skill_library.get_skill('_test_haste_follow_up', self.__stats.job_class), SkillModifier(), [True, True]),
            (SnapshotAndApplicationEvents.EventTimes(2440+5+0.75*2440-500, 2440+5+0.75*2440), self.__skill_library.get_skill('test_gcd', self.__stats.job_class), SkillModifier(), [True, True]),
            (SnapshotAndApplicationEvents.EventTimes(2440+5+0.75*2440+5,  None), self.__skill_library.get_skill('test_haste_buff2', self.__stats.job_class), SkillModifier(), [True, True]),
            (SnapshotAndApplicationEvents.EventTimes((2440+5+0.75*2440+5) + 980-500, (2440+5+0.75*2440+5) + 980), self.__skill_library.get_skill('test_gcd_1500_lock', self.__stats.job_class), SkillModifier(), [True, True]),
            (SnapshotAndApplicationEvents.EventTimes(((2440+5+0.75*2440+5)+1640)+980-500, ((2440+5+0.75*2440+5)+1640)+980), self.__skill_library.get_skill('test_gcd_1500_lock', self.__stats.job_class), SkillModifier(), [True, True]),)

    result = rb.get_skill_timing().get_q()
    result = [result[i][1:] for i in range(0, len(result))]
    return self._compare_sequential(result, expected)

  @TestClass.is_a_test
  def test_gcd_ogcd_with_animation_locks(self):
    rb = RotationBuilder(self.__stats, self.__skill_library)
    rb.add_next('test_gcd')
    rb.add_next('test_ogcd_animation_lock')
    rb.add_next('test_instant_gcd')
    rb.add_next('test_gcd_1500_lock')
    rb.add_next('test_ogcd_animation_lock')
    rb.add_next('test_ogcd_animation_lock')
    rb.add_next('test_gcd')

    expected = ((SnapshotAndApplicationEvents.EventTimes(1940, 2440), self.__skill_library.get_skill('test_gcd', self.__stats.job_class), SkillModifier(), [True, True]),
            (SnapshotAndApplicationEvents.EventTimes(2445, None), self.__skill_library.get_skill('test_ogcd_animation_lock', self.__stats.job_class), SkillModifier(), [True, True]),
            (SnapshotAndApplicationEvents.EventTimes(2480, None), self.__skill_library.get_skill('test_instant_gcd', self.__stats.job_class), SkillModifier(), [True, True]),
            (SnapshotAndApplicationEvents.EventTimes(5880, 6380), self.__skill_library.get_skill('test_gcd_1500_lock', self.__stats.job_class), SkillModifier(), [True, True]),
            (SnapshotAndApplicationEvents.EventTimes(6430, None), self.__skill_library.get_skill('test_ogcd_animation_lock', self.__stats.job_class), SkillModifier(), [True, True]),
            (SnapshotAndApplicationEvents.EventTimes(6465, None), self.__skill_library.get_skill('test_ogcd_animation_lock', self.__stats.job_class), SkillModifier(), [True, True]),
            (SnapshotAndApplicationEvents.EventTimes(9300, 9800), self.__skill_library.get_skill('test_gcd', self.__stats.job_class), SkillModifier(), [True, True]),)

    result = rb.get_skill_timing().get_q()
    result = [result[i][1:] for i in range(0, len(result))]
    return self._compare_sequential(result, expected)

  @TestClass.is_a_test
  def test_add_next_with_instants(self):
    rb = RotationBuilder(self.__stats, self.__skill_library)
    rb.add_next('test_gcd')
    rb.add_next('test_instant_gcd')
    rb.add_next('test_gcd')
    rb.add_next('test_gcd')

    expected = ((SnapshotAndApplicationEvents.EventTimes(1940, 2440), self.__skill_library.get_skill('test_gcd', self.__stats.job_class), SkillModifier(), [True, True]),
            (SnapshotAndApplicationEvents.EventTimes(2445, None), self.__skill_library.get_skill('test_instant_gcd', self.__stats.job_class), SkillModifier(), [True, True]),
            (SnapshotAndApplicationEvents.EventTimes(2*2440 + 1940 +5, 3*2440 + 5), self.__skill_library.get_skill('test_gcd', self.__stats.job_class), SkillModifier(), [True, True]),
            (SnapshotAndApplicationEvents.EventTimes(3*2440 + 1940 + 10, 4*2440 + 10), self.__skill_library.get_skill('test_gcd', self.__stats.job_class), SkillModifier(), [True, True]),)

    result = rb.get_skill_timing().get_q()
    result = [result[i][1:] for i in range(0, len(result))]
    return self._compare_sequential(result, expected)

  @TestClass.is_a_test
  def test_add_next_simple(self):
    rb = RotationBuilder(self.__stats, create_test_skill_library())
    rb.add_next('test_gcd')
    rb.add_next('test_gcd')
    rb.add_next('test_gcd')
    rb.add_next('test_gcd')

    expected = ((SnapshotAndApplicationEvents.EventTimes(1940, 2440), self.__skill_library.get_skill('test_gcd', self.__stats.job_class), SkillModifier(), [True, True]),
            (SnapshotAndApplicationEvents.EventTimes(2440+1940+5, 2*2440+5), self.__skill_library.get_skill('test_gcd', self.__stats.job_class), SkillModifier(), [True, True]),
            (SnapshotAndApplicationEvents.EventTimes(2*2440+1940 + 10, 3*2440+10), self.__skill_library.get_skill('test_gcd', self.__stats.job_class), SkillModifier(), [True, True]),
            (SnapshotAndApplicationEvents.EventTimes(3*2440+1940+15, 4*2440+15), self.__skill_library.get_skill('test_gcd', self.__stats.job_class), SkillModifier(), [True, True]),)

    result = rb.get_skill_timing().get_q()
    result = [result[i][1:] for i in range(0, len(result))]
    return self._compare_sequential(result, expected)

  @TestClass.is_a_test
  def test_autos_with_buff_on_follow_up(self):
    rb = RotationBuilder(self.__stats, self.__skill_library)
    rb.add(0.1, 'test_auto_attack_buff_on_follow_up') #comes out after first auto
    rb.add(2.54, 'test_instant_gcd')
    rb.add(4, 'test_simple_buff_gcd') #should do nothing
    rb.add(17.44, 'test_instant_gcd')

    expected = ((SnapshotAndApplicationEvents.EventTimes(0, None), self.__skill_library.get_skill('Auto', self.__stats.job_class), SkillModifier(), [True, True]),
            (SnapshotAndApplicationEvents.EventTimes(2040, 2540), self.__skill_library.get_skill('test_auto_attack_buff_on_follow_up', self.__stats.job_class), SkillModifier(), [True, True]),
            (SnapshotAndApplicationEvents.EventTimes(2040, 2540), self.__skill_library.get_skill('test_auto_attack_buff_instant_follow_up', self.__stats.job_class), SkillModifier(), [True, True]),
            (SnapshotAndApplicationEvents.EventTimes(2540, None), self.__skill_library.get_skill('test_instant_gcd', self.__stats.job_class), SkillModifier(), [True, True]),
            (SnapshotAndApplicationEvents.EventTimes(3440, None), self.__skill_library.get_skill('Auto', self.__stats.job_class), SkillModifier(), [True, True]),
            (SnapshotAndApplicationEvents.EventTimes(4000, None), self.__skill_library.get_skill('test_simple_buff_gcd', self.__stats.job_class), SkillModifier(), [True, True]),
            (SnapshotAndApplicationEvents.EventTimes(6020, None), self.__skill_library.get_skill('Auto', self.__stats.job_class), SkillModifier(), [True, True]),
            (SnapshotAndApplicationEvents.EventTimes(8600, None), self.__skill_library.get_skill('Auto', self.__stats.job_class), SkillModifier(), [True, True]),
            (SnapshotAndApplicationEvents.EventTimes(11180, None), self.__skill_library.get_skill('Auto', self.__stats.job_class), SkillModifier(), [True, True]),
            (SnapshotAndApplicationEvents.EventTimes(13760, None), self.__skill_library.get_skill('Auto', self.__stats.job_class), SkillModifier(), [True, True]),
            (SnapshotAndApplicationEvents.EventTimes(16340, None), self.__skill_library.get_skill('Auto', self.__stats.job_class), SkillModifier(), [True, True]),
            (SnapshotAndApplicationEvents.EventTimes(17440, None), self.__skill_library.get_skill('test_instant_gcd', self.__stats.job_class), SkillModifier(), [True, True]))

    result = rb.get_skill_timing(add_autos=True).get_q()
    result = [result[i][1:] for i in range(0, len(result))]
    return self._compare_sequential(result, expected)

  @TestClass.is_a_test
  def test_autos_with_casts(self):
    rb = RotationBuilder(self.__stats, self.__skill_library)
    rb.add(14.0, 'test_gcd') #will NOT auto
    rb.add(2.5, 'test_gcd') #will delay auto
    rb.add(8.0, 'test_gcd') #will delay auto

    expected = ((SnapshotAndApplicationEvents.EventTimes(0, None), self.__skill_library.get_skill('Auto', self.__stats.job_class), SkillModifier(), [True, True]),
                (SnapshotAndApplicationEvents.EventTimes(4440, 4940), self.__skill_library.get_skill('test_gcd', self.__stats.job_class), SkillModifier(), [True, True]),
                (SnapshotAndApplicationEvents.EventTimes(4940, None), self.__skill_library.get_skill('Auto', self.__stats.job_class), SkillModifier(), [True, True]),
                (SnapshotAndApplicationEvents.EventTimes(9940, 10440), self.__skill_library.get_skill('test_gcd', self.__stats.job_class), SkillModifier(), [True, True]),
                (SnapshotAndApplicationEvents.EventTimes(10440, None), self.__skill_library.get_skill('Auto', self.__stats.job_class), SkillModifier(), [True, True]),
                (SnapshotAndApplicationEvents.EventTimes(13880, None), self.__skill_library.get_skill('Auto', self.__stats.job_class), SkillModifier(), [True, True]),
                (SnapshotAndApplicationEvents.EventTimes(15940, 16440), self.__skill_library.get_skill('test_gcd', self.__stats.job_class), SkillModifier(), [True, True]))

    result = rb.get_skill_timing(add_autos=True).get_q()
    result = [result[i][1:] for i in range(0, len(result))]
    return self._compare_sequential(result, expected)

  @TestClass.is_a_test
  def test_autos_with_buffs(self):
    rb = RotationBuilder(self.__stats, self.__skill_library)
    rb.add(0.1, 'test_auto_attack_buff') #comes out after first auto
    rb.add(2.54, 'test_instant_gcd')
    rb.add(4, 'test_simple_buff_gcd') #should do nothing
    rb.add(17.44, 'test_instant_gcd')

    expected = ((SnapshotAndApplicationEvents.EventTimes(0, None), self.__skill_library.get_skill('Auto', self.__stats.job_class), SkillModifier(), [True, True]),
                (SnapshotAndApplicationEvents.EventTimes(2040, 2540), self.__skill_library.get_skill('test_auto_attack_buff', self.__stats.job_class), SkillModifier(), [True, True]),
                (SnapshotAndApplicationEvents.EventTimes(2540, None), self.__skill_library.get_skill('test_instant_gcd', self.__stats.job_class), SkillModifier(), [True, True]),
                (SnapshotAndApplicationEvents.EventTimes(3440, None), self.__skill_library.get_skill('Auto', self.__stats.job_class), SkillModifier(), [True, True]),
                (SnapshotAndApplicationEvents.EventTimes(4000, None), self.__skill_library.get_skill('test_simple_buff_gcd', self.__stats.job_class), SkillModifier(), [True, True]),
                (SnapshotAndApplicationEvents.EventTimes(6020, None), self.__skill_library.get_skill('Auto', self.__stats.job_class), SkillModifier(), [True, True]),
                (SnapshotAndApplicationEvents.EventTimes(8600, None), self.__skill_library.get_skill('Auto', self.__stats.job_class), SkillModifier(), [True, True]),
                (SnapshotAndApplicationEvents.EventTimes(11180, None), self.__skill_library.get_skill('Auto', self.__stats.job_class), SkillModifier(), [True, True]),
                (SnapshotAndApplicationEvents.EventTimes(13760, None), self.__skill_library.get_skill('Auto', self.__stats.job_class), SkillModifier(), [True, True]),
                (SnapshotAndApplicationEvents.EventTimes(16340, None), self.__skill_library.get_skill('Auto', self.__stats.job_class), SkillModifier(), [True, True]),
                (SnapshotAndApplicationEvents.EventTimes(17440, None), self.__skill_library.get_skill('test_instant_gcd', self.__stats.job_class), SkillModifier(), [True, True]))

    result = rb.get_skill_timing(add_autos=True).get_q()
    result = [result[i][1:] for i in range(0, len(result))]
    return self._compare_sequential(result, expected)

  @TestClass.is_a_test
  def test_autos(self):
    rb = RotationBuilder(self.__stats, self.__skill_library)
    rb.add(2.54, 'test_instant_gcd')
    rb.add(17.44, 'test_instant_gcd')

    expected = ((SnapshotAndApplicationEvents.EventTimes(0, None), self.__skill_library.get_skill('Auto', self.__stats.job_class), SkillModifier(), [True, True]),
                (SnapshotAndApplicationEvents.EventTimes(2540, None), self.__skill_library.get_skill('test_instant_gcd', self.__stats.job_class), SkillModifier(), [True, True]),
                (SnapshotAndApplicationEvents.EventTimes(3440, None), self.__skill_library.get_skill('Auto', self.__stats.job_class), SkillModifier(), [True, True]),
                (SnapshotAndApplicationEvents.EventTimes(6880, None), self.__skill_library.get_skill('Auto', self.__stats.job_class), SkillModifier(), [True, True]),
                (SnapshotAndApplicationEvents.EventTimes(10320, None), self.__skill_library.get_skill('Auto', self.__stats.job_class), SkillModifier(), [True, True]),
                (SnapshotAndApplicationEvents.EventTimes(13760, None), self.__skill_library.get_skill('Auto', self.__stats.job_class), SkillModifier(), [True, True]),
                (SnapshotAndApplicationEvents.EventTimes(17200, None), self.__skill_library.get_skill('Auto', self.__stats.job_class), SkillModifier(), [True, True]),
                (SnapshotAndApplicationEvents.EventTimes(17440, None), self.__skill_library.get_skill('test_instant_gcd', self.__stats.job_class), SkillModifier(), [True, True]))

    result = rb.get_skill_timing(add_autos=True).get_q()
    result = [result[i][1:] for i in range(0, len(result))]
    return self._compare_sequential(result, expected)

  @TestClass.is_a_test
  def pass_through_skill_modifier(self):
    rb = RotationBuilder(self.__stats, self.__skill_library)
    rb.add(0.2, 'test_gcd', SkillModifier(guaranteed_crit=ForcedCritOrDH.FORCE_YES))

    expected = ((SnapshotAndApplicationEvents.EventTimes(2140, 2640), self.__skill_library.get_skill('test_gcd', 'test_job'), SkillModifier(guaranteed_crit=ForcedCritOrDH.FORCE_YES), [True, True]),)

    result = rb.get_skill_timing().get_q()
    result = [result[i][1:] for i in range(0, len(result))]
    return self._compare_sequential(result, expected)

  @TestClass.is_a_test
  def simple_dot(self):
    test_passed = True
    err_msg = ""

    rb = RotationBuilder(self.__stats, self.__skill_library)
    rb.add(1.0, 'test_dot_gcd')

    expected = ((SnapshotAndApplicationEvents.EventTimes(2940, 3440), self.__skill_library.get_skill('test_dot_gcd', 'test_job'), SkillModifier(), [True, True]),
                (SnapshotAndApplicationEvents.EventTimes(2940, 3440), self.__skill_library.get_skill('test_dot_tick', 'test_job'), SkillModifier(), [True, True]),
                (SnapshotAndApplicationEvents.EventTimes(2940, 6440), self.__skill_library.get_skill('test_dot_tick', 'test_job'), SkillModifier(), [True, True]),
                (SnapshotAndApplicationEvents.EventTimes(2940, 9440), self.__skill_library.get_skill('test_dot_tick', 'test_job'), SkillModifier(), [True, True]),
                (SnapshotAndApplicationEvents.EventTimes(2940, 12440), self.__skill_library.get_skill('test_dot_tick', 'test_job'), SkillModifier(), [True, True]),
                (SnapshotAndApplicationEvents.EventTimes(2940, 15440), self.__skill_library.get_skill('test_dot_tick', 'test_job'), SkillModifier(), [True, True]))

    result = rb.get_skill_timing().get_q()
    result = [result[i][1:] for i in range(0, len(result))]
    return self._compare_sequential(result, expected)

  @TestClass.is_a_test
  def simple_ground_dot(self):
    test_passed = True
    err_msg = ""

    rb = RotationBuilder(self.__stats, self.__skill_library)
    rb.add(1.0, 'test_ground_dot_gcd')

    expected = ((SnapshotAndApplicationEvents.EventTimes(2940, 3440), self.__skill_library.get_skill('test_ground_dot_gcd', 'test_job'), SkillModifier(), [True, True]),
                (SnapshotAndApplicationEvents.EventTimes(2940, 3440), self.__skill_library.get_skill('test_dot_tick', 'test_job'), SkillModifier(), [True, False]),
                (SnapshotAndApplicationEvents.EventTimes(2940, 6440), self.__skill_library.get_skill('test_dot_tick', 'test_job'), SkillModifier(), [True, False]),
                (SnapshotAndApplicationEvents.EventTimes(2940, 9440), self.__skill_library.get_skill('test_dot_tick', 'test_job'), SkillModifier(), [True, False]),
                (SnapshotAndApplicationEvents.EventTimes(2940, 12440), self.__skill_library.get_skill('test_dot_tick', 'test_job'),SkillModifier(), [True, False]),
                (SnapshotAndApplicationEvents.EventTimes(2940, 15440), self.__skill_library.get_skill('test_dot_tick', 'test_job'), SkillModifier(), [True, False]))

    result = rb.get_skill_timing().get_q()
    result = [result[i][1:] for i in range(0, len(result))]
    return self._compare_sequential(result, expected)

  @TestClass.is_a_test
  def dot_with_early_refresh(self):
    test_passed = True
    err_msg = ""

    rb = RotationBuilder(self.__stats, self.__skill_library)
    rb.add(1.0, 'test_dot_gcd')
    rb.add(8.0, 'test_dot_gcd')

    expected = ((SnapshotAndApplicationEvents.EventTimes(2940, 3440), self.__skill_library.get_skill('test_dot_gcd', 'test_job'), SkillModifier(), [True, True]),
                (SnapshotAndApplicationEvents.EventTimes(2940, 3440), self.__skill_library.get_skill('test_dot_tick', 'test_job'), SkillModifier(), [True, True]),
                (SnapshotAndApplicationEvents.EventTimes(2940, 6440), self.__skill_library.get_skill('test_dot_tick', 'test_job'), SkillModifier(), [True, True]),
                (SnapshotAndApplicationEvents.EventTimes(2940, 9440), self.__skill_library.get_skill('test_dot_tick', 'test_job'), SkillModifier(), [True, True]),
                (SnapshotAndApplicationEvents.EventTimes(9940, 10440), self.__skill_library.get_skill('test_dot_gcd', 'test_job'), SkillModifier(), [True, True]),
                (SnapshotAndApplicationEvents.EventTimes(9940, 10440), self.__skill_library.get_skill('test_dot_tick', 'test_job'), SkillModifier(), [True, True]),
                (SnapshotAndApplicationEvents.EventTimes(9940, 13440), self.__skill_library.get_skill('test_dot_tick', 'test_job'), SkillModifier(), [True, True]),
                (SnapshotAndApplicationEvents.EventTimes(9940, 16440), self.__skill_library.get_skill('test_dot_tick', 'test_job'), SkillModifier(), [True, True]),
                (SnapshotAndApplicationEvents.EventTimes(9940, 19440), self.__skill_library.get_skill('test_dot_tick', 'test_job'), SkillModifier(), [True, True]),
                (SnapshotAndApplicationEvents.EventTimes(9940, 22440), self.__skill_library.get_skill('test_dot_tick', 'test_job'), SkillModifier(), [True, True]))

    result = rb.get_skill_timing().get_q()
    result = [result[i][1:] for i in range(0, len(result))]
    return self._compare_sequential(result, expected)

  @TestClass.is_a_test
  def dot_with_early_refresh_from_instant_dot_gcd(self):
    test_passed = True
    err_msg = ""

    rb = RotationBuilder(self.__stats, self.__skill_library)
    rb.add(1.0, 'test_dot_gcd')
    rb.add(8.0, 'test_dot_instant_gcd')

    expected = ((SnapshotAndApplicationEvents.EventTimes(2940, 3440), self.__skill_library.get_skill('test_dot_gcd', 'test_job'), SkillModifier(), [True, True]),
                (SnapshotAndApplicationEvents.EventTimes(2940, 3440), self.__skill_library.get_skill('test_dot_tick', 'test_job'), SkillModifier(), [True, True]),
                (SnapshotAndApplicationEvents.EventTimes(2940, 6440), self.__skill_library.get_skill('test_dot_tick', 'test_job'), SkillModifier(), [True, True]),
                (SnapshotAndApplicationEvents.EventTimes(8000, None), self.__skill_library.get_skill('test_dot_instant_gcd', 'test_job'), SkillModifier(), [True, True]),
                (SnapshotAndApplicationEvents.EventTimes(8000, None), self.__skill_library.get_skill('test_dot_tick', 'test_job'), SkillModifier(), [True, True]),
                (SnapshotAndApplicationEvents.EventTimes(8000, 11000), self.__skill_library.get_skill('test_dot_tick', 'test_job'), SkillModifier(), [True, True]),
                (SnapshotAndApplicationEvents.EventTimes(8000, 14000), self.__skill_library.get_skill('test_dot_tick', 'test_job'), SkillModifier(), [True, True]),
                (SnapshotAndApplicationEvents.EventTimes(8000, 17000), self.__skill_library.get_skill('test_dot_tick', 'test_job'), SkillModifier(), [True, True]),
                (SnapshotAndApplicationEvents.EventTimes(8000, 20000), self.__skill_library.get_skill('test_dot_tick', 'test_job'), SkillModifier(), [True, True]))

    result = rb.get_skill_timing().get_q()
    result = [result[i][1:] for i in range(0, len(result))]
    return self._compare_sequential(result, expected)

  @TestClass.is_a_test
  def dot_refresh_with_other_follow_up(self):
    # this is a more complex test because dot refreshes are more likely to mess up the ordering
    # of skills
    test_passed = True
    err_msg = ""

    rb = RotationBuilder(self.__stats, self.__skill_library)

    rb.add(1.0, 'test_dot_gcd_with_other_follow_up')
    rb.add(8.0, 'test_dot_gcd_with_other_follow_up')

    expected = ((SnapshotAndApplicationEvents.EventTimes(2940, 3440), self.__skill_library.get_skill('test_dot_gcd_with_other_follow_up', 'test_job'), SkillModifier(), [True, True]),
                (SnapshotAndApplicationEvents.EventTimes(2940, 3440), self.__skill_library.get_skill('test_non_dot_follow_up', 'test_job'), SkillModifier(), [True, True]),
                (SnapshotAndApplicationEvents.EventTimes(2940, 3440), self.__skill_library.get_skill('test_dot_tick', 'test_job'), SkillModifier(), [True, True]),
                (SnapshotAndApplicationEvents.EventTimes(2940, 6440), self.__skill_library.get_skill('test_dot_tick', 'test_job'), SkillModifier(), [True, True]),
                (SnapshotAndApplicationEvents.EventTimes(2940, 9440), self.__skill_library.get_skill('test_dot_tick', 'test_job'), SkillModifier(), [True, True]),
                (SnapshotAndApplicationEvents.EventTimes(9940, 10440), self.__skill_library.get_skill('test_dot_gcd_with_other_follow_up', 'test_job'), SkillModifier(), [True, True]),
                (SnapshotAndApplicationEvents.EventTimes(9940, 10440), self.__skill_library.get_skill('test_non_dot_follow_up', 'test_job'), SkillModifier(), [True, True]),
                (SnapshotAndApplicationEvents.EventTimes(9940, 10440), self.__skill_library.get_skill('test_dot_tick', 'test_job'), SkillModifier(), [True, True]),
                (SnapshotAndApplicationEvents.EventTimes(9940, 13440), self.__skill_library.get_skill('test_dot_tick', 'test_job'), SkillModifier(), [True, True]),
                (SnapshotAndApplicationEvents.EventTimes(9940, 16440), self.__skill_library.get_skill('test_dot_tick', 'test_job'), SkillModifier(), [True, True]),
                (SnapshotAndApplicationEvents.EventTimes(9940, 19440), self.__skill_library.get_skill('test_dot_tick', 'test_job'), SkillModifier(), [True, True]),
                (SnapshotAndApplicationEvents.EventTimes(9940, 22440), self.__skill_library.get_skill('test_dot_tick', 'test_job'), SkillModifier(), [True, True]))

    result = rb.get_skill_timing().get_q()
    result = [result[i][1:] for i in range(0, len(result))]
    return self._compare_sequential(result, expected)

  @TestClass.is_a_test
  def skill_follow_up_test_non_dot(self):
    test_passed = True
    err_msg = ""

    rb = RotationBuilder(self.__stats, self.__skill_library)
    rb.add(0, 'test_follow_up')

    expected = ((SnapshotAndApplicationEvents.EventTimes(1940, 2440), self.__skill_library.get_skill('test_follow_up', 'test_job'), SkillModifier(), [True, True]),
                (SnapshotAndApplicationEvents.EventTimes(1940, 2440), self.__skill_library.get_skill('test_non_dot_follow_up', 'test_job'), SkillModifier(), [True, True]),
                (SnapshotAndApplicationEvents.EventTimes(1940, 3000+2440), self.__skill_library.get_skill('test_non_dot_follow_up', 'test_job'), SkillModifier(), [False, True]),
                (SnapshotAndApplicationEvents.EventTimes(7000+2440, None), self.__skill_library.get_skill('test_non_dot_follow_up', 'test_job'), SkillModifier(), [True, True]))

    result = rb.get_skill_timing().get_q()
    result = [result[i][1:] for i in range(0, len(result))]
    return self._compare_sequential(result, expected)

  @TestClass.is_a_test
  def party_buff_application(self):
    test_passed = True
    err_msg=""

    # For a 2500 ms gcd, this spell speed should result in a 2440 sm (2.44s) GCD.
    rb = RotationBuilder(self.__stats, self.__skill_library)

    rb.add(0, 'test_party_buff', job_class='test_job2')
    rb.add(3, 'test_gcd')

    expected = ((SnapshotAndApplicationEvents.EventTimes(0, None), self.__skill_library.get_skill('test_party_buff', 'test_job2'), SkillModifier(), [True, True]),
                (SnapshotAndApplicationEvents.EventTimes(4940, 5440), self.__skill_library.get_skill('test_gcd', 'test_job'), SkillModifier(), [True, True]))

    result = rb.get_skill_timing().get_q()
    result = [result[i][1:] for i in range(0, len(result))]
    return self._compare_sequential(result, expected)

  @TestClass.is_a_test
  def skill_timing_test_no_buffs_no_dot(self):
    test_passed = True
    err_msg=""

    # For a 2500 ms gcd, this spell speed should result in a 2440 sm (2.44s) GCD.
    rb = RotationBuilder(self.__stats, self.__skill_library)
    rb.add(0, 'test_gcd')
    rb.add(3.5, 'test_gcd')
    rb.add(2.4, 'test_ogcd')
    rb.add(4.0, 'test_gcd_with_app_delay')

    expected = ((SnapshotAndApplicationEvents.EventTimes(1940, 2440), self.__skill_library.get_skill('test_gcd', 'test_job'), SkillModifier(), [True, True]),
                (SnapshotAndApplicationEvents.EventTimes(2400, None), self.__skill_library.get_skill('test_ogcd', 'test_job'), SkillModifier(), [True, True]),
                (SnapshotAndApplicationEvents.EventTimes(3500+1940, 3500+2440), self.__skill_library.get_skill('test_gcd', 'test_job'), SkillModifier(), [True, True]),
                (SnapshotAndApplicationEvents.EventTimes(4000+1940, 4000+2440+100), self.__skill_library.get_skill('test_gcd_with_app_delay', 'test_job'), SkillModifier(), [True, True]))

    result = rb.get_skill_timing().get_q()
    result = [result[i][1:] for i in range(0, len(result))]
    return self._compare_sequential(result, expected)

In [66]:
#@title DamageSimulatorTest
#TODO: add in tests that checks actual damage?
class DamageSimulatorTest(TestClass):
  def __init__(self):
    stats = Stats(wd=126, weapon_delay=3.44, main_stat=2945, det_stat=1620, crit_stat=2377, dh_stat=1048, speed_stat=708, job_class = 'test_job')
    self.__processed_stats = ProcessedStats(stats, job_class_fns=JobClassFnsTest)
    self.__skill_library = create_test_skill_library()

  @TestClass.is_a_test
  def test_trait_damage_mult_override(self):
    stats = Stats(wd=126, weapon_delay=3.44, main_stat=2945, det_stat=1620, crit_stat=2377, dh_stat=1048, speed_stat=708, job_class = 'test_job2')
    dmg_instances = [
                 (0, self.__skill_library.get_skill('test_gcd', 'test_job2'), SkillModifier(), (StatusEffects(), StatusEffects())),
                 (1000, self.__skill_library.get_skill('test_gcd_trait_override', 'test_job2'), SkillModifier(), (StatusEffects(), StatusEffects()))]

    sim = DamageSimulator(stats, dmg_instances, 2, job_class_fns=JobClassFnsTest)
    expected_trait_damage_mult = [1.4, 1.0]
    actual_trait_damage_mult = sim.get_trait_damage_mult()
    return self._compare_sequential(actual_trait_damage_mult, expected_trait_damage_mult)

  @TestClass.is_a_test
  def test_guaranteed_crit_dh_dot(self):
    dmg_instances = [
                 (0, self.__skill_library.get_skill('test_guaranteed_crit_dot', 'test_job'), SkillModifier(), (StatusEffects(), StatusEffects())),
                 (1000, self.__skill_library.get_skill('test_guaranteed_dh_dot', 'test_job'), SkillModifier(), (StatusEffects(), StatusEffects())),
                 (2000, self.__skill_library.get_skill('test_guaranteed_crit_dh_dot', 'test_job'), SkillModifier(), (StatusEffects(), StatusEffects()))]

    sim = DamageSimulator(self.__processed_stats.stats, dmg_instances, 2, job_class_fns=JobClassFnsTest)
    expected_crit = [1, self.__processed_stats.crit_rate, 1]
    expected_dh = [self.__processed_stats.dh_rate, 1, 1]
    (actual_crit, actual_dh) = sim.get_crit_and_dh_rates()
    (test_passed_crit, err_msg_crit) = self._compare_sequential(actual_crit, expected_crit)
    (test_passed_dh, err_msg_dh) = self._compare_sequential(actual_dh, expected_dh)
    return test_passed_crit and test_passed_dh, err_msg_crit+err_msg_dh

  @TestClass.is_a_test
  def test_guaranteed_no_crit_dh(self):
    dmg_instances = [
                 (0, self.__skill_library.get_skill('test_gcd', 'test_job'), SkillModifier(), (StatusEffects(), StatusEffects())),
                 (2000, self.__skill_library.get_skill('test_guaranteed_no_crit_dh', 'test_job'), SkillModifier(), (StatusEffects(), StatusEffects()))]

    sim = DamageSimulator(self.__processed_stats.stats, dmg_instances, 2, job_class_fns=JobClassFnsTest)
    expected_crit = [self.__processed_stats.crit_rate, 0]
    expected_dh = [self.__processed_stats.dh_rate, 0]
    (actual_crit, actual_dh) = sim.get_crit_and_dh_rates()
    (test_passed_crit, err_msg_crit) = self._compare_sequential(actual_crit, expected_crit)
    (test_passed_dh, err_msg_dh) = self._compare_sequential(actual_dh, expected_dh)
    return test_passed_crit and test_passed_dh, err_msg_crit+err_msg_dh

  @TestClass.is_a_test
  def test_guaranteed_crit_dh(self):
    dmg_instances = [
                 (0, self.__skill_library.get_skill('test_guaranteed_crit', 'test_job'), SkillModifier(), (StatusEffects(), StatusEffects())),
                 (1000, self.__skill_library.get_skill('test_guaranteed_dh', 'test_job'), SkillModifier(), (StatusEffects(), StatusEffects())),
                 (2000, self.__skill_library.get_skill('test_guaranteed_crit_dh', 'test_job'), SkillModifier(), (StatusEffects(), StatusEffects()))]

    sim = DamageSimulator(self.__processed_stats.stats, dmg_instances, 2, job_class_fns=JobClassFnsTest)
    expected_crit = [1, self.__processed_stats.crit_rate, 1]
    expected_dh = [self.__processed_stats.dh_rate, 1, 1]
    (actual_crit, actual_dh) = sim.get_crit_and_dh_rates()
    (test_passed_crit, err_msg_crit) = self._compare_sequential(actual_crit, expected_crit)
    (test_passed_dh, err_msg_dh) = self._compare_sequential(actual_dh, expected_dh)
    return test_passed_crit and test_passed_dh, err_msg_crit+err_msg_dh

  @TestClass.is_a_test
  def test_guaranteed_crit_dh_with_skill_modifier(self):
    dmg_instances = [
                 (0, self.__skill_library.get_skill('test_gcd', 'test_job'), SkillModifier(guaranteed_crit=ForcedCritOrDH.FORCE_YES), (StatusEffects(), StatusEffects())),
                 (1000, self.__skill_library.get_skill('test_gcd', 'test_job'), SkillModifier(guaranteed_dh=ForcedCritOrDH.FORCE_YES), (StatusEffects(), StatusEffects())),
                 (2000, self.__skill_library.get_skill('test_gcd', 'test_job'), SkillModifier(guaranteed_crit=ForcedCritOrDH.FORCE_YES, guaranteed_dh=ForcedCritOrDH.FORCE_YES), (StatusEffects(), StatusEffects()))]

    sim = DamageSimulator(self.__processed_stats.stats, dmg_instances, 2, job_class_fns=JobClassFnsTest)
    expected_crit = [1, self.__processed_stats.crit_rate, 1]
    expected_dh = [self.__processed_stats.dh_rate, 1, 1]
    (actual_crit, actual_dh) = sim.get_crit_and_dh_rates()
    (test_passed_crit, err_msg_crit) = self._compare_sequential(actual_crit, expected_crit)
    (test_passed_dh, err_msg_dh) = self._compare_sequential(actual_dh, expected_dh)
    return test_passed_crit and test_passed_dh, err_msg_crit+err_msg_dh

In [67]:
#@title ComputeDamageUtilsTest

class ComputeDamageUtilsTest(TestClass):
  def __init__(self):
    stats = Stats(wd=126, weapon_delay=3.44, main_stat=2945, det_stat=1620, crit_stat=2377, dh_stat=1048, speed_stat=708, job_class = 'test_job')
    self.__processed_stats = ProcessedStats(stats, job_class_fns=JobClassFnsTest)
    self.__skill_library = create_test_skill_library()

  @TestClass.is_a_test
  def test_pet_tank(self):
    test_passed = True
    err_msg = ""

    stats = Stats(wd=126, weapon_delay=2.96, main_stat=2906, det_stat=1883, crit_stat=2352, dh_stat=868, speed_stat=650, tenacity=631, job_class = 'test_tank_job')
    processed_stats = ProcessedStats(stats, job_class_fns=JobClassFnsTest)
    skill = self.__skill_library.get_skill('test_pet_gcd', 'test_tank_job')
    skill_modifier = SkillModifier()
    actual_base_damage = ComputeDamageUtils.get_base_damage(skill,
                                                            skill_modifier,
                                                            processed_stats,
                                                            (StatusEffects(), StatusEffects()),
                                                            job_class_fns=JobClassFnsTest)
    expected_base_damage = 8745.0
    if not math.isclose(actual_base_damage, expected_base_damage, abs_tol=1e-4):
      test_passed = False
      err_msg = "Base damage is not close. Expected: {}. Actual: {}.".format(expected_base_damage, actual_base_damage)
    return test_passed, err_msg

  @TestClass.is_a_test
  def test_pet_non_tank(self):
    test_passed = True
    err_msg = ""

    skill = self.__skill_library.get_skill('test_pet_gcd', 'test_job')
    skill_modifier = SkillModifier()
    actual_base_damage = ComputeDamageUtils.get_base_damage(skill,
                                                            skill_modifier,
                                                            self.__processed_stats,
                                                            (StatusEffects(), StatusEffects()),
                                                            job_class_fns=JobClassFnsTest)
    expected_base_damage = 8654.0
    if not math.isclose(actual_base_damage, expected_base_damage, abs_tol=1e-4):
      test_passed = False
      err_msg = "Base damage is not close. Expected: {}. Actual: {}.".format(expected_base_damage, actual_base_damage)
    return test_passed, err_msg

  @TestClass.is_a_test
  def test_guaranteed_no_crit_no_dh(self):
    test_passed = True
    err_msg = ""

    stats = Stats(wd=126, weapon_delay=3.36, main_stat=2910, det_stat=1980, crit_stat=2313, dh_stat=868, speed_stat=592, tenacity=631, job_class = 'test_job')
    processed_stats =  ProcessedStats(stats, job_class_fns=JobClassFnsTest)
    skill = self.__skill_library.get_skill('test_guaranteed_no_crit_dh', 'test_job')
    skill_modifier = SkillModifier()
    actual_base_damage = ComputeDamageUtils.get_base_damage(skill,
                                                            skill_modifier,
                                                            processed_stats,
                                                            (StatusEffects(crit_rate_add=0.25, dh_rate_add=0.2), StatusEffects(crit_rate_add=0.07, dh_rate_add=0.12)),
                                                            job_class_fns=JobClassFnsTest)
    expected_base_damage = 17945.0
    if not math.isclose(actual_base_damage, expected_base_damage, abs_tol=1e-4):
      test_passed = False
      err_msg = "Base damage is not close. Expected: {}. Actual: {}.".format(expected_base_damage, actual_base_damage)
    return test_passed, err_msg

  @TestClass.is_a_test
  def test_autos(self):
    test_passed = True
    err_msg = ""

    stats = Stats(wd=126, weapon_delay=3.36, main_stat=2910, det_stat=1980, crit_stat=2313, dh_stat=868, speed_stat=592, tenacity=631, job_class = 'test_job2')
    processed_stats =  ProcessedStats(stats, job_class_fns=JobClassFnsTest)
    skill = self.__skill_library.get_skill('Auto', 'test_job2')
    skill_modifier = SkillModifier()
    actual_base_damage = ComputeDamageUtils.get_base_damage(skill,
                                                            skill_modifier,
                                                            processed_stats,
                                                            (StatusEffects(), StatusEffects()),
                                                            job_class_fns=JobClassFnsTest)
    expected_base_damage = 3296.0
    if not math.isclose(actual_base_damage, expected_base_damage, abs_tol=1e-4):
      test_passed = False
      err_msg = "Base damage is not close. Expected: {}. Actual: {}.".format(expected_base_damage, actual_base_damage)
    return test_passed, err_msg

  @TestClass.is_a_test
  def test_tank_autos(self):
    test_passed = True
    err_msg = ""

    stats = Stats(wd=126, weapon_delay=3.36, main_stat=2910, det_stat=1980, crit_stat=2313, dh_stat=868, speed_stat=592, tenacity=631, job_class = 'test_tank_job')
    processed_stats =  ProcessedStats(stats, job_class_fns=JobClassFnsTest)
    skill = self.__skill_library.get_skill('Auto', 'test_tank_job')
    skill_modifier = SkillModifier()
    actual_base_damage = ComputeDamageUtils.get_base_damage(skill,
                                                            skill_modifier,
                                                            processed_stats,
                                                            (StatusEffects(), StatusEffects()),
                                                            job_class_fns=JobClassFnsTest)
    expected_base_damage = 2623.0
    if not math.isclose(actual_base_damage, expected_base_damage, abs_tol=1e-4):
      test_passed = False
      err_msg = "Base damage is not close. Expected: {}. Actual: {}.".format(expected_base_damage, actual_base_damage)
    return test_passed, err_msg

  @TestClass.is_a_test
  def test_healer_autos(self):
    test_passed = True
    err_msg = ""

    stats = Stats(wd=126, weapon_delay=3.12, main_stat=2947, det_stat=1695, crit_stat=2255, dh_stat=904, speed_stat=839, job_class = 'test_healer_job', healer_or_caster_strength=351)
    processed_stats =  ProcessedStats(stats, job_class_fns=JobClassFnsTest)
    skill = self.__skill_library.get_skill('Auto', 'test_healer_job')
    skill_modifier = SkillModifier()
    actual_base_damage = ComputeDamageUtils.get_base_damage(skill,
                                                            skill_modifier,
                                                            processed_stats,
                                                            (StatusEffects(), StatusEffects()),
                                                            job_class_fns=JobClassFnsTest)
    expected_base_damage = 126.0
    if not math.isclose(actual_base_damage, expected_base_damage, abs_tol=1e-4):
      test_passed = False
      err_msg = "Base damage is not close. Expected: {}. Actual: {}.".format(expected_base_damage, actual_base_damage)
    return test_passed, err_msg

  @TestClass.is_a_test
  def test_guaranteed_crit_dh_dot_with_buffs(self):
    test_passed = True
    err_msg = ""

    stats = Stats(wd=126, weapon_delay=3.36, main_stat=2910, det_stat=1980, crit_stat=2313, dh_stat=868, speed_stat=592, tenacity=631, job_class = 'test_job')
    processed_stats =  ProcessedStats(stats, job_class_fns=JobClassFnsTest)
    skill = self.__skill_library.get_skill('test_guaranteed_crit_dh_dot', 'test_job')
    skill_modifier = SkillModifier()
    actual_base_damage = ComputeDamageUtils.get_base_damage(skill,
                                                            skill_modifier,
                                                            processed_stats,
                                                            (StatusEffects(crit_rate_add=0.25, dh_rate_add=0.2), StatusEffects(crit_rate_add=0.07, dh_rate_add=0.12)),
                                                            job_class_fns=JobClassFnsTest)
    expected_base_damage = 1804.0
    if not math.isclose(actual_base_damage, expected_base_damage, abs_tol=1e-4):
      test_passed = False
      err_msg = "Base damage is not close. Expected: {}. Actual: {}.".format(expected_base_damage, actual_base_damage)
    return test_passed, err_msg

  @TestClass.is_a_test
  def test_guaranteed_crit_dh_dot_no_buffs(self):
    test_passed = True
    err_msg = ""

    stats = Stats(wd=126, weapon_delay=3.36, main_stat=2910, det_stat=1980, crit_stat=2313, dh_stat=868, speed_stat=592, tenacity=631, job_class = 'test_job')
    processed_stats =  ProcessedStats(stats, job_class_fns=JobClassFnsTest)
    skill = self.__skill_library.get_skill('test_guaranteed_crit_dh_dot', 'test_job')
    skill_modifier = SkillModifier()
    actual_base_damage = ComputeDamageUtils.get_base_damage(skill,
                                                            skill_modifier,
                                                            processed_stats,
                                                            (StatusEffects(), StatusEffects()),
                                                            job_class_fns=JobClassFnsTest)
    expected_base_damage = 1419.0
    if not math.isclose(actual_base_damage, expected_base_damage, abs_tol=1e-4):
      test_passed = False
      err_msg = "Base damage is not close. Expected: {}. Actual: {}.".format(expected_base_damage, actual_base_damage)
    return test_passed, err_msg

  @TestClass.is_a_test
  def test_guaranteed_crit_dot_with_buffs(self):
    test_passed = True
    err_msg = ""

    stats = Stats(wd=126, weapon_delay=3.36, main_stat=2910, det_stat=1980, crit_stat=2313, dh_stat=868, speed_stat=592, tenacity=631, job_class = 'test_job')
    processed_stats =  ProcessedStats(stats, job_class_fns=JobClassFnsTest)
    skill = self.__skill_library.get_skill('test_guaranteed_crit_dot', 'test_job')
    skill_modifier = SkillModifier()
    actual_base_damage = ComputeDamageUtils.get_base_damage(skill,
                                                            skill_modifier,
                                                            processed_stats,
                                                            (StatusEffects(crit_rate_add=0.32), StatusEffects()),
                                                            job_class_fns=JobClassFnsTest)
    expected_base_damage = 1640.0
    if not math.isclose(actual_base_damage, expected_base_damage, abs_tol=1e-4):
      test_passed = False
      err_msg = "Base damage is not close. Expected: {}. Actual: {}.".format(expected_base_damage, actual_base_damage)
    return test_passed, err_msg

  @TestClass.is_a_test
  def test_guaranteed_crit_dot_no_buffs(self):
    test_passed = True
    err_msg = ""

    stats = Stats(wd=126, weapon_delay=3.36, main_stat=2910, det_stat=1980, crit_stat=2313, dh_stat=868, speed_stat=592, tenacity=631, job_class = 'test_job')
    processed_stats =  ProcessedStats(stats, job_class_fns=JobClassFnsTest)
    skill = self.__skill_library.get_skill('test_guaranteed_crit_dot', 'test_job')
    skill_modifier = SkillModifier()
    actual_base_damage = ComputeDamageUtils.get_base_damage(skill,
                                                            skill_modifier,
                                                            processed_stats,
                                                            (StatusEffects(), StatusEffects()),
                                                            job_class_fns=JobClassFnsTest)
    expected_base_damage = 1376.0
    if not math.isclose(actual_base_damage, expected_base_damage, abs_tol=1e-4):
      test_passed = False
      err_msg = "Base damage is not close. Expected: {}. Actual: {}.".format(expected_base_damage, actual_base_damage)
    return test_passed, err_msg

  @TestClass.is_a_test
  def test_guaranteed_dh_dot_with_buffs(self):
    test_passed = True
    err_msg = ""

    stats = Stats(wd=126, weapon_delay=3.36, main_stat=2910, det_stat=1980, crit_stat=2313, dh_stat=868, speed_stat=592, tenacity=631, job_class = 'test_job')
    processed_stats =  ProcessedStats(stats, job_class_fns=JobClassFnsTest)
    skill = self.__skill_library.get_skill('test_guaranteed_dh_dot', 'test_job')
    skill_modifier = SkillModifier()
    actual_base_damage = ComputeDamageUtils.get_base_damage(skill,
                                                            skill_modifier,
                                                            processed_stats,
                                                            (StatusEffects(), StatusEffects(dh_rate_add=0.32)),
                                                            job_class_fns=JobClassFnsTest)
    expected_base_damage = 1532.0
    if not math.isclose(actual_base_damage, expected_base_damage, abs_tol=1e-4):
      test_passed = False
      err_msg = "Base damage is not close. Expected: {}. Actual: {}.".format(expected_base_damage, actual_base_damage)
    return test_passed, err_msg

  @TestClass.is_a_test
  def test_guaranteed_dh_dot_no_buffs(self):
    test_passed = True
    err_msg = ""

    stats = Stats(wd=126, weapon_delay=3.36, main_stat=2910, det_stat=1980, crit_stat=2313, dh_stat=868, speed_stat=592, tenacity=631, job_class = 'test_job')
    processed_stats =  ProcessedStats(stats, job_class_fns=JobClassFnsTest)
    skill = self.__skill_library.get_skill('test_guaranteed_dh_dot', 'test_job')
    skill_modifier = SkillModifier()
    actual_base_damage = ComputeDamageUtils.get_base_damage(skill,
                                                            skill_modifier,
                                                            processed_stats,
                                                            (StatusEffects(), StatusEffects()),
                                                            job_class_fns=JobClassFnsTest)
    expected_base_damage = 1419.0
    if not math.isclose(actual_base_damage, expected_base_damage, abs_tol=1e-4):
      test_passed = False
      err_msg = "Base damage is not close. Expected: {}. Actual: {}.".format(expected_base_damage, actual_base_damage)
    return test_passed, err_msg

  @TestClass.is_a_test
  def test_guaranteed_dh_with_buffs(self):
    test_passed = True
    err_msg = ""

    stats = Stats(wd=126, weapon_delay=3.36, main_stat=2910, det_stat=1980, crit_stat=2313, dh_stat=868, speed_stat=592, tenacity=631, job_class = 'test_job')
    processed_stats =  ProcessedStats(stats, job_class_fns=JobClassFnsTest)
    skill = self.__skill_library.get_skill('test_guaranteed_dh', 'test_job')
    skill_modifier = SkillModifier()
    actual_base_damage = ComputeDamageUtils.get_base_damage(skill,
                                                            skill_modifier,
                                                            processed_stats,
                                                            (StatusEffects(dh_rate_add=0.2), StatusEffects(dh_rate_add=0.12)),
                                                            job_class_fns=JobClassFnsTest)
    expected_base_damage = 19971.0
    if not math.isclose(actual_base_damage, expected_base_damage, abs_tol=1e-4):
      test_passed = False
      err_msg = "Base damage is not close. Expected: {}. Actual: {}.".format(expected_base_damage, actual_base_damage)
    return test_passed, err_msg

  @TestClass.is_a_test
  def test_guaranteed_crit_with_buffs(self):
    test_passed = True
    err_msg = ""

    stats = Stats(wd=126, weapon_delay=3.36, main_stat=2910, det_stat=1980, crit_stat=2313, dh_stat=868, speed_stat=592, tenacity=631, job_class = 'test_job')
    processed_stats =  ProcessedStats(stats, job_class_fns=JobClassFnsTest)
    skill = self.__skill_library.get_skill('test_guaranteed_crit', 'test_job')
    skill_modifier = SkillModifier()
    actual_base_damage = ComputeDamageUtils.get_base_damage(skill,
                                                            skill_modifier,
                                                            processed_stats,
                                                            (StatusEffects(crit_rate_add=0.2), StatusEffects(crit_rate_add=0.12)),
                                                            job_class_fns=JobClassFnsTest)
    expected_base_damage = 21396.0
    if not math.isclose(actual_base_damage, expected_base_damage, abs_tol=1e-4):
      test_passed = False
      err_msg = "Base damage is not close. Expected: {}. Actual: {}.".format(expected_base_damage, actual_base_damage)
    return test_passed, err_msg

  @TestClass.is_a_test
  def test_guaranteed_crit_no_buffs(self):
    test_passed = True
    err_msg = ""

    stats = Stats(wd=126, weapon_delay=3.36, main_stat=2910, det_stat=1980, crit_stat=2313, dh_stat=868, speed_stat=592, tenacity=631, job_class = 'test_job')
    processed_stats =  ProcessedStats(stats, job_class_fns=JobClassFnsTest)
    skill = self.__skill_library.get_skill('test_guaranteed_crit', 'test_job')
    skill_modifier = SkillModifier()
    actual_base_damage = ComputeDamageUtils.get_base_damage(skill,
                                                            skill_modifier,
                                                            processed_stats,
                                                            (StatusEffects(), StatusEffects()),
                                                            job_class_fns=JobClassFnsTest)
    expected_base_damage = 17945.0
    if not math.isclose(actual_base_damage, expected_base_damage, abs_tol=1e-4):
      test_passed = False
      err_msg = "Base damage is not close. Expected: {}. Actual: {}.".format(expected_base_damage, actual_base_damage)
    return test_passed, err_msg

  @TestClass.is_a_test
  def test_guaranteed_dh_no_buffs(self):
    test_passed = True
    err_msg = ""

    stats = Stats(wd=126, weapon_delay=3.36, main_stat=2910, det_stat=1980, crit_stat=2313, dh_stat=868, speed_stat=592, tenacity=631, job_class = 'test_job')
    processed_stats =  ProcessedStats(stats, job_class_fns=JobClassFnsTest)
    skill = self.__skill_library.get_skill('test_guaranteed_dh', 'test_job')
    skill_modifier = SkillModifier()
    actual_base_damage = ComputeDamageUtils.get_base_damage(skill,
                                                            skill_modifier,
                                                            processed_stats,
                                                            (StatusEffects(), StatusEffects()),
                                                            job_class_fns=JobClassFnsTest)
    expected_base_damage = 18492.0
    if not math.isclose(actual_base_damage, expected_base_damage, abs_tol=1e-4):
      test_passed = False
      err_msg = "Base damage is not close. Expected: {}. Actual: {}.".format(expected_base_damage, actual_base_damage)
    return test_passed, err_msg

  @TestClass.is_a_test
  def test_guaranteed_crit_dh_with_buffs(self):
    test_passed = True
    err_msg = ""

    stats = Stats(wd=126, weapon_delay=3.36, main_stat=2910, det_stat=1980, crit_stat=2313, dh_stat=868, speed_stat=592, tenacity=631, job_class = 'test_job')
    processed_stats =  ProcessedStats(stats, job_class_fns=JobClassFnsTest)
    skill = self.__skill_library.get_skill('test_guaranteed_crit_dh', 'test_job')
    skill_modifier = SkillModifier()
    actual_base_damage = ComputeDamageUtils.get_base_damage(skill,
                                                            skill_modifier,
                                                            processed_stats,
                                                            (StatusEffects(crit_rate_add=0.32), StatusEffects(dh_rate_add=0.32)),
                                                            job_class_fns=JobClassFnsTest)
    expected_base_damage = 23527.0
    if not math.isclose(actual_base_damage, expected_base_damage, abs_tol=1e-4):
      test_passed = False
      err_msg = "Base damage is not close. Expected: {}. Actual: {}.".format(expected_base_damage, actual_base_damage)
    return test_passed, err_msg

  @TestClass.is_a_test
  def test_guaranteed_crit_dh_skill_modifier_with_buffs(self):
    test_passed = True
    err_msg = ""

    stats = Stats(wd=126, weapon_delay=3.36, main_stat=2910, det_stat=1980, crit_stat=2313, dh_stat=868, speed_stat=592, tenacity=631, job_class = 'test_job')
    processed_stats =  ProcessedStats(stats, job_class_fns=JobClassFnsTest)
    skill = self.__skill_library.get_skill('test_gcd', 'test_job')
    skill_modifier = SkillModifier(guaranteed_crit=ForcedCritOrDH.FORCE_YES, guaranteed_dh=ForcedCritOrDH.FORCE_YES)
    actual_base_damage = ComputeDamageUtils.get_base_damage(skill,
                                                            skill_modifier,
                                                            processed_stats,
                                                            (StatusEffects(crit_rate_add=0.32), StatusEffects(dh_rate_add=0.32)),
                                                            job_class_fns=JobClassFnsTest)
    expected_base_damage = 23527.0
    if not math.isclose(actual_base_damage, expected_base_damage, abs_tol=1e-4):
      test_passed = False
      err_msg = "Base damage is not close. Expected: {}. Actual: {}.".format(expected_base_damage, actual_base_damage)
    return test_passed, err_msg

  @TestClass.is_a_test
  def test_guaranteed_crit_skill_modifier_with_buffs(self):
    test_passed = True
    err_msg = ""

    stats = Stats(wd=126, weapon_delay=3.36, main_stat=2910, det_stat=1980, crit_stat=2313, dh_stat=868, speed_stat=592, tenacity=631, job_class = 'test_job')
    processed_stats =  ProcessedStats(stats, job_class_fns=JobClassFnsTest)
    skill = self.__skill_library.get_skill('test_gcd', 'test_job')
    skill_modifier = SkillModifier(guaranteed_crit=ForcedCritOrDH.FORCE_YES)
    actual_base_damage = ComputeDamageUtils.get_base_damage(skill,
                                                            skill_modifier,
                                                            processed_stats,
                                                            (StatusEffects(crit_rate_add=0.2), StatusEffects(crit_rate_add=0.12)),
                                                            job_class_fns=JobClassFnsTest)
    expected_base_damage = 21396.0
    if not math.isclose(actual_base_damage, expected_base_damage, abs_tol=1e-4):
      test_passed = False
      err_msg = "Base damage is not close. Expected: {}. Actual: {}.".format(expected_base_damage, actual_base_damage)
    return test_passed, err_msg

  @TestClass.is_a_test
  def test_guaranteed_dh_skill_modifier_no_buffs(self):
    test_passed = True
    err_msg = ""

    stats = Stats(wd=126, weapon_delay=3.36, main_stat=2910, det_stat=1980, crit_stat=2313, dh_stat=868, speed_stat=592, tenacity=631, job_class = 'test_job')
    processed_stats =  ProcessedStats(stats, job_class_fns=JobClassFnsTest)
    skill = self.__skill_library.get_skill('test_gcd', 'test_job')
    skill_modifier = SkillModifier(guaranteed_dh=ForcedCritOrDH.FORCE_YES)
    actual_base_damage = ComputeDamageUtils.get_base_damage(skill,
                                                            skill_modifier,
                                                            processed_stats,
                                                            (StatusEffects(), StatusEffects()),
                                                            job_class_fns=JobClassFnsTest)
    expected_base_damage = 18492.0
    if not math.isclose(actual_base_damage, expected_base_damage, abs_tol=1e-4):
      test_passed = False
      err_msg = "Base damage is not close. Expected: {}. Actual: {}.".format(expected_base_damage, actual_base_damage)
    return test_passed, err_msg

  @TestClass.is_a_test
  def test_base_damage(self):
    test_passed = True
    err_msg = ""

    skill = self.__skill_library.get_skill('test_gcd', self.__processed_stats.stats.job_class)
    skill_modifier = SkillModifier()
    actual_base_damage = ComputeDamageUtils.get_base_damage(skill,
                                                            skill_modifier,
                                                            self.__processed_stats,
                                                            (StatusEffects(), StatusEffects()),
                                                            job_class_fns=JobClassFnsTest)
    expected_base_damage = 17742.0

    if not math.isclose(actual_base_damage, expected_base_damage, abs_tol=1e-4):
      test_passed = False
      err_msg = "Base damage is not close. Expected: {}. Actual: {}.".format(expected_base_damage, actual_base_damage)
    return test_passed, err_msg

  @TestClass.is_a_test
  def test_dot_tick_damage(self):
    test_passed = True
    err_msg = ""

    skill = self.__skill_library.get_skill('test_dot_tick', self.__processed_stats.stats.job_class)
    skill_modifier = SkillModifier()
    actual_base_damage = ComputeDamageUtils.get_base_damage(skill,
                                                            skill_modifier,
                                                            self.__processed_stats,
                                                            (StatusEffects(), StatusEffects()),
                                                            job_class_fns=JobClassFnsTest)
    expected_base_damage = 1920.0

    if not math.isclose(actual_base_damage, expected_base_damage, abs_tol=1e-4):
      test_passed = False
      err_msg = "Base damage is not close. Expected: {}. Actual: {}.".format(expected_base_damage, actual_base_damage)
    return test_passed, err_msg

  @TestClass.is_a_test
  def test_tank_dot_tick_damage(self):
    test_passed = True
    err_msg = ""

    stats = Stats(wd=126, weapon_delay=2.96, main_stat=2906, det_stat=1883, crit_stat=2352, dh_stat=868, speed_stat=650, tenacity=631, job_class = 'test_tank_job')
    processed_stats = ProcessedStats(stats, job_class_fns=JobClassFnsTest)

    skill = self.__skill_library.get_skill('test_tank_dot_tick', processed_stats.stats.job_class)
    skill_modifier = SkillModifier()
    actual_base_damage = ComputeDamageUtils.get_base_damage(skill,
                                                            skill_modifier,
                                                            processed_stats,
                                                            (StatusEffects(), StatusEffects()),
                                                            job_class_fns=JobClassFnsTest)
    expected_base_damage = 1013.0

    if not math.isclose(actual_base_damage, expected_base_damage, abs_tol=1e-4):
      test_passed = False
      err_msg = "Base damage is not close. Expected: {}. Actual: {}.".format(expected_base_damage, actual_base_damage)
    return test_passed, err_msg

  @TestClass.is_a_test
  def test_tank_base_damage(self):
    test_passed = True
    err_msg = ""

    stats = Stats(wd=126, weapon_delay=3.36, main_stat=2910, det_stat=1980, crit_stat=2313, dh_stat=868, speed_stat=592, tenacity=631, job_class = 'test_tank_job')
    processed_stats = ProcessedStats(stats, job_class_fns=JobClassFnsTest)

    skill = self.__skill_library.get_skill('test_tank_gcd', processed_stats.stats.job_class)
    skill_modifier = SkillModifier()
    actual_base_damage = ComputeDamageUtils.get_base_damage(skill,
                                                            skill_modifier,
                                                            processed_stats,
                                                            (StatusEffects(), StatusEffects()),
                                                            job_class_fns=JobClassFnsTest)
    expected_base_damage = 4316.0
    if not math.isclose(actual_base_damage, expected_base_damage, abs_tol=1e-4):
      test_passed = False
      err_msg = "Base damage is not close. Expected: {}. Actual: {}.".format(expected_base_damage, actual_base_damage)
    return test_passed, err_msg

In [68]:
#@title DamageBuilderTest
class DamageBuilderTest(TestClass):
  def __init__(self):
    self.__stats = Stats(wd=126, weapon_delay=3.44, main_stat=2945, det_stat=1620, crit_stat=2377, dh_stat=1048, speed_stat=708, job_class = 'test_job')
    self.__skill_library = create_test_skill_library()

  @TestClass.is_a_test
  def status_effect_denylist(self):
    rb_result = SnapshotAndApplicationEvents()
    rb_result.add(0, 0, 0, self.__skill_library.get_skill('test_simple_buff_gcd', 'test_job'), SkillModifier(), [True, True])
    rb_result.add(0, 0, 0, self.__skill_library.get_skill('test_simple_buff_gcd_2', 'test_job'), SkillModifier(), [True, True])
    rb_result.add(0, 0, 0, self.__skill_library.get_skill('test_simple_debuff_gcd', 'test_job'), SkillModifier(), [True, True])
    rb_result.add(0, 0, 0, self.__skill_library.get_skill('test_simple_debuff_gcd_2', 'test_job'), SkillModifier(), [True, True])
    rb_result.add(Utils.transform_time_to_prio(2940)+1, 2940, 3440, self.__skill_library.get_skill('test_gcd_with_denylist', 'test_job'), SkillModifier(), [True, True])

    db = DamageBuilder(self.__skill_library)
    result = db.get_damage_instances(rb_result)

    expected = ((3440, self.__skill_library.get_skill('test_gcd_with_denylist', 'test_job'), SkillModifier(), (StatusEffects(crit_rate_add=0.05), StatusEffects(damage_mult=1.3))),)
    return self._compare_sequential(result, expected)

  @TestClass.is_a_test
  def buff_and_debuff_snapshotting(self):
    rb_result = SnapshotAndApplicationEvents()
    rb_result.add(0, 0, 0, self.__skill_library.get_skill('test_simple_buff_gcd', 'test_job'), SkillModifier(), [True, True])
    rb_result.add(0, 0, 0, self.__skill_library.get_skill('test_simple_debuff_gcd_2', 'test_job'), SkillModifier(), [True, True])
    rb_result.add(Utils.transform_time_to_prio(2540), 2540, 3040, self.__skill_library.get_skill('test_gcd', 'test_job'), SkillModifier(guaranteed_dh=ForcedCritOrDH.FORCE_YES), [True, True])
    rb_result.add(Utils.transform_time_to_prio(2940), 2940, 3440, self.__skill_library.get_skill('test_dot_gcd', 'test_job'), SkillModifier(), [True, True])
    rb_result.add(Utils.transform_time_to_prio(2940)+1, 2940, 3440, self.__skill_library.get_skill('test_dot_tick', 'test_job'), SkillModifier(), [True, True])
    rb_result.add(Utils.transform_time_to_prio(2940)+2, 2940, 6440, self.__skill_library.get_skill('test_dot_tick', 'test_job'), SkillModifier(), [True, True])
    rb_result.add(Utils.transform_time_to_prio(2940)+3, 2940, 9440, self.__skill_library.get_skill('test_dot_tick', 'test_job'), SkillModifier(), [True, True])
    rb_result.add(Utils.transform_time_to_prio(2940)+4, 2940, 12440, self.__skill_library.get_skill('test_dot_tick', 'test_job'), SkillModifier(), [True, True])
    rb_result.add(Utils.transform_time_to_prio(2940)+5, 2940, 15440, self.__skill_library.get_skill('test_dot_tick', 'test_job'), SkillModifier(), [True, True])
    rb_result.add(Utils.transform_time_to_prio(12940), 12940, 13440, self.__skill_library.get_skill('test_gcd', 'test_job'), SkillModifier(), [True, True])

    db = DamageBuilder(self.__skill_library)
    result = db.get_damage_instances(rb_result)

    expected = [(3040, self.__skill_library.get_skill('test_gcd', 'test_job'), SkillModifier(guaranteed_dh=ForcedCritOrDH.FORCE_YES), (StatusEffects(crit_rate_add=0.05), StatusEffects(damage_mult=1.3))),
                (3440, self.__skill_library.get_skill('test_dot_tick', 'test_job'), SkillModifier(), (StatusEffects(crit_rate_add=0.05), StatusEffects(damage_mult=1.3))),
                (6440, self.__skill_library.get_skill('test_dot_tick', 'test_job'), SkillModifier(), (StatusEffects(crit_rate_add=0.05), StatusEffects(damage_mult=1.3))),
                (9440, self.__skill_library.get_skill('test_dot_tick', 'test_job'), SkillModifier(), (StatusEffects(crit_rate_add=0.05), StatusEffects(damage_mult=1.3))),
                (12440, self.__skill_library.get_skill('test_dot_tick', 'test_job'), SkillModifier(), (StatusEffects(crit_rate_add=0.05), StatusEffects(damage_mult=1.3))),
                (13440, self.__skill_library.get_skill('test_gcd', 'test_job'), SkillModifier(), (StatusEffects(crit_rate_add=0.05), StatusEffects(damage_mult=1.0))),
                (15440, self.__skill_library.get_skill('test_dot_tick', 'test_job'), SkillModifier(), (StatusEffects(crit_rate_add=0.05), StatusEffects(damage_mult=1.3)))]
    return self._compare_sequential(result, expected)

  @TestClass.is_a_test
  def buff_and_debuff_snapshotting_with_no_debuff_snapshot(self):
    rb_result = SnapshotAndApplicationEvents()
    rb_result.add(0, 0, 0, self.__skill_library.get_skill('test_simple_buff_gcd', 'test_job'), SkillModifier(), [True, True])
    rb_result.add(0, 0, 0, self.__skill_library.get_skill('test_simple_debuff_gcd_2', 'test_job'), SkillModifier(), [True, True])
    rb_result.add(Utils.transform_time_to_prio(1000), 1000, 1500, self.__skill_library.get_skill('test_dot_tick', 'test_job'), SkillModifier(), [True, False])
    rb_result.add(Utils.transform_time_to_prio(1000), 1000, 11000, self.__skill_library.get_skill('test_dot_tick', 'test_job'), SkillModifier(), [True, False])

    db = DamageBuilder(self.__skill_library)
    result = db.get_damage_instances(rb_result)
    expected = [(1500, self.__skill_library.get_skill('test_dot_tick', 'test_job'), SkillModifier(), (StatusEffects(crit_rate_add=0.05), StatusEffects(damage_mult=1.3))),
                (11000, self.__skill_library.get_skill('test_dot_tick', 'test_job'), SkillModifier(), (StatusEffects(crit_rate_add=0.05), StatusEffects(damage_mult=1.0)))]
    return self._compare_sequential(result, expected)

  @TestClass.is_a_test
  def priority_test(self):
    rb_result = SnapshotAndApplicationEvents()
    rb_result.add(0, 0, 500, self.__skill_library.get_skill('test_gcd', 'test_job'), SkillModifier(), [True, True])
    rb_result.add(1, 0, 0, self.__skill_library.get_skill('test_simple_buff_gcd', 'test_job'), SkillModifier(), [True, True])

    db = DamageBuilder(self.__skill_library)
    result = db.get_damage_instances(rb_result)
    expected = [(500, self.__skill_library.get_skill('test_gcd', 'test_job'), SkillModifier(), (StatusEffects(), StatusEffects())),]
    return self._compare_sequential(result, expected)

  @TestClass.is_a_test
  def default_buff_and_damage_order(self):
    rb_result = SnapshotAndApplicationEvents()
    rb_result.add(0, 0, 0, self.__skill_library.get_skill('test_default_buff_damage_order', 'test_job'), SkillModifier(), [True, True])

    db = DamageBuilder(self.__skill_library)
    result = db.get_damage_instances(rb_result)
    expected = [(0, self.__skill_library.get_skill('test_default_buff_damage_order', 'test_job'), SkillModifier(), (StatusEffects(), StatusEffects())),]
    return self._compare_sequential(result, expected)

In [69]:
#@title RotationBuilderAndDamageBuilderIntegrationTest
class RotationBuilderAndDamageBuilderIntegrationTest(TestClass):
  def __init__(self):
    self.__stats = Stats(wd=126, weapon_delay=3.44, main_stat=2945, det_stat=1620, crit_stat=2377, dh_stat=1048, speed_stat=708, job_class = 'test_job')
    self.__skill_library = create_test_skill_library()

  @TestClass.is_a_test
  def skill_modifier_pass_through(self):
    rb = RotationBuilder(self.__stats, self.__skill_library)
    rb.add(0.6, 'test_gcd', SkillModifier(guaranteed_crit=ForcedCritOrDH.FORCE_YES))

    db = DamageBuilder(self.__skill_library)
    result = db.get_damage_instances(rb.get_skill_timing())
    expected = [(3040, self.__skill_library.get_skill('test_gcd', 'test_job'), SkillModifier(guaranteed_crit=ForcedCritOrDH.FORCE_YES), (StatusEffects(), StatusEffects())),]
    return self._compare_sequential(result, expected)

  @TestClass.is_a_test
  def test_buff_then_damage(self):
    rb = RotationBuilder(self.__stats, self.__skill_library)
    rb.add(0, 'test_buff_then_damage')

    db = DamageBuilder(self.__skill_library)
    result = db.get_damage_instances(rb.get_skill_timing())
    expected = [(0, self.__skill_library.get_skill('_test_buff_then_damage', 'test_job'), SkillModifier(), (StatusEffects(crit_rate_add=0.05), StatusEffects())),]
    return self._compare_sequential(result, expected)

  @TestClass.is_a_test
  def buff_and_debuff_snapshotting(self):
    rb = RotationBuilder(self.__stats, self.__skill_library)
    rb.add(0.0, 'test_simple_buff_gcd')
    rb.add(0.0, 'test_simple_debuff_gcd_2')
    rb.add(0.6, 'test_gcd')
    rb.add(1.0, 'test_dot_gcd')
    rb.add(11.0, 'test_gcd')

    db = DamageBuilder(self.__skill_library)
    result = db.get_damage_instances(rb.get_skill_timing())

    expected = [(3040, self.__skill_library.get_skill('test_gcd', 'test_job'), SkillModifier(), (StatusEffects(crit_rate_add=0.05), StatusEffects(damage_mult=1.3))),
                (3440, self.__skill_library.get_skill('test_dot_tick', 'test_job'), SkillModifier(), (StatusEffects(crit_rate_add=0.05), StatusEffects(damage_mult=1.3))),
                (6440, self.__skill_library.get_skill('test_dot_tick', 'test_job'), SkillModifier(), (StatusEffects(crit_rate_add=0.05), StatusEffects(damage_mult=1.3))),
                (9440, self.__skill_library.get_skill('test_dot_tick', 'test_job'), SkillModifier(), (StatusEffects(crit_rate_add=0.05), StatusEffects(damage_mult=1.3))),
                (12440, self.__skill_library.get_skill('test_dot_tick', 'test_job'), SkillModifier(), (StatusEffects(crit_rate_add=0.05), StatusEffects(damage_mult=1.3))),
                (13440, self.__skill_library.get_skill('test_gcd', 'test_job'), SkillModifier(), (StatusEffects(crit_rate_add=0.05), StatusEffects(damage_mult=1.0))),
                (15440, self.__skill_library.get_skill('test_dot_tick', 'test_job'), SkillModifier(), (StatusEffects(crit_rate_add=0.05), StatusEffects(damage_mult=1.3)))]
    return self._compare_sequential(result, expected)

In [70]:
#@title StatusEffectTrackerTest
class StatusEffectTrackerTest(TestClass):
  def __init__(self):
    self.__skill_library = create_test_skill_library()

  @TestClass.is_a_test
  def simple_buff_expiry(self):
    test_passed = True
    err_msg = ""
    se = StatusEffectTracker()
    se.add_to_status_effects(1000, self.__skill_library.get_skill('test_simple_buff_gcd', 'test_job'))
    #test_simple_buff_gcd_2 should expire after 18000 ms
    se.add_to_status_effects(8000, self.__skill_library.get_skill('test_simple_buff_gcd_2', 'test_job'))
    #test_simple_buff_gcd_2 should expire after 19000 ms
    se.add_to_status_effects(9000, self.__skill_library.get_skill('test_simple_debuff_gcd_2', 'test_job'))

    se.expire_status_effects(19001)
    result = se.compile_status_effects()
    expected = (StatusEffects(crit_rate_add=0.05), StatusEffects())

    if (result != expected):
      test_passed = False
      err_msg = "Expected and actual status effects do not match.\nExpected{}.\nActual:{}".format(expected, result)
    return test_passed, err_msg

  @TestClass.is_a_test
  def multiple_auto_attack_buffs(self):
    test_passed = True
    err_msg = ""
    se = StatusEffectTracker()
    se.add_to_status_effects(100, self.__skill_library.get_skill('test_auto_attack_buff', 'test_job'))
    se.add_to_status_effects(100, self.__skill_library.get_skill('test_auto_attack_buff2', 'test_job'))

    result = se.compile_status_effects()
    expected = (StatusEffects(auto_attack_delay_mult=math.pow(0.75,2)), StatusEffects())

    if (result != expected):
      test_passed = False
      err_msg = "Expected and actual status effects do not match.\nExpected{}.\nActual:{}".format(expected, result)
    return test_passed, err_msg

  @TestClass.is_a_test
  def simple_buffs(self):
    test_passed = True
    err_msg = ""
    se = StatusEffectTracker()
    se.add_to_status_effects(100, self.__skill_library.get_skill('test_simple_buff_gcd', 'test_job'))
    se.add_to_status_effects(100, self.__skill_library.get_skill('test_simple_buff_gcd_2', 'test_job'))
    se.add_to_status_effects(100, self.__skill_library.get_skill('test_simple_debuff_gcd', 'test_job'))
    se.add_to_status_effects(100, self.__skill_library.get_skill('test_simple_debuff_gcd_2', 'test_job'))

    result = se.compile_status_effects()
    expected = (StatusEffects(crit_rate_add=0.11, dh_rate_add=0.2), StatusEffects(damage_mult=1.56))

    if (result != expected):
      test_passed = False
      err_msg = "Expected and actual status effects do not match.\nExpected{}.\nActual:{}".format(expected, result)
    return test_passed, err_msg

  @TestClass.is_a_test
  def test_buff_override(self):
    test_passed = True
    err_msg = ""
    se = StatusEffectTracker()
    se.add_to_status_effects(100, self.__skill_library.get_skill('test_simple_buff_gcd', 'test_job'))
    # this should simply override the first buff
    se.add_to_status_effects(500, self.__skill_library.get_skill('test_simple_buff_gcd', 'test_job'))

    result = se.compile_status_effects()
    expected = (StatusEffects(crit_rate_add=0.05, dh_rate_add=0.0), StatusEffects())

    if (result != expected):
      test_passed = False
      err_msg = "Expected and actual status effects do not match.\nExpected{}.\nActual:{}".format(expected, result)
    return test_passed, err_msg

  @TestClass.is_a_test
  def late_refresh(self):
    test_passed = True
    err_msg = ""
    se = StatusEffectTracker()
    se.add_to_status_effects(0, self.__skill_library.get_skill('test_simple_buff_gcd', 'test_job'))
    #test_simple_buff_gcd should expire after 30000 ms
    se.add_to_status_effects(31000, self.__skill_library.get_skill('test_simple_buff_gcd', 'test_job'))
    se.expire_status_effects(61000-1)
    result = se.compile_status_effects()
    expected = (StatusEffects(crit_rate_add=0.05), StatusEffects())

    if (result != expected):
      test_passed = False
      err_msg = "Expected and actual status effects do not match.\nExpected{}.\nActual:{}".format(expected, result)
    return test_passed, err_msg

In [71]:
#@title StatusEffectSpecTest
class StatusEffectSpecTest(TestClass):
  @TestClass.is_a_test
  def test_max_duration(self):
    test_passed = True
    err_msg = ""

    duration = 10000
    status_effect_spec = StatusEffectSpec(duration=duration)
    return test_passed, err_msg
    if (status_effect_spec.max_duration != duration):
      test_passed = False
      err_msg = "Max duration set incorrectly. Expected{} but got {}".format(duration, status_effect_spec.max_duration)

    return test_passed, err_msg

  @TestClass.is_a_test
  def test_set_max_duration(self):
    test_passed = True
    err_msg = ""

    duration = 10000
    max_duration = 30000
    status_effect_spec = StatusEffectSpec(duration=duration, max_duration = max_duration)
    return test_passed, err_msg
    if (status_effect_spec.max_duration != duration):
      test_passed = False
      err_msg = "Max duration set incorrectly. Expected{} but got {}".format(max_duration, status_effect_spec.max_duration)

    return test_passed, err_msg

In [72]:
#@title StatFnsTest
#TODO: fdetdh test
class StatFnsTest(TestClass):
  @TestClass.is_a_test
  def get_time_using_speed_stat_test(self):
    test_passed = True
    err_msg=""

    gcd_inputs_and_expcted= {2500: ((1408, 1409, 1410), (2330, 2320, 2320)),
                             3500: ((999, 1000, 1001), (3360, 3350, 3350))}

    for gcd_time, inputs_and_outputs in gcd_inputs_and_expcted.items():
      speed_stat = inputs_and_outputs[0]
      expected = inputs_and_outputs[1]
      for i in range(0, len(speed_stat)):
          gcd_actual = StatFns.get_time_using_speed_stat(gcd_time, speed_stat[i])
          gcd_expected = expected[i]
          if gcd_actual != gcd_expected:
            err_msg += "gcd_time expected: {}. Actual: {}. For {} gcd.\n".format(gcd_expected, gcd_actual, gcd_time)
            test_passed = False

    return test_passed, err_msg

  @TestClass.is_a_test
  def get_crit_stats_test(self):
    test_passed = True
    err_msg=""

    crit_stat_inputs_and_expected = {400: (0.05, 0.40),
                                     475: (0.057, 0.407),
                                     476: (0.058, 0.408),
                                     1150: (0.128, 0.478),
                                     1151: (0.129, 0.479),
                                     1152: (0.129, 0.479)}

    for crit_stat, expected in crit_stat_inputs_and_expected.items():
      #TODO: consider using math.isclose isntead for float comparison fiddliness.
      crit_rate_actual, crit_bonus_actual = StatFns.get_crit_stats(crit_stat)
      if (crit_rate_actual, crit_bonus_actual) != expected:
        err_msg += "crit stat: {}. crit rate/bonus expected: {}/{}. Actual: {}/{}.\n".format(crit_stat, expected[0], expected[1], crit_rate_actual, crit_bonus_actual)
        test_passed = False

    return test_passed, err_msg

  @TestClass.is_a_test
  def fDetDH_test(self):
    test_passed = True
    err_msg=""

    #(det_stat, dh_stat, fDetDh_expected)
    stat_inputs_and_expected = ((390, 400, 1000+0),
                                (1706, 1360, 1096+70),
                                (1707, 1369, 1097+71))

    for (det_stat, dh_stat, expected) in stat_inputs_and_expected:
      fDetDH_actual = StatFns.fDetDH(det_stat, dh_stat)
      if fDetDH_actual != expected:
        err_msg += "fDetDh expected: {}. Actual: {}.\n".format(expected, fDetDH_actual)
        test_passed = False

    return test_passed, err_msg

  @TestClass.is_a_test
  def fDet_test(self):
    test_passed = True
    err_msg=""

    det_stat_inputs_and_expected = {390: 1000,
                                    566: 1012,
                                    567: 1013,
                                    1706: 1096,
                                    1707: 1097,
                                    1708: 1097}

    for det_stat, expected in det_stat_inputs_and_expected.items():
      fDet_actual = StatFns.fDet(det_stat)
      if fDet_actual != expected:
        err_msg += "fDet expected: {}. Actual: {}.\n".format(expected, fDet_actual)
        test_passed = False

    return test_passed, err_msg

  @TestClass.is_a_test
  def dh_rate_test(self):
    test_passed = True
    err_msg=""

    dh_stat_inputs_and_expected = {400: 0,
                                   472: 0.020,
                                   473: 0.021,
                                   1360: 0.277,
                                   1361: 0.278,
                                   1362: 0.278}

    for dh_stat, expected in dh_stat_inputs_and_expected.items():
      dh_rate_actual = StatFns.get_dh_rate(dh_stat)
      if not math.isclose(dh_rate_actual, expected, abs_tol=1e-4):
        err_msg += "dh_rate expected: {}. Actual: {}.\n".format(expected, dh_rate_actual)
        test_passed = False

    return test_passed, err_msg

  @TestClass.is_a_test
  def wd_test(self):
    test_passed = True
    err_msg=""

    #(wd, job_mod, fWd_expected)
    wd_stat_inputs_and_expected = ((120, 100, 159),
                                   (126, 115, 170))

    for (wd, job_mod, expected) in wd_stat_inputs_and_expected:
      fWD_actual = StatFns.fWD(wd, job_mod)
      if fWD_actual != expected:
        err_msg += "fWd expected: {}. Actual: {}.\n".format(expected, fWD_actual)
        test_passed = False

    return test_passed, err_msg

  @TestClass.is_a_test
  def fAuto_test(self):
    test_passed = True
    err_msg=""

    stat_inputs_and_expected = ((120, 3.44, 105, 183),
                                (126, 2.6, 115, 147))

    for (wd, weapon_delay, job_mod, expected) in stat_inputs_and_expected:
      fAuto_actual = StatFns.fAuto(wd, weapon_delay, job_mod)
      if fAuto_actual != expected:
        err_msg += "fAuto expected: {}. Actual: {}.\n".format(expected, fAuto_actual)
        test_passed = False

    return test_passed, err_msg

  @TestClass.is_a_test
  def spd_test(self):
    test_passed = True
    err_msg=""

    #(spd, fSpd_expected)
    spd_stat_inputs_and_expected = ((859, 1031),
                                    (1700,  1088))

    for (spd, expected) in spd_stat_inputs_and_expected:
      fSpd_actual_actual = StatFns.fSpd(spd)
      if fSpd_actual_actual != expected:
        err_msg += "fSpd expected: {}. Actual: {}.\n".format(expected, fSpd_actual_actual)
        test_passed = False

    return test_passed, err_msg

  @TestClass.is_a_test
  def fTnc_test(self):
    test_passed = True
    err_msg=""

    tnc_inputs_and_expected = {751: 1018,
                               1100: 1036,
                               500: 1005}

    for tnc, expected in tnc_inputs_and_expected.items():
      fTnc_actual = StatFns.fTnc(tnc)
      if fTnc_actual != expected:
        err_msg += "fTnc expected: {}. Actual: {}.\n".format(expected, fTnc_actual)
        test_passed = False

    return test_passed, err_msg

  @TestClass.is_a_test
  def fAP_tank_test(self):
    test_passed = True
    err_msg=""

    ap_inputs_and_expected = {390: 100,
                              699: 223,
                              1219: 431}

    for ap, expected in ap_inputs_and_expected.items():
      fAP_actual = StatFns.fAP(ap, is_tank = True)
      if fAP_actual != expected:
        err_msg += "fAP expected: {}. Actual: {}.\n".format(expected, fAP_actual)
        test_passed = False

    return test_passed, err_msg

  @TestClass.is_a_test
  def fAP_non_tank_test(self):
    test_passed = True
    err_msg=""

    ap_inputs_and_expected = {390: 100,
                              699: 254,
                              1219: 514}

    for ap, expected in ap_inputs_and_expected.items():
      fAP_actual = StatFns.fAP(ap, is_tank = False)
      if fAP_actual != expected:
        err_msg += "fAP expected: {}. Actual: {}.\n".format(expected, fAP_actual)
        test_passed = False

    return test_passed, err_msg

In [73]:
#@title EndToEndTest
class EndToEndTest(TestClass):
  def __init__(self):
    self.__skill_library = create_test_skill_library()

  @TestClass.is_a_test
  def test_simple(self):
    test_passed = True
    err_msg=""
    stats = Stats(wd=126, weapon_delay=3.44, main_stat=2945, det_stat=1620, crit_stat=2377, dh_stat=1048, speed_stat=708, job_class = 'test_job')

    rb = RotationBuilder(stats, self.__skill_library)
    db = DamageBuilder(self.__skill_library)

    rb.add(1.0, 'test_dot_gcd')
    dmg_instances = db.get_damage_instances(rb.get_skill_timing())
    # just make sure it actually runs and doesn't die
    sim = DamageSimulator(stats, dmg_instances, 2, job_class_fns=JobClassFnsTest)

    return test_passed, err_msg

In [74]:
#@title JobsTest
# The purpose of these tests is not to test that we have the truly correct
# DPS numbers, but to check that nothing has inadvertently changed.

class JobsTest(TestClass):
  def __init__(self):
    self.__skill_library = create_skill_library()
    self.__relative_tol = 5e-3

  def __test_skills(self, stats, skills_and_expected_damage):
    test_passed=True
    err_msg=""

    for sk, expected_damage in skills_and_expected_damage.items():
      rb = RotationBuilder(stats, self.__skill_library)
      rb.add_next(sk)

      db = DamageBuilder(self.__skill_library)
      sim = DamageSimulator(rb.get_stats(), db.get_damage_instances(rb.get_skill_timing()), 100000)
      dps = sim.get_dps()
      actual_damage = np.mean(sim.get_raw_damage())
      diff = abs(float(expected_damage - actual_damage))
      if diff/expected_damage >= self.__relative_tol:
        test_passed = False
        err_msg += "Did not get expected damage for {}. Expected: {}. Actual: {}.\n".format(sk, expected_damage, actual_damage)
    return test_passed, err_msg

  def __test_rotation(self, stats, rb, expected_damage, expected_total_time):
    test_passed=True
    err_msg=""

    db = DamageBuilder(self.__skill_library)
    sim = DamageSimulator(rb.get_stats(), db.get_damage_instances(rb.get_skill_timing()), 100000)
    dps = sim.get_dps()
    actual_damage = np.mean(sim.get_raw_damage())
    damage_diff = abs(float(expected_damage - actual_damage))
    if damage_diff/expected_damage >= self.__relative_tol:
      test_passed = False
      err_msg += "Did not get expected damage for rotation. Expected: {}. Actual: {}.\n".format(expected_damage, actual_damage)

    actual_total_time = max(sim.get_damage_time())
    if abs(expected_total_time - actual_total_time) > 1e-3:
      test_passed = False
      err_msg += "Did not get expected total time for rotation. Expected: {}. Actual: {}.\n".format(expected_total_time, actual_total_time)

    return test_passed, err_msg

  @TestClass.is_a_test
  def test_war_skills(self):
    stats = Stats(wd=126, weapon_delay=3.36, main_stat=2910, det_stat=1980, crit_stat=2313, dh_stat=868, speed_stat=592, tenacity=631, job_class = 'WAR')
    skills_and_expected_damage = {'Auto': 2450.2,
                                  'Heavy Swing': 5130.7,
                                  'Maim': 7701.5,
                                  "Storm's Path": 11259.5,
                                  "Storm's Eye": 11255.7,
                                  'Upheaval': 10270.0,
                                  'Onslaught': 3751.8,
                                  'Fell Cleave': 13430.3,
                                  'Primal Rend': 18168.5,
                                  'Inner Chaos': 29564.9,
                                  'Tomahawk': 3751.5,
                                  'Overpower': 2764.0,
                                  'Mythril Tempest': 3754.5,
                                  'Orogeny': 3749.9,
                                  'Decimate': 5133.2}
    return self.__test_skills(stats, skills_and_expected_damage)

  @TestClass.is_a_test
  def test_war_rotation(self):
    stats = Stats(wd=126, weapon_delay=3.36, main_stat=2910, det_stat=1980, crit_stat=2313, dh_stat=868, speed_stat=592, tenacity=631, job_class = 'WAR')

    rb = RotationBuilder(stats, self.__skill_library)
    rb.add_next('Heavy Swing')
    rb.add_next('Maim')
    rb.add_next('Grade 8 Tincture', job_class='GENERAL')
    rb.add_next("Storm's Eye")
    rb.add_next('Inner Chaos')
    rb.add_next('Upheaval')
    rb.add_next('Onslaught')
    rb.add_next('Primal Rend')
    rb.add_next('Inner Chaos')
    rb.add_next('Onslaught')
    rb.add_next('Fell Cleave', SkillModifier(guaranteed_crit=ForcedCritOrDH.FORCE_YES, guaranteed_dh=ForcedCritOrDH.FORCE_YES))
    rb.add_next('Onslaught')
    rb.add_next('Fell Cleave', SkillModifier(guaranteed_crit=ForcedCritOrDH.FORCE_YES, guaranteed_dh=ForcedCritOrDH.FORCE_YES))
    rb.add_next('Fell Cleave', SkillModifier(guaranteed_crit=ForcedCritOrDH.FORCE_YES, guaranteed_dh=ForcedCritOrDH.FORCE_YES))
    rb.add_next('Heavy Swing')
    rb.add_next('Maim')
    rb.add_next("Storm's Path")
    rb.add_next('Fell Cleave')
    rb.add_next('Inner Chaos')
    expected_damage = 310081.8
    expected_total_time = 31980.0

    return self.__test_rotation(stats, rb, expected_damage, expected_total_time)

  @TestClass.is_a_test
  def test_gnb_skills(self):
    stats = Stats(wd=126, weapon_delay=2.80, main_stat=2891, det_stat=1844, crit_stat=2377, dh_stat=1012, speed_stat=400, tenacity=751, job_class = 'GNB')
    skills_and_expected_damage = {'Auto': 2031.8,
                                  'Keen Edge': 5175.9,
                                  'Brutal Shell': 7773.0,
                                  'Demon Slice': 2594.8,
                                  'Lightning Shot': 3782.5,
                                  'Solid Barrel': 9166.3,
                                  'Burst Strike': 9765.6,
                                  'Demon Slaughter': 3988.4,
                                  'Sonic Break': 21729.7,
                                  'Rough Divide': 3789.3,
                                  'Gnashing Fang': 9762.1,
                                  'Savage Claw': 11950.9,
                                  'Wicked Talon': 13957.8,
                                  'Bow Shock': 10767.7,
                                  'Jugular Rip': 5185.7,
                                  'Abdomen Tear': 6185.1,
                                  'Eye Gouge': 7177.4,
                                  'Fated Circle': 7781.5,
                                  'Blasting Zone': 18543.0,
                                  'Double Down': 31124.2,
                                  'Hypervelocity': 4580.7}
    return self.__test_skills(stats, skills_and_expected_damage)

  @TestClass.is_a_test
  def test_gnb_rotation(self):
    stats = Stats(wd=126, weapon_delay=2.80, main_stat=2891, det_stat=1844, crit_stat=2377, dh_stat=1012, speed_stat=400, tenacity=751, job_class = 'GNB')

    rb = RotationBuilder(stats, self.__skill_library)
    rb.add_next('Keen Edge')
    rb.add_next('Grade 8 Tincture', job_class='GENERAL')
    rb.add_next('Brutal Shell')
    rb.add_next('No Mercy')
    rb.add_next('Gnashing Fang')
    rb.add_next('Jugular Rip')
    rb.add_next('Sonic Break')
    rb.add_next('Blasting Zone')
    rb.add_next('Bow Shock')
    rb.add_next('Double Down')
    rb.add_next('Rough Divide')
    rb.add_next('Savage Claw')
    rb.add_next('Abdomen Tear')
    rb.add_next('Rough Divide')
    rb.add_next('Wicked Talon')
    rb.add_next('Eye Gouge')
    rb.add_next('Solid Barrel')
    rb.add_next('Burst Strike')
    rb.add_next('Hypervelocity')
    rb.add_next('Keen Edge')
    expected_damage = 241489.1
    expected_total_time = 34500.0

    return self.__test_rotation(stats, rb, expected_damage, expected_total_time)

  @TestClass.is_a_test
  def test_pld_skills(self):
    stats = Stats(wd=126, weapon_delay=2.24, main_stat=2891, det_stat=1844, crit_stat=2377, dh_stat=1012, speed_stat=400, tenacity=751, job_class = 'PLD')
    skills_and_expected_damage = {'Auto': 1635.9,
                                  'Fast Blade': 5176.2,
                                  'Riot Blade': 7792.7,
                                  'Total Eclipse': 2589.3,
                                  'Shield Bash': 2592.1,
                                  'Shield Lob': 2588.7,
                                  'Prominence': 4385.4,
                                  'Circle of Scorn': 6582.1,
                                  'Goring Blade': 18159.6,
                                  'Royal Authority': 10370.0,
                                  'Holy Spirit': 8976.4,
                                  'Holy Spirit (Divine Might)': 11556.7,
                                  'Holy Spirit (Requiescat)': 16721.4,
                                  'Requiescat': 8168.9,
                                  'Holy Circle': 2592.8,
                                  'Holy Circle (Divine Might)': 5174.1,
                                  'Holy Circle (Requiescat)': 7773.7,
                                  'Intervene': 3783.2,
                                  'Atonement': 10376.6,
                                  'Confiteor': 10767.4,
                                  'Explacion': 11557.3,
                                  'Blade of Faith': 5580.8,
                                  'Blade of Faith (Requiescat)': 18536.6,
                                  'Blade of Truth': 8170.1,
                                  'Blade of Truth (Requiescat)': 21129.6,
                                  'Blade of Valor': 10759.5,
                                  'Blade of Valor (Requiescat)': 23948.3}
    return self.__test_skills(stats, skills_and_expected_damage)

  @TestClass.is_a_test
  def test_pld_rotation(self):
    stats = Stats(wd=126, weapon_delay=2.24, main_stat=2891, det_stat=1844, crit_stat=2377, dh_stat=1012, speed_stat=400, tenacity=751, job_class = 'PLD')

    rb = RotationBuilder(stats, self.__skill_library)
    rb.add_next('Holy Spirit')
    rb.add_next('Fast Blade')
    rb.add_next('Grade 8 Tincture', job_class='GENERAL')
    rb.add_next('Riot Blade')
    rb.add_next('Royal Authority')
    rb.add_next('Fight or Flight')
    rb.add_next('Requiescat')
    rb.add_next('Goring Blade')
    rb.add_next('Circle of Scorn')
    rb.add_next('Explacion')
    rb.add_next('Confiteor (Requiescat)')
    rb.add_next('Intervene')
    rb.add_next('Blade of Faith (Requiescat)')
    rb.add_next('Intervene')
    rb.add_next('Blade of Truth (Requiescat)')
    rb.add_next('Blade of Valor (Requiescat)')
    rb.add_next('Holy Spirit (Divine Might)')
    rb.add_next('Atonement')
    rb.add_next('Atonement')
    rb.add_next('Atonement')
    expected_damage = 281593.8
    expected_total_time = 28500.0

    return self.__test_rotation(stats, rb, expected_damage, expected_total_time)


  @TestClass.is_a_test
  def test_drk_skills(self):
    stats = Stats(wd=126, weapon_delay=2.96, main_stat=2906, det_stat=1883, crit_stat=2352, dh_stat=868, speed_stat=650, tenacity=631, job_class = 'DRK')
    skills_and_expected_damage = {'Auto': 2175.1,
                                  'Hard Slash': 4350.8,
                                  'Syphon Strike': 6532.2,
                                  'Unleash': 2969.1,
                                  'Unmend': 3759.2,
                                  'Souleater': 8714.3,
                                  'Flood of Shadow': 3958.1,
                                  'Stalwart Soul': 3562.0,
                                  'Edge of Shadow': 11889.3,
                                  'Salted Earth': 6041.9,
                                  'Salt and Darkness': 12885.7,
                                  'Plunge': 3765.7,
                                  'Abyssal Drain': 6139.5,
                                  'Carve and Spit': 13066.5,
                                  'Bloodspiller': 12880.7,
                                  'Quietus': 5154.5,
                                  'Shadowbringer': 15454.4,
                                  'Living Shadow': 67148.1}
    return self.__test_skills(stats, skills_and_expected_damage)

  @TestClass.is_a_test
  def test_drk_rotation(self):
    stats = Stats(wd=126, weapon_delay=2.96, main_stat=2906, det_stat=1883, crit_stat=2352, dh_stat=868, speed_stat=650, tenacity=631, job_class = 'DRK')
    rb = RotationBuilder(stats, self.__skill_library)

    rb.add_next('Hard Slash')
    rb.add_next('Edge of Shadow')
    rb.add_next('Syphon Strike')
    rb.add_next('Grade 8 Tincture', job_class='GENERAL')
    rb.add_next('Souleater')
    rb.add_next('Living Shadow')
    rb.add_next('Salted Earth')
    rb.add_next('Bloodspiller')
    rb.add_next('Shadowbringer')
    rb.add_next('Edge of Shadow')
    rb.add_next('Bloodspiller')
    rb.add_next('Carve and Spit')
    rb.add_next('Plunge')
    rb.add_next('Bloodspiller')
    rb.add_next('Edge of Shadow')
    rb.add_next('Salt and Darkness')
    rb.add_next('Hard Slash')
    rb.add_next('Plunge')
    rb.add_next('Edge of Shadow')
    rb.add_next('Syphon Strike')
    rb.add_next('Shadowbringer')
    rb.add_next('Edge of Shadow')
    rb.add_next('Souleater')
    rb.add_next('Hard Slash')
    rb.add_next('Syphon Strike')
    rb.add_next('Souleater')
    expected_damage = 342915.1
    expected_total_time = 26950.0

    return self.__test_rotation(stats, rb, expected_damage, expected_total_time)

  @TestClass.is_a_test
  def test_sch_skills(self):
    stats = Stats(wd=126, weapon_delay=3.12, main_stat=3366, det_stat=1948, crit_stat=2498, dh_stat=688, speed_stat=954, job_class = 'SCH', healer_or_caster_strength=351)
    skills_and_expected_damage = {'Auto': 152.8,
                                  'Broil IV': 14476.0,
                                  'Ruin II': 10793.4,
                                  'Energy Drain': 4901.0,
                                  'Art of War II': 8830.6,
                                  'Biolysis': 35583.0}
    return self.__test_skills(stats, skills_and_expected_damage)

  @TestClass.is_a_test
  def test_sch_rotation(self):
    stats = Stats(wd=126, weapon_delay=3.12, main_stat=3366, det_stat=1948, crit_stat=2498, dh_stat=688, speed_stat=954, job_class = 'SCH', healer_or_caster_strength=351)

    rb = RotationBuilder(stats, self.__skill_library)
    rb.add_next('Grade 8 Tincture', job_class='GENERAL')
    rb.add_next('Broil IV')
    rb.add_next('Biolysis')
    rb.add_next('Broil IV')
    rb.add_next('Broil IV')
    rb.add_next('Chain Strategem')
    rb.add_next('Energy Drain')
    rb.add_next('Broil IV')
    rb.add_next('Energy Drain')
    rb.add_next('Broil IV')
    rb.add_next('Energy Drain')
    rb.add_next('Broil IV')
    rb.add_next('Energy Drain')
    rb.add_next('Broil IV')
    rb.add_next('Energy Drain')
    rb.add_next('Broil IV')
    rb.add_next('Energy Drain')
    rb.add_next('Broil IV')
    rb.add_next('Broil IV')
    rb.add_next('Broil IV')
    rb.add_next('Broil IV')
    rb.add_next('Broil IV')
    rb.add_next('Broil IV')
    rb.add_next('Broil IV')
    expected_damage = 307031.4
    expected_total_time = 36440.0

    return self.__test_rotation(stats, rb, expected_damage, expected_total_time)
  @TestClass.is_a_test
  def test_whm_skills(self):
    stats = Stats(wd=132, weapon_delay=3.44, main_stat=3366, det_stat=2047, crit_stat=2502, dh_stat=616, speed_stat=1062, job_class = 'WHM', healer_or_caster_strength=214)
    skills_and_expected_damage = {'Auto': 23.0,
                                  'Glare III': 15776.6,
                                  'Assize': 20346.5,
                                  'Dia': 37846.1,
                                  'Afflatus Misery': 63011.8,
                                  'Holy III': 63187.1}
    return self.__test_skills(stats, skills_and_expected_damage)

  @TestClass.is_a_test
  def test_whm_rotation(self):
    stats = Stats(wd=132, weapon_delay=3.44, main_stat=3366, det_stat=2047, crit_stat=2502, dh_stat=616, speed_stat=1062, job_class = 'WHM', healer_or_caster_strength=214)

    rb = RotationBuilder(stats, self.__skill_library)
    rb.add_next('Grade 8 Tincture', job_class='GENERAL')
    rb.add_next('Glare III')
    rb.add_next('Dia')
    rb.add_next('Glare III')
    rb.add_next('Glare III')
    rb.add_next('Presence of Mind')
    rb.add_next('Glare III')
    rb.add_next('Assize')
    rb.add_next('Glare III')
    rb.add_next('Glare III')
    rb.add_next('Glare III')
    rb.add_next('Glare III')
    rb.add_next('Glare III')
    rb.add_next('Glare III')
    rb.add_next('Glare III')
    rb.add_next('Glare III')
    rb.add_next('Glare III')
    rb.add_next('Glare III')
    expected_damage = 302093.5
    expected_total_time = 27950.0

    return self.__test_rotation(stats, rb, expected_damage, expected_total_time)

  @TestClass.is_a_test
  def test_sge_skills(self):
    stats = Stats(wd=132, weapon_delay=2.8, main_stat=3366, det_stat=2047, crit_stat=2502, dh_stat=1012, speed_stat=664, job_class = 'SGE', healer_or_caster_strength=214)
    skills_and_expected_damage = {'Auto': 18.7,
                                  'Dosis III': 17288.4,
                                  'Phlegma III': 31353.7,
                                  'Toxikon II': 17293.5,
                                  'Dykrasia II': 8886.4,
                                  'Pneuma': 17292.5,
                                  'Eukrasian Dosis III': 39958.0}
    return self.__test_skills(stats, skills_and_expected_damage)

  @TestClass.is_a_test
  def test_sge_rotation(self):
    stats = Stats(wd=132, weapon_delay=2.8, main_stat=3366, det_stat=2047, crit_stat=2502, dh_stat=1012, speed_stat=664, job_class = 'SGE', healer_or_caster_strength=214)

    rb = RotationBuilder(stats, self.__skill_library)
    rb.add_next('Grade 8 Tincture', job_class='GENERAL')
    rb.add_next('Dosis III')
    rb.add_next('Eukrasia')
    rb.add_next('Eukrasian Dosis III')
    rb.add_next('Dosis III')
    rb.add_next('Dosis III')
    rb.add_next('Phlegma III')
    rb.add_next('Phlegma III')
    rb.add_next('Dosis III')
    rb.add_next('Dosis III')
    rb.add_next('Dosis III')
    rb.add_next('Dosis III')
    rb.add_next('Dosis III')
    rb.add_next('Dosis III')
    rb.add_next('Dosis III')
    rb.add_next('Dosis III')
    rb.add_next('Dosis III')
    expected_damage = 329801.1
    expected_total_time = 36800.0

    return self.__test_rotation(stats, rb, expected_damage, expected_total_time)

  @TestClass.is_a_test
  def test_ast_skills(self):
    stats = Stats(wd=132, weapon_delay=3.2, main_stat=3366, det_stat=2047, crit_stat=2430, dh_stat=400, speed_stat=1350, job_class = 'AST', healer_or_caster_strength=214)
    skills_and_expected_damage = {'Auto': 21.4,
                                  'Fall Malefic': 12454.1,
                                  'Combust III': 29158.2,
                                  'Earthly Star': 14684.8,
                                  'Stellar Detonation': 14654.9,
                                  'Stellar Detonation (Small)': 9699.6,
                                  'Gravity II': 6469.3,
                                  'Macrocosmos': 12454.1,
                                  'Lord of Crowns': 12454.1}
    return self.__test_skills(stats, skills_and_expected_damage)

  @TestClass.is_a_test
  def test_ast_rotation(self):
    stats = Stats(wd=132, weapon_delay=3.2, main_stat=3366, det_stat=2047, crit_stat=2430, dh_stat=400, speed_stat=1350, job_class = 'AST', healer_or_caster_strength=214)

    rb = RotationBuilder(stats, self.__skill_library)
    rb.add_next('Earthly Star')
    rb.add_next('Grade 8 Tincture', job_class='GENERAL')
    rb.add_next('Fall Malefic')
    rb.add_next('Lightspeed')
    rb.add_next('Combust III')
    rb.add_next('Fall Malefic')
    rb.add_next('Fall Malefic')
    rb.add_next('Divination')
    rb.add_next('Astrodyne')
    rb.add_next('Lord of Crowns')
    rb.add_next('Fall Malefic')
    rb.add_next('Fall Malefic')
    rb.add_next('Fall Malefic')
    rb.add_next('Fall Malefic')
    rb.add_next('Fall Malefic')
    rb.add_next('Fall Malefic')
    rb.add_next('Fall Malefic')
    rb.add_next('Fall Malefic')
    expected_damage = 214189.5
    expected_total_time = 27930.0

    return self.__test_rotation(stats, rb, expected_damage, expected_total_time)

In [75]:
all_tests = AllTest()
all_tests.register_test_class(TimingSpecTest())
all_tests.register_test_class(StatusEffectSpecTest())
all_tests.register_test_class(StatFnsTest())
all_tests.register_test_class(RotationBuilderTest())
all_tests.register_test_class(SnapshotAndApplicationEventsTest())
all_tests.register_test_class(StatusEffectTrackerTest())
all_tests.register_test_class(RotationBuilderAndDamageBuilderIntegrationTest())
all_tests.register_test_class(DamageBuilderTest())
all_tests.register_test_class(ComputeDamageUtilsTest())
all_tests.register_test_class(DamageSimulatorTest())
all_tests.register_test_class(EndToEndTest())
all_tests.register_test_class(JobsTest())

st = time.time()
all_tests.run_all()
end = time.time()
print('Tests completed in {}s.'.format(end-st))

Running: TimingSpecTest
Running: StatusEffectSpecTest
Running: StatFnsTest
Running: RotationBuilderTest
Running: SnapshotAndApplicationEventsTest
Running: StatusEffectTrackerTest
Running: RotationBuilderAndDamageBuilderIntegrationTest
Running: DamageBuilderTest
Running: ComputeDamageUtilsTest
Running: DamageSimulatorTest
Running: EndToEndTest
Running: JobsTest
All tests done. Num passing: 104. Num failing: 0
Tests completed in 1.5471136569976807s.
